In [2]:
import uproot
import sys
import glob
import os
import struct
import time
import csv
import pickle
import math
import gc

import numpy                  as np
import pandas                 as pd
import matplotlib             as mpl
import matplotlib.pyplot      as plt
import tensorflow             as tf
from   tensorflow             import keras
import sympy

import scipy
from   scipy             import interpolate, integrate
from   scipy.interpolate import CubicSpline, splev, splrep, PPoly
from   scipy.optimize    import curve_fit, minimize

# from   scipy.stats            import kde
# from   mpl_toolkits           import mplot3d
# from   mpl_toolkits.mplot3d   import Axes3D

# sys.path.insert(0, "/home/chan/2_ROOT/0_FROST/0_FROSTpython")
# from   rbDST2dict import rbDST2dict_chan as frz
# from   rbDST2dict.rbDST2dict_chan import rbDST2dict_chan as frz
# from   rbDST2dict             import rbDST2dict_chan as frz      #rbDST2dict=dir, rbDST2dict.py=module, rbDST2dict()=class
                                                                 #you need to specify the folder -> "from rbDST2dict"

#To increase the cell width + load Images
from IPython.core.display import display, HTML, Image
# display(HTML("<style>.container { width:100% !important; }</style
#Pandas options
pd.options.display.max_columns = None
# pd.options.display.max_rows    = 100

#Plotting Config
mpl.style.use("default")
mpl.rcParams["axes.facecolor"] = "#EAEAF2" 
mpl.rcParams['figure.dpi']     = 100 
mpl.rcParams['savefig.dpi']    = 100 

# Helicity Asymmetry E - High energy

#### Scale Factor

###### Partition - Butanol and Carbon in each bins (MUST)

In [8]:
#Partition df in epho & cos bins
def part_df(df=pd.DataFrame(), epho_bins=[], cos_bins={}):
    #================================================================================================================
    #Dictionaries to data frames for each energy + angle bins
    buta_dict = {}
    carb_dict = {}
    
    #================================================================================================================
    #For each targets
    df_buta   = df[["cthe_cm", "epho", "el_pc_mmsq_pi0", "T_fin"]][(df["T_fin"] == 0) & (df["el_pc_mmsq_pi0"] > -1.5) & (df["el_pc_mmsq_pi0"] < 1.)]
    df_carb   = df[["cthe_cm", "epho", "el_pc_mmsq_pi0", "T_fin"]][(df["T_fin"] == 1) & (df["el_pc_mmsq_pi0"] > -1.5) & (df["el_pc_mmsq_pi0"] < 1.)]

    #================================================================================================================
    #For each Eg_bon
    for i, j in zip(range(len(epho_bins)-1), sorted(cos_bins.keys())):
        df_buta_Eg = df_buta[["cthe_cm", "el_pc_mmsq_pi0"]] \
                [(df_buta["epho"] > epho_bins[i]) & (df_buta["epho"] < epho_bins[i+1])]
        
        df_carb_Eg = df_carb[["cthe_cm", "el_pc_mmsq_pi0"]] \
                [(df_carb["epho"] > epho_bins[i]) & (df_carb["epho"] < epho_bins[i+1])]
        
        #List of dataframes in same epho bins
        buta_epho_list = []
        carb_epho_list = []
        
        #========================================================================================================
        #For each cos_bin
        for k in range(len(cos_bins[j])-1):
            df_buta_Eg_cos = df_buta_Eg[["el_pc_mmsq_pi0"]] \
                             [(df_buta_Eg["cthe_cm"] > cos_bins[j][k]) & (df_buta_Eg["cthe_cm"] < cos_bins[j][k+1])]
            
            
            df_carb_Eg_cos = df_carb_Eg[["el_pc_mmsq_pi0"]] \
                             [(df_carb_Eg["cthe_cm"] > cos_bins[j][k]) & (df_carb_Eg["cthe_cm"] < cos_bins[j][k+1])]
        
            #REMOVE OUTLIER
            #    -For epho_bin=1, cos_bin==3, error: multiple copies of same event
            if (i==1 and k==3):
                df_carb_Eg_cos.drop(df_carb_Eg_cos[(df_carb_Eg_cos["el_pc_mmsq_pi0"]>-0.2837) & \
                                                   (df_carb_Eg_cos["el_pc_mmsq_pi0"] < -0.28365)].index, \
                                    inplace=True)
        
            #Append to its epho list
            buta_epho_list.append(df_buta_Eg_cos)
            carb_epho_list.append(df_carb_Eg_cos)
            
        #========================================================================================================            
        #Append to buta_dict & carb_dict for each energy bins
        buta_dict["Eg_{:07.3f}".format(epho_bins[i])] = buta_epho_list
        carb_dict["Eg_{:07.3f}".format(epho_bins[i])] = carb_epho_list            
        
    return buta_dict, carb_dict, epho_bins, cos_bins    

#Apply part_df
buta_dict, carb_dict, epho_bins, cos_bins = part_df(df_GMSH, epho_bins=epho_bins, cos_bins=cos_bins)

###### Initial SF function (MUST) 

In [9]:
#Scale Factor phase 1 (MUST)
#    -Returns # of butanol and # of carbon in each cos_bins and epho_bins for mmsq regions of interest 

#=================================================================================================================
#Selecton Range for SF calculation (i.e. [peak - \sig, peak - 2\sig])
#    -#THIS "sf_select_range" WILL BE DETERMINED BY NATURE OF FREE PEAK OF BUTANOL FITTING PARAMETERS
sf_select_range = (1., 1.5)

#SF function
def init_scale_fact_1(df = pd.DataFrame(), epho_bins=[], cos_bins={}):
    
    #================================================================================================================
    #Dictionaries to hold scale factor for each energy + angle bins
    sf_1_dict          = {}
    
    #================================================================================================================
    #For each targets
    df_buta   = df[["cthe_cm", "epho", "el_pc_mmsq_pi0", "T_fin"]][(df["T_fin"] == 0) & \
                                                                   (df["el_pc_mmsq_pi0"] > -1.5) & (df["el_pc_mmsq_pi0"] < 1.)]
    df_carb   = df[["cthe_cm", "epho", "el_pc_mmsq_pi0", "T_fin"]][(df["T_fin"] == 1) & \
                                                                   (df["el_pc_mmsq_pi0"] > -1.5) & (df["el_pc_mmsq_pi0"] < 1.)]
    
    #================================================================================================================
    #For each Eg_bon
    for i, j in zip(range(len(epho_bins)-1), sorted(cos_bins.keys())):
        df_buta_Eg = df_buta[["cthe_cm", "el_pc_mmsq_pi0"]] \
                [(df_buta["epho"] > epho_bins[i]) & (df_buta["epho"] < epho_bins[i+1])]
        
        df_carb_Eg = df_carb[["cthe_cm", "el_pc_mmsq_pi0"]] \
                [(df_carb["epho"] > epho_bins[i]) & (df_carb["epho"] < epho_bins[i+1])]
        
        #list: For each epho bin, containing a list for each angle bin
        sf_1epho_cos_list = []
        
        #========================================================================================================
        #For each cos_bin
        for k in range(len(cos_bins[j])-1):
            df_buta_Eg_cos = df_buta_Eg[["el_pc_mmsq_pi0"]] \
                             [(df_buta_Eg["cthe_cm"] > cos_bins[j][k]) & (df_buta_Eg["cthe_cm"] < cos_bins[j][k+1])]
            
            df_carb_Eg_cos = df_carb_Eg[["el_pc_mmsq_pi0"]] \
                             [(df_carb_Eg["cthe_cm"] > cos_bins[j][k]) & (df_carb_Eg["cthe_cm"] < cos_bins[j][k+1])]
            
            #inner list: For each angle bin, containg a scalar for each mmsq bin
            sf_1epho_1cos_mmsq_list = []
            
            #========================================================================================================
            #Histogram for MMSQ
            buta_data = np.array(df_buta_Eg_cos["el_pc_mmsq_pi0"])
            carb_data = np.array(df_carb_Eg_cos["el_pc_mmsq_pi0"])
            buta_mmsq_hist, buta_bins = np.histogram(buta_data, bins="fd")
            carb_mmsq_hist, carb_bins = np.histogram(carb_data, bins=buta_bins)
            
            #Find Peak of Butanol
            peak = buta_bins[np.argmax(buta_mmsq_hist)]
            
            #Find stddev of Butanol
            mids   = 0.5*(buta_bins[:-1] + buta_bins[1:])
            mean   = np.average(mids, weights=buta_mmsq_hist)
            var    = np.average((mids - mean)**2, weights=buta_mmsq_hist)
            stddev = np.sqrt(var)
            
            #=====================================================================================================
            #Find ranges of mmsq: [\sigma, 2*\sigma] with Butanol
            range_list = [0.035 + sf_select_range[0]*stddev, 0.035 + sf_select_range[1]*stddev]
#             range_list_low = [peak + sf_select_range_low[0]*stddev, peak + sf_select_range_low[1]*stddev]
#             range_list_up  = [peak + sf_select_range_up[0]*stddev, peak + sf_select_range_up[1]*stddev]
            
            #Make New Histogram within defined ranges above: Events within [\sigma, 2*\sigma]
            n_b_i, buta_select_bins = np.histogram(buta_data[(buta_data>=range_list[0]) & (buta_data<=range_list[1])],\
                                                                        bins="fd")
            n_c_i, carb_select_bins = np.histogram(carb_data[(carb_data>=range_list[0]) & (carb_data<=range_list[1])], \
                                                                        bins=buta_select_bins)
#             n_b_i, buta_select_bins = np.histogram(buta_data[((buta_data>=range_list_low[0]) & (buta_data<=range_list_low[1])) | \
#                                                              ((buta_data>=range_list_up[0])  & (buta_data<=range_list_up[1]))], \
#                                                    bins="fd")
#             n_c_i, carb_select_bins = np.histogram(carb_data[((carb_data>=range_list_low[0]) & (carb_data<=range_list_low[1])) | \
#                                                              ((carb_data>=range_list_up[0])  & (carb_data<=range_list_up[1]))], \
#                                                    bins=buta_select_bins)
            
            #========================================================================================================    
            #Append n_b_i and n_c_i lists to sf_1epho_1cos_mmsq_list
            sf_1epho_1cos_mmsq_list.extend([n_b_i.tolist(), n_c_i.tolist()])
            
            #========================================================================================================    
            #Append sf_1epho_1cos_mmsq_list and to sf_1epho_cos_list
            sf_1epho_cos_list.append(sf_1epho_1cos_mmsq_list)

        #========================================================================================================            
        #Append to sf_eg_list to dict
        sf_1_dict["Eg_{:07.3f}".format(epho_bins[i])] = sf_1epho_cos_list
        
    return sf_1_dict, epho_bins, cos_bins

In [10]:
#Defined Chi-Squared function (MUST)
def chi_sq_fn(x, n_b_i, n_c_i):
    #As defined in Nick's analysis paper
    chi_sq = np.sum(np.power(n_b_i - x*n_c_i, 2) / (n_b_i + (x**2)*n_c_i))
    return chi_sq

In [11]:
#Scae factor phase 2 (MUST)
#    -Chi-Square miminization to get scale factor
def init_scale_fact_2(sf_1_dict={}, epho_bins=[], cos_bins={}):

    #==========================================================================================================
    #Dictionary to hold final sf values for each epho/cos bins
    sf_2_dict = {}
    
    #Keys of sf_1_dict corresponding to epho bins
    keys = sorted(sf_1_dict.keys())

    #==========================================================================================================
    #For each epho bins
#     for i in range(len(keys)):
    for i, j in zip(range(len(keys)), sorted(cos_bins.keys())):
        
        #list: For each epho bin, containing a list for each angle bin
        sf_1epho_cos_list = []
        
        #======================================================================================================
        #For each cos bins
        for k in range(len(cos_bins[j])-1):
    
            #Minimize using scipy
            result = scipy.optimize.minimize(chi_sq_fn, x0=5, \
                                             args=(np.array(sf_1_dict[keys[i]][k][0]), np.array(sf_1_dict[keys[i]][k][1])), \
                                             method="CG")
            
            if (result.success == True):
                sf = result.x[0]
            elif (result.success == False):
                #Set sf=5 just for now. Come back and change later
                sf = np.nan
            
            #Append final sf for each cos bins to sf_1epho_cos_lisabs
            sf_1epho_cos_list.append(sf)
    
        #==========================================================================================================
        sf_2_dict["Eg_{:07.3f}".format(epho_bins[i])] = sf_1epho_cos_list

    
    return sf_2_dict, epho_bins, cos_bins

In [12]:
#Apply scale_fact_1 (MUST)
init_sf_1_dict, epho_bins, cos_bins = init_scale_fact_1(df_GMSH, epho_bins, cos_bins)

In [13]:
#Call scale_fact_2(MUST)
init_sf_2_dict, epho_bins, cos_bins = init_scale_fact_2(init_sf_1_dict, epho_bins, cos_bins)

###### Plot Initial SF (CHECK) 

In [ ]:
#Plot SF (CHECK)
sf, subplot = plt.subplots(2, 5, figsize=(30, 10))

#Keys and new cos_bins to indicate epho/cos bins in sf_2_dict
keys            = sorted(init_sf_2_dict.keys())

#Cos_bins in 2d list
# cos_bins_center = [round(i+0.05, 2) for i in cos_bins if (i!=cos_bins[-1])]
cos_bins_cnts = []
for i in sorted(cos_bins.keys()):
    cos_bins_cnts_i = []
    for j in range(len(cos_bins[i])-1):
        cnt_i           = 0.5*(cos_bins[i][j] + cos_bins[i][j+1])
        cnt_i           = round(cnt_i, 4)
        cos_bins_cnts_i.append(cnt_i)
    cos_bins_cnts.append(cos_bins_cnts_i)

#Plot SF
for i in range(2):
    for j in range(5):
#         subplot[i, j].errorbar(cos_bins_center, init_sf_2_dict[keys[5*i+j]], yerr=0.5, fmt="o", \
#                                markeredgewidth=1, capsize=5, elinewidth=0.5, label="Direct SF", color="#4C72B0")

        subplot[i, j].errorbar(cos_bins_cnts[5*i+j], init_sf_2_dict[keys[5*i+j]], yerr=0.5, fmt="o", \
                               markeredgewidth=1, capsize=5, elinewidth=0.5, label="Direct SF", color="#4C72B0")
        
#Set title
for i in range(2):
    for j in range(5):
        subplot[i, j].set_title("$E_{{\gamma}} = [ {:1.2f}, {:1.2f}]$"\
                      .format(epho_bins[5*i+j], epho_bins[5*i+j+1]), fontsize=25) 
        
#labels, lim, grids, ticks font, legends
for i in subplot:
    for j in i:
        j.set_xlabel("$cos\\theta_{cm}$", fontsize=20)
        j.set_ylabel("SF",fontsize=20)
        #lim
        j.set_ylim(0, 10)
        #grids
        j.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.grid(b=True, which="major", color="gray", linewidth=1.0, linestyle="--")
        j.grid(b=True, which="minor", color="gray", linewidth=0.5, linestyle="--")
        #Ticks fontsize change
        j.tick_params(labelsize=15) 
        #Legends
        j.legend()

#Spacing
sf.subplots_adjust(hspace=0.5)

plt.show()
# sf.savefig("./run_period_1/init_sf")

In [ ]:
# mmsq_epho_cos_1 - Half epho bins (CHECK)
mmsq_epho_cos_1, subplot = plt.subplots(10, 10, figsize=(30, 20))

#Spacing for subplots
mmsq_epho_cos_1.subplots_adjust(hspace=0.5)
mmsq_epho_cos_1.subplots_adjust(wspace=0.2)

#Keys for epho bins
epho_keys = sorted(buta_dict.keys())
cos_keys  = sorted(cos_bins.keys())

#Bins and hist
n, b, p = [], [], []

#Peak, Mean, Stddev
peak, mean, stddev = [], [], []

#Plot
for i in range(0, 10):
    #inner list for Bins and hist list
    n_inner, b_inner, p_inner = [], [], []
    
    for j in range(10):

        #Ranges
        range_list = carb_dict[epho_keys[i]][j]["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
        
        #Bin Number
        bin_num = int(np.sqrt(len(buta_dict[epho_keys[i]][j]["el_pc_mmsq_pi0"] \
                                 [(buta_dict[epho_keys[i]][j]["el_pc_mmsq_pi0"] >= range_list[0]) & \
                                  (buta_dict[epho_keys[i]][j]["el_pc_mmsq_pi0"] <= range_list[1])])))
        if (bin_num < 10):
            bin_num=50
        
        #Plot Butanol
        n_b_i, b_b_i, p_b_i = subplot[i, j].hist(buta_dict[epho_keys[i]][j]["el_pc_mmsq_pi0"], label="butanol", \
                                                 bins=bin_num, alpha=0.7, range=(range_list[0], range_list[1]), \
                                                 color="#4C72B0")
        n_c_i, b_c_i, p_c_i = subplot[i, j].hist(carb_dict[epho_keys[i]][j]["el_pc_mmsq_pi0"], label="carbon", \
                                                 bins=bin_num, alpha=1., range=(range_list[0], range_list[1]), \
                                                 color="#55A868")        

        #Append all BUTANOL bins and hists
        n_inner.append(n_b_i)
        b_inner.append(b_b_i)
        p_inner.append(p_b_i)
        
        #lim
        subplot[i, j].set_xlim(range_list[0], range_list[1])

    #Append inner list to total list for bins and hists
    n.append(n_inner)
    b.append(b_inner)
    p.append(p_inner)

#Stats
for i in range(0, 10):
    #Inner list for peak, mean, stddev
    peak_inner, mean_inner, stddev_inner = [], [], []
    
    for j in range(0, 10):
        peak_i   = b[i][j][np.argmax(n[i][j])]
        mids_i   = 0.5*(b[i][j][1:] + b[i][j][:-1])
        mean_i   = np.average(mids_i, weights=n[i][j])
        var_i    = np.average((mids_i - mean_i)**2, weights=n[i][j])
        stddev_i = np.sqrt(var_i)
        
        #Append to peak, mean, stddev list
        peak_inner.append(peak_i)
        mean_inner.append(mean_i)
        stddev_inner.append(stddev_i)

    #Append inner list to total list for bins and hists
    peak.append(peak_inner)
    mean.append(mean_inner)
    stddev.append(stddev_inner)    
    
#Vertical lines to indicate SF computing regions
for i in range(10):
    for j in range(10):
        subplot[i, j].axvline(0.035 + sf_select_range[0]*stddev[i][j], linewidth=2.0, linestyle="--", color="red", alpha=0.5)
        subplot[i, j].axvline(0.035 + sf_select_range[1]*stddev[i][j], linewidth=2.0, linestyle="-", color="red", alpha=0.5)
    
#grids
for i in subplot:
    for j in i:    
        #grids
        j.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
        j.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")
        
#Set COS Title
for i in range(10):
    for j in range(10):
        subplot[i, j].set_title("[{:.2f}, {:.2f}]".format(cos_bins[cos_keys[i]][j], cos_bins[cos_keys[i]][j+1]), \
                                fontsize=15)
#Set EPHO Title
for i in range(10):
    for j in range(1):
        subplot[i, j].set_ylabel("[{:.2f}, {:.2f}]".format(epho_bins[i], epho_bins[i+1]), fontsize=15)
        
    
#Label
mmsq_epho_cos_1.text(0.9, 0.075, "Missing Mass Sq (${GeV}^2$)", ha="right", fontsize=40, weight="bold")
mmsq_epho_cos_1.text(0.9, 0.925, "$\pi^0$ COS$\\theta_{cm}$", ha="right", fontsize=40, weight="bold")        
mmsq_epho_cos_1.text(0.07, 0.5, "$E_{\gamma}$ (GeV)", va="center", rotation="vertical", fontsize=40, weight="bold")

#Legends
custom_label = [mpl.patches.Patch(facecolor="#4C72B0", alpha=0.7, label="Butanol"), \
                mpl.patches.Patch(facecolor="#55A868", alpha=0.7, label="Carbon"), \
                mpl.lines.Line2D([0], [0], color="red", alpha=0.7, linewidth=3, label="{:.2f}$\sigma$".format(sf_select_range[0]), linestyle="--"), \
                mpl.lines.Line2D([0], [0], color="red", alpha=0.7, linewidth=3, label="{:.2f}$\sigma$".format(sf_select_range[1]), linestyle="-")               
               ]
mmsq_epho_cos_1.legend(handles=custom_label, loc="upper left", bbox_to_anchor=(0.05, 0.9), ncol=6, fontsize=30)


plt.show()
# mmsq_epho_cos_1.savefig("./run_period_1/mmsq_epho_cos_1_high")

In [19]:
#Plot initial SF - Individual (CHECK)
def plot_init_sf_indi(epho_bin=0, cos_bin=0, \
                      epho_bins=[], cos_bins={}, \
                      buta_dict={}, carb_dict={}):
    
    #Declare figure
    plot_init_sf_indi = plt.figure(figsize=(15, 10))
    
    #keys
    epho_keys     = sorted(carb_dict.keys())
    cos_keys      = sorted(cos_bins.keys())
    
    #Ranges
    range_list = carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
    
    #Bin Number
    bin_num = int(np.sqrt(len(buta_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"] \
                             [(buta_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (buta_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"] <= range_list[1])])))
    if (bin_num < 10):
        bin_num=50

    #Plot Histogram
    n_b_i, b_b_i, p_b_i = plt.hist(buta_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="butanol", \
                                             bins=bin_num, alpha=0.7, range=(range_list[0], range_list[1]), \
                                             color="#4C72B0")
    n_c_i, b_c_i, p_c_i = plt.hist(carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="carbon", \
                                             bins=bin_num, alpha=1., range=(range_list[0], range_list[1]), \
                                             color="#55A868")        
    
    #lim
    plt.xlim(range_list[0], range_list[1])
    
    #Stats
    peak   = b_b_i[np.argmax(n_b_i)] 
    mids   = 0.5*(b_b_i[1:] + b_b_i[:-1])
    mean   = np.average(mids, weights=n_b_i)
    var    = np.average((mids - mean)**2, weights=n_b_i)
    stddev = np.sqrt(var)
    
    #Vertical lines to indicate SF computing regions
    plt.axvline(peak + sf_select_range[0]*stddev, linewidth=2.0, linestyle="--", color="red", alpha=1., \
                label="{:.2f}$\sigma$".format(sf_select_range[0]))
    plt.axvline(peak + sf_select_range[1]*stddev, linewidth=2.0, linestyle="-", color="red", alpha=1., \
                label="{:.2f}$\sigma$".format(sf_select_range[1]))
    
    #Grids
    plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
    plt.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")

    #Y-Axis Scaling
    def adjust_y_axis(x, pos):
        return "{:.0f}".format(x/100)
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(adjust_y_axis))    
    
    #Labels, ticks, lim
    plt.xlabel("$M^2_X$. $[GeV^2/c^4]$", fontsize=30)
    plt.ylabel("Counts $(x10^2)$", fontsize=30)
    plt.xticks(fontsize=25, fontweight="ultralight")
    plt.yticks(fontsize=25, fontweight="ultralight")
    plt.xlim(range_list[0], range_list[1])
    
    #Texts
    plt.text(0.025, 0.65, " Init. SF Calc. strt {:>10.2f}\n Init. SF Calc. end {:>10.2f}" \
#     plt.text(0.65, 0.65, " Init. SF Calc. strt {:>10.2f}\n Init. SF Calc. end {:>10.2f}" \
             .format(peak + sf_select_range[0]*stddev, peak + sf_select_range[1]*stddev), \
             fontsize=20, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)

    #Set Title
    plt.title("$E_{{\gamma}} \in [ {:.2f}, {:.2f}]$ \t $cos\\theta_{{cm}} \in [ {:.2f}, {:.2f}]$" \
              .format(epho_bins[epho_bin], epho_bins[epho_bin+1], cos_bins[cos_keys[epho_bin]][cos_bin], \
                      cos_bins[cos_keys[epho_bin]][cos_bin+1]),\
              fontsize=30)
    
    #Legends
    plt.legend(fontsize=20, loc="upper left")
#     plt.legend(fontsize=20, loc="upper right")

    plt.show()
    plot_init_sf_indi.savefig("./run_period_1/plot_init_sf_indi_high_{:d}_{:d}".format(epho_bin, cos_bin))    

In [ ]:
#Plot
epho_bin, cos_bin = 9, 6

plot_init_sf_indi(epho_bin=epho_bin, cos_bin=cos_bin, \
                 epho_bins=epho_bins, cos_bins=cos_bins, \
                 buta_dict=buta_dict, carb_dict=carb_dict)

###### Plot Initial Scaled Carbon (CHECK)

In [ ]:
#Plot Scale Factor Function
def plot_Scaled_carb(buta_dict={}, carb_dict={}, sf_dict={}, dilu_dict={}, epho_bins=[], cos_bins={}, \
                     epho_bin=0, cos_bin=0):

    #=====================================================================================
    #For 1 bin
    scale_factor_1bin = plt.figure(figsize=(15, 10))
    
    #=====================================================================================
    #keys for epho_bins and cos_bins
    epho_keys = sorted(carb_dict.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #df declare for easy reading
    df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
    df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]

    #Corresponding Scale factor for the interested bin
    sf   = sf_dict[epho_keys[epho_bin]][cos_bin]
    #dilu = dilu_dict[epho_keys[epho_bin]][cos_bin]
    
    #=====================================================================================
    #Determine Bin sizes
    bin_num = int(round(sf))*int(np.sqrt(len(df_carb))) 
    
    #Make histograms first
    hist_carb, bins_carb = np.histogram(np.array(df_carb["el_pc_mmsq_pi0"]), bins=bin_num)
    hist_buta, bins_buta = np.histogram(np.array(df_buta["el_pc_mmsq_pi0"]), bins=bins_carb)     
    hist_sc,   bins_sc   = sf*hist_carb, bins_carb 

    #Mids for plotting scale carbon
    mids = [0.5*(bins_carb[i] + bins_carb[i+1]) for i in range(len(bins_carb)-1)]

    #=====================================================================================    
    #Plot Butanol
    plt.step(mids, hist_buta, color="#4C72B0", label="Butanol", linewidth=2)
    #Plot Carbon
    plt.step(mids, hist_carb, color="#C44E52", label="Carbon", linewidth=2)
    #Plot Scaled Carbon
    plt.bar(mids, hist_sc, width=bins_sc[1]-bins_sc[0], alpha=0.5, color="#006367", label="Scaled Carbon")

    #=====================================================================================
    #Stat
    peak_buta   = bins_buta[np.argmax(hist_buta)]
    mean_buta   = np.average(mids, weights=hist_buta)
    var_buta    = np.average((mids - mean_buta)**2, weights=hist_buta)
    stddev_buta = np.sqrt(var_buta)
    peak_carb   = bins_carb[np.argmax(hist_carb)]
    mean_carb   = np.average(mids, weights=hist_carb)
    var_carb    = np.average((mids - mean_carb)**2, weights=hist_carb)
    stddev_carb = np.sqrt(var_carb)
    #Scaled Carbon STATS
    peak_sc   = bins_sc[np.argmax(hist_sc)]
    mean_sc   = np.average(mids, weights=hist_sc)
    var_sc    = np.average((mids - mean_sc)**2, weights=hist_sc)
    stddev_sc = np.sqrt(var_sc)
    
    #Put Stats
    plt.text(0.02, 0.75, " Scale Factor {:>14.3f}".format(sf), \
             fontsize=15, bbox=dict(facecolor='none', pad=2.0), transform=plt.gca().transAxes)
    plt.text(0.02, 0.6, "Butanol \n Peak {:>10.3f} \n Mean {:>10.3f} \n $\sigma$  {:>15.3f}" \
             .format(peak_buta, mean_buta, stddev_buta), \
             fontsize=15, bbox=dict(facecolor='none', pad=1.0), transform=plt.gca().transAxes)
    plt.text(0.02, 0.45, "Carbon \n Peak {:>10.3f} \n Mean {:>10.3f} \n $\sigma$  {:>15.3f}" \
             .format(peak_carb, mean_carb, stddev_carb), \
             fontsize=15, bbox=dict(facecolor='none', pad=1.0), transform=plt.gca().transAxes)

    #=====================================================================================
    #Labels and lim
    range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.9999]).tolist()
    plt.xlim(range_list[0], range_list[1])
    plt.xlabel("Missing mass sq. $[GeV^2/c^4]$", fontsize=20)
    plt.ylabel("Counts $(x10^2)$", fontsize=20)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)

    #Y-Axis Scaling
    def adjust_y_axis(x, pos):
        return "{:.0f}".format(x/100)
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(adjust_y_axis))

    #Grids
    plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.grid(b=True, which="major", color="gray", linewidth=1.0, linestyle="--")
    plt.grid(b=True, which="minor", color="gray", linewidth=0.5, linestyle="--")

    #Legends
    plt.legend(fontsize=20, loc="upper left")
    
    
    #Title
    plt.title("$E_{{\gamma}} = [{:.2f}, {:.2f}] $ \t COS$\\theta$ = [{:.2f}, {:.2f}]"\
              .format(epho_bins[epho_bin], epho_bins[epho_bin+1], cos_bins[cos_keys[epho_bin]][cos_bin], \
                      cos_bins[cos_keys[epho_bin]][cos_bin+1]), fontsize=20)

    #=====================================================================================
    plt.show()
    scale_factor_1bin.savefig("./run_period_1/scaled_carb_1bin_{:d}_{:d}".format(epho_bin, cos_bin))

In [ ]:
#Plot Scale Factor for epho_bin = 10
for i in range(10):
    plot_Scaled_carb(buta_dict, carb_dict, init_sf_2_dict, epho_bins=epho_bins, cos_bins=cos_bins, \
                     epho_bin=0, cos_bin=i)

###### Butanol Free-proton Peak Parameter Extraction (MUST)

In [14]:
#Butanol - SF*Carbon Parameter Exraction
def buta_free_params(buta_dict={}, carb_dict={}, sf_dict={}, epho_bins=[], cos_bins={}):
    #=====================================================================================
    #Dictionary to be returned
    bf_params_dict = {}
    
    #keys for epho_bins
    epho_keys = sorted(carb_dict.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
        #list: For each epho bin, containing a list for each angle bin
        bf_1epho_cos_list = []
        
        for cos_bin in range(len(cos_bins[j])-1):
            #df declare for easy reading
            df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
            df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]
            
            #Remove Outliers - region selection
            range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            
            #Remove Outliers - remove
            df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
            df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

            #Corresponding Scale factor for the interested bin
            sf = sf_dict[epho_keys[epho_bin]][cos_bin]

            #=====================================================================================
            #Determine Bin sizes
            bin_num = int(round(sf))*int(np.sqrt(len(df_carb))) 

            #Make histograms first
            hist_carb, bins_carb = np.histogram(np.array(df_carb["el_pc_mmsq_pi0"]), bins=bin_num)
            hist_buta, bins_buta = np.histogram(np.array(df_buta["el_pc_mmsq_pi0"]), bins=bins_carb)

            #=====================================================================================
            #If SF is too large that hist_bf have negative value, change sf and compute again
            hist_bf, bins_bf = hist_buta - sf*hist_carb, bins_carb
            counter_bad_bin  = 0 
            for i in range(len(hist_bf)):
                if (hist_bf[i] <= 0):
                    counter_bad_bin += 1
            #If # of bins that have larger sf*carb than butanol is more than 1/20 of butanol bins, half the sf
            if (counter_bad_bin >= int(len(bins_bf)/10)):
                #CHANGE the DENOMENATOR if too manyless bins of hist_bf have negative values!!!!
                sf = sf/2.5
                                
            #=====================================================================================
            #Free butanol peak histograms
            hist_bf, bins_bf = hist_buta - sf*hist_carb, bins_carb

            #If any bin in hist_bf is negative, set it to 0
            for i in range(len(hist_bf)):
                if(hist_bf[i] < 0):
                    hist_bf[i] = 0
            
            #Mids for plotting scale carbon HISTOGRAM
            mids     = np.array([0.5*(bins_bf[i] + bins_bf[i+1]) for i in range(len(bins_bf)-1)])    

            #Stats for free butanol histogram
            #    -Manualy set peak_bf for high_energy
            peak_bf  = 0.035 
            #peak_bf  = bins_buta[np.argmax(hist_buta)]
            mean_bf  = np.average(mids, weights=hist_bf)
            var_bf   = np.average((mids - mean_bf)**2, weights=hist_bf)
            stddv_bf = np.sqrt(var_bf) 
            max_bf   = np.max(hist_bf)

            #=====================================================================================
            #Shorten the fit range
            #    -peak_bf +/- certain amount of range to enclose the peak and 1 background.
            #    -If the fit range is too wide, the background consists of many gaussians, hindering bg fits.
            xmin_fit = peak_bf - 2*stddv_bf
            xmax_fit = peak_bf + 2*stddv_bf
                        
            #=================================================================================================
            #Bins within fitting range
            boolean_arry_bins = (mids > xmin_fit) & (mids < xmax_fit)
            mids_fit          = mids[boolean_arry_bins]
            
            #subset of hist_bf for fitting
            hist_bf_fit = hist_bf[boolean_arry_bins]
            
            #Stats for free butanol histogram
            peak_bf_fit  = bins_buta[np.argmax(hist_buta)]
            mean_bf_fit  = np.average(mids_fit, weights=hist_bf_fit)
            var_bf_fit   = np.average((mids_fit - mean_bf_fit)**2, weights=hist_bf_fit)
            stddv_bf_fit = np.sqrt(var_bf_fit) 
            max_bf_fit   = np.max(hist_bf_fit)     
            pedestal     = max_bf_fit/20
            
            #=====================================================================================
            #Fit Function (inner)
            def fit_func_combined(x, A, sigma, B, sigma2):
                #Background function
                #Set mu fixed to mean_bf to reduce dof
                gaussian  = A*np.exp(-1.0*(x - mean_bf)**2  / (2*sigma**2))
                #Peak function
                #Set mu fixed to peak_bf to reduce dof
                gaussian2 = B*np.exp(-1.0*(x - peak_bf)**2 / (2*sigma2**2))
                #Final Func
                final_func = gaussian + gaussian2 + pedestal
                return final_func    

            #=====================================================================================
            #Initial guess
            init_guess = [round(max_bf_fit/2, 3), round(stddv_bf_fit, 3), \
                          round(max_bf_fit, 3), round(stddv_bf_fit/5, 3)]
            
            #Fit
            try:
                params, pconv = curve_fit(fit_func_combined, xdata=mids_fit, ydata=hist_bf_fit, \
                                          p0=init_guess)

            except RuntimeError:
                #Set empty list
                params  = [np.nan for i in range(len(init_guess))]
                pconv   = [np.nan for i in range(len(init_guess))]                
                
            
            #=====================================================================================
            #Butanol free peak start/end positions
            buta_start_end = (peak_bf - 3*params[3], peak_bf + 3*params[3])

            #=====================================================================================
            #Append mids, bins_bf, hist_bf, and final params for each cos bins to bf_1epho_cos_lisabs
            bf_1epho_cos_list.append([mids, bins_bf, hist_bf, params, mean_bf, peak_bf, buta_start_end, \
                                      mids_fit, pedestal])
    
        #=========================================================================================
        bf_params_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = bf_1epho_cos_list
        
    return bf_params_dict, epho_bins, cos_bins    

In [15]:
#bf_params_dist has Bins, x_coordinates, hist_bf, and fitting parameters for each epho/cos_bins 
bf_params_dict, epho_bins, cos_bins = buta_free_params(buta_dict, carb_dict, init_sf_2_dict, \
                                                       epho_bins, cos_bins)

In [ ]:
#Plot BF - First Try - First Half (CHECK)
bf_mmsq_epho_cos_1, subplot = plt.subplots(10, 10, figsize=(30, 20))

#Spacing for subplots
bf_mmsq_epho_cos_1.subplots_adjust(hspace=0.5)
bf_mmsq_epho_cos_1.subplots_adjust(wspace=0.2)

#Keys for epho bins
epho_keys = sorted(bf_params_dict.keys())
cos_keys  = sorted(cos_bins.keys())

#Fit function
def fit_func_combined(x, A, mu, sigma, B, mu2, sigma2, pedestal):
    #Background function
    gaussian  = A*np.exp(-1.0*(x - mu)**2  / (2*sigma**2))
    #Peak function
    gaussian2 = B*np.exp(-1.0*(x - mu2)**2 / (2*sigma2**2))
    #Final Func
    final_func = gaussian + gaussian2 + pedestal
    return final_func

#Plot
for i in range(0, 10):
    for j in range(10):
        #Extracted fit parameters
        A_fit              = bf_params_dict[epho_keys[i]][j][3][0]
        sigma_fit          = bf_params_dict[epho_keys[i]][j][3][1]
        B_fit              = bf_params_dict[epho_keys[i]][j][3][2]
        sigma2_fit         = bf_params_dict[epho_keys[i]][j][3][3]
        mu_fit             = bf_params_dict[epho_keys[i]][j][4]
        mu2_fit            = bf_params_dict[epho_keys[i]][j][5]
        buta_start_end_tpl = bf_params_dict[epho_keys[i]][j][6]
        pedestal           = bf_params_dict[epho_keys[i]][j][8]

        
        #Bins, x_coordinates, hist_bf, fitting parameters
        x_coor      = np.array(bf_params_dict[epho_keys[i]][j][0]) 
        x_coor_fit  = np.array(bf_params_dict[epho_keys[i]][j][7]) 
        y_val       = fit_func_combined(x_coor_fit, A_fit, mu_fit, sigma_fit, \
                                        B_fit, mu2_fit, sigma2_fit, pedestal)
        bins_bf     = bf_params_dict[epho_keys[i]][j][1]
        hist_bf     = bf_params_dict[epho_keys[i]][j][2]
        
        #Plot butanol free histogram
        subplot[i, j].bar(x_coor, hist_bf, width=bins_bf[1]-bins_bf[0], alpha=0.7, label="buta-sf*carb", color="#4C72B0")
        
        #Plot fitting 
        subplot[i, j].plot(x_coor_fit, y_val, linewidth=2, label="Buta free-nucl", color="#C44E52")
        
        #Butanol free nucleon sharp peak start/end positions
        subplot[i, j].axvline(buta_start_end_tpl[0], linestyle="--", color="red", alpha=0.5, label="Buta Free-Nucl Peak Start/End pos")
        subplot[i, j].axvline(buta_start_end_tpl[1], linestyle="--", color="red", alpha=0.5)
        
#CROSS OUT bad bins
for i in range(0, 10):
    for j in range(10):
        #epho=0
        if (i==0 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=1
        elif (i==1 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==1 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=2
        elif (i==2 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==2 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=3
        elif (i==3 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==3 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==3 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=4
        elif (i==4 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==4 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==4 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==4 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=5
        elif (i==5 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==5 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==5 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==5 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=6
        elif (i==6 and j==5):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=7
        elif (i==7 and j==4):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==5):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==7 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=8
        elif (i==8 and j==0):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==8 and j==3):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==4):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==5):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==8 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=9
        elif (i==9 and j==0):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==9 and j==2):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==9 and j==3):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==4):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==5):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==9 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
            
#grids
for i in subplot:
    for j in i:    
        #grids
        j.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
        j.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")
        
#Set COS Title
for i in range(10):
    for j in range(10):
        subplot[i, j].set_title("[{:.2f}, {:.2f}]".format(cos_bins[cos_keys[i]][j], cos_bins[cos_keys[i]][j+1]), \
                                fontsize=15)
#Set EPHO Title
for i in range(10):
    for j in range(1):
        subplot[i, j].set_ylabel("[{:.2f}, {:.2f}]".format(epho_bins[i], epho_bins[i+1]), fontsize=15)
        
#Label
bf_mmsq_epho_cos_1.text(0.9, 0.07, "Missing Mass Sq (${GeV}^2$)", ha="right", fontsize=40, weight="bold")
bf_mmsq_epho_cos_1.text(0.9, 0.925, "$\pi^0$ COS$\\theta_{cm}$", ha="right", fontsize=40, weight="bold")        
bf_mmsq_epho_cos_1.text(0.07, 0.5, "$E_{\gamma}$ (GeV)", va="center", rotation="vertical", fontsize=40, weight="bold")

#Legends
custom_label = [mpl.patches.Patch(facecolor="#4C72B0", alpha=1, linewidth=3, label="Buta - SF*Carb", linestyle="-"), \
                mpl.lines.Line2D([0], [0], color="#C44E52", alpha=1, linewidth=3, label="Fitting Buta Free-Nucl Peak", linestyle="-"), \
                mpl.lines.Line2D([0], [0], color="red", alpha=1, linewidth=3, label="Buta Free-Nucl Peak Start/End pos", linestyle="--")]

bf_mmsq_epho_cos_1.legend(handles=custom_label, loc="upper left", bbox_to_anchor=(0.05, 0.9), \
                              ncol=3, fontsize=20, facecolor="none")


plt.show()
# bf_mmsq_epho_cos_1.savefig("./run_period_1/bf_mmsq_epho_cos_1")        

In [ ]:
#Plot overview of peaks + buta_free_peak positions (CHECK) 
Fit_params, subplot = plt.subplots(2, 5, figsize=(30, 10))

#Keys and new cos_bins to indicate epho/cos bins in sf_2_dict
epho_keys       = sorted(init_sf_2_dict.keys())


# cos_bins_center = [round(0.5 * (cos_bins[i] + cos_bins[i+1]), 2) for i in range(len(cos_bins) - 1)]

#Cos_bins_cnt in 2d list
cos_bins_cnts = []
for i in sorted(cos_bins.keys()):
    cos_bins_cnts_i = []
    for j in range(len(cos_bins[i])-1):
        cnt_i           = 0.5*(cos_bins[i][j] + cos_bins[i][j+1])
        cnt_i           = round(cnt_i, 4)
        cos_bins_cnts_i.append(cnt_i)
    cos_bins_cnts.append(cos_bins_cnts_i)
    
    

#Plot fit parameters
for i in range(2):
    for j in range(5):
        
        #Create list of wanted values from the dictionary
        peak_ls  = [bf_params_dict[epho_keys[5*i+j]][k][5] for k in range(len(bf_params_dict[epho_keys[5*i+j]]))]
        bf_start = [bf_params_dict[epho_keys[5*i+j]][k][6][0] for k in range(len(bf_params_dict[epho_keys[5*i+j]]))]
        bf_end   = [bf_params_dict[epho_keys[5*i+j]][k][6][1] for k in range(len(bf_params_dict[epho_keys[5*i+j]]))]
        
        #Peak plot
        subplot[i, j].plot(cos_bins_cnts[5*i+j], peak_ls, "o", color="#C44E52", label="Peak")
        subplot[i, j].plot(cos_bins_cnts[5*i+j], bf_start, "o", color="#55A868", label="Peak - $3\sigma$")
        subplot[i, j].plot(cos_bins_cnts[5*i+j], bf_end, "o", color="#4C72B0", label="Peak + $3\sigma$")

#Set title
for i in range(2):
    for j in range(5):
        subplot[i, j].set_title("$E_{{\gamma}} = [ {:1.2f}, {:1.2f}]$"\
                      .format(epho_bins[5*i+j], epho_bins[5*i+j+1]), fontsize=25) 
        
#labels, lim, grids, ticks font, legends
for i in subplot:
    for j in i:
        j.set_xlabel("$cos\\theta_{cm}$", fontsize=20)
        j.set_ylabel("MM",fontsize=20)
        #lim
        j.set_ylim(-0.2, 0.2)
        #grids
        j.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.grid(b=True, which="major", color="gray", linewidth=1.0, linestyle="--")
        j.grid(b=True, which="minor", color="gray", linewidth=0.5, linestyle="--")
        #Ticks fontsize change
        j.tick_params(labelsize=15) 
        #Legends
        #j.legend()

#Spacing
Fit_params.subplots_adjust(hspace=0.5)
Fit_params.subplots_adjust(wspace=0.5)        
# Fit_params.tight_layout()
    
#Legends
custom_label = [mpl.lines.Line2D([0], [0], color="#C44E52", marker="o", ls="none", label="Peak$"), \
                mpl.lines.Line2D([0], [0], color="#55A868", marker="o", ls="none", label="Peak - $3\sigma$"), \
                mpl.lines.Line2D([0], [0], color="#4C72B0", marker="o", ls="none", label="Peak + $3\sigma$")]

Fit_params.legend(handles=custom_label, loc="upper left", ncol=3, fontsize=20)
# Fit_params.legend(handles=custom_label, loc="upper left", bbox_to_anchor=(0.1, 0.95), ncol=3, fontsize=20)


plt.show()
# Fit_params.savefig("./run_period_1/Fit_params")

###### Butanol Free-proton Peak Parameter Extraction - Individually (MUST)

In [16]:
#bf_params_final_indi
def buta_free_params_indi(epho_bin=0, cos_bin=0, bf_start=-0.01, bf_end=0.07, 
                         epho_bins=[], cos_bins={}):
    #Keys for epho bins
    epho_keys = sorted(cos_bins.keys())
    
    #Store redone bf ranges
    bf_params_dict[epho_keys[epho_bin]][cos_bin][6] = (bf_start, bf_end)

In [17]:
#Plot refits
def plot_fixed(epho_bin=0, cos_bin=0, bf_params_dict={}, epho_bins=[], cos_bins={}):

    #Plot size
    fit_redo = plt.figure(figsize=(10,6))
    
    #Keys for epho
    epho_keys = sorted(bf_params_dict.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #Fit function
    def fit_func_combined(x, A, mu, sigma, B, mu2, sigma2, pedestal):
        #Background function
        gaussian  = A*np.exp(-1.0*(x - mu)**2  / (2*sigma**2))
        #Peak function
        gaussian2 = B*np.exp(-1.0*(x - mu2)**2 / (2*sigma2**2))
        #Pedestal
        pedestal  = pedestal
        #Final Func
        final_func = gaussian + gaussian2 + pedestal 
        return final_func

    #1 fit fucnt
    def fit_func_1(x, A, mu, sigma, pedestal):
        #Background function
        gaussian  = A*np.exp(-1.0*(x - mu)**2  / (2*sigma**2))
        #Final Func
        final_func = gaussian + pedestal 
        return final_func
    
    #Extracted fit parameters
    A_fit              = bf_params_dict[epho_keys[epho_bin]][cos_bin][3][0]
    sigma_fit          = bf_params_dict[epho_keys[epho_bin]][cos_bin][3][1]
    B_fit              = bf_params_dict[epho_keys[epho_bin]][cos_bin][3][2]
    sigma2_fit         = bf_params_dict[epho_keys[epho_bin]][cos_bin][3][3]
    buta_start_end_tpl = bf_params_dict[epho_keys[epho_bin]][cos_bin][6]
    pedestal           = bf_params_dict[epho_keys[epho_bin]][cos_bin][8]
    mu_fit             = bf_params_dict[epho_keys[epho_bin]][cos_bin][4]
    mu2_fit            = bf_params_dict[epho_keys[epho_bin]][cos_bin][5]

    #Bins, x_coordinates, hist_bf
    mids        = np.array(bf_params_dict[epho_keys[epho_bin]][cos_bin][0]) 
    mids_fit    = np.array(bf_params_dict[epho_keys[epho_bin]][cos_bin][7]) 
    bins_bf     = bf_params_dict[epho_keys[epho_bin]][cos_bin][1]
    hist_bf     = bf_params_dict[epho_keys[epho_bin]][cos_bin][2]

    #Examine each fit funcations separately
    y_val       = fit_func_combined(mids_fit, \
                                    A_fit, mu_fit, sigma_fit, \
                                    B_fit, mu2_fit, sigma2_fit, \
                                    pedestal)
    y_val_1 = fit_func_1(mids_fit, A_fit, mu_fit, sigma_fit, pedestal)
    y_val_2 = fit_func_1(mids_fit, B_fit, mu2_fit, sigma2_fit, pedestal)

    #Plot each fit function separately
    plt.plot(mids_fit, y_val_1, linewidth=2, label="fit_func_1")
    plt.plot(mids_fit, y_val_2, linewidth=2, label="fit_func_2")

    #Plot butanol free histogram
    plt.bar(mids, hist_bf, width=bins_bf[1]-bins_bf[0], alpha=0.7, label="buta-sf*carb", color="#4C72B0")

    #Plot fitting 
    plt.plot(mids_fit, y_val, linewidth=2, label="Buta free-nucl", color="#C44E52")

    #Butanol free nucleon sharp peak start/end positions
    plt.axvline(buta_start_end_tpl[0], linestyle="--", color="red", alpha=0.5, label="Buta Free-Nucl Peak Start/End pos")
    plt.axvline(buta_start_end_tpl[1], linestyle="--", color="red", alpha=0.5)

    #Grids
    plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
    plt.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")

    #Texts
    plt.text(0.65, 0.65, " Free-nucl peak start {:>14.3f}\n Free-nucl peak end {:>14.3f}" \
             .format(buta_start_end_tpl[0], buta_start_end_tpl[1]), \
             fontsize=10, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)

    #Labels, ticks, lim
    plt.xlabel("Missing mass sq. $[GeV^2/c^4]$", fontsize=15)
    plt.ylabel("Counts", fontsize=15)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    # plt.xlim(-0.3, 0.3)

    #Set Title
    plt.title("$E_{{\gamma}} \in [ {:.2f}, {:.2f}]$ \t $cos\\theta_{{cm}} \in [ {:.2f}, {:.2f}]$" \
              .format(epho_bins[epho_bin], epho_bins[epho_bin+1], cos_bins[cos_keys[epho_bin]][cos_bin], \
                      cos_bins[cos_keys[epho_bin]][cos_bin+1]),\
              fontsize=15)
        
    #Legends
    plt.legend(fontsize=10, loc="upper right")

    plt.show()
    fit_redo.savefig("./run_period_1/fit_redo_high_{:d}_{:d}".format(epho_bin, cos_bin))    

In [18]:
##COS=0

epho_bin, cos_bin = 7, 0
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.064, bf_end=0.13, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

In [ ]:
##Call plot_fixed() (CHECK)
epho_bin, cos_bin = 2, 6

plot_fixed(epho_bin, cos_bin, bf_params_dict, epho_bins, cos_bins)

###### BUTA fit + CARB fit: Separately - GLOBAL (MUST) 

In [20]:
#Fit Buta:
def fit_buta_global(buta_dict={}, carb_dict={}, bf_params_dict={}, init_sf_2_dict={}, \
                    epho_bins=[], cos_bins={}, \
                    left_fit_range=2.5, right_fit_range=2.5, \
                    bin_size_mltp = 0.5, \
#                     buta_init_guess_mltplr = [1, 0.5, 1, 1, 1, 1, 1], \
#                     bounds_buta_mltplr = ([0.5, 0.1, 0, -np.inf, -np.inf, -np.inf, -np.inf], \
#                                           [1.5, 1, np.inf, np.inf, np.inf, np.inf, np.inf])):
                    buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
                    bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 2, 1, 1, 5, 1])):
    
    #Dict to store final
    buta_fit_dict = {}
    
    #Dict to store err msgs
    err_dict = {}
    cnt      = 0    
    
    #keys for epho_bins
    epho_keys = sorted(carb_dict.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
        
        #Dict to store dict or whole cos
        buta_1epho_cos_dict = {}
        
        for cos_bin in range(len(cos_bins[j])-1):
            #df declare for easy reading
            df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
            df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]
            
            #Remove Outliers - region selection
            range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            
            #Remove Outliers - remove
            df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
            df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

            #=================================================================================================
            #Into numpy
            butanol_data = np.array(df_buta)
            carbon_data  = np.array(df_carb)

            #Bin Number Determine
            raw_sf = int(init_sf_2_dict[epho_keys[epho_bin]][cos_bin])
            if (raw_sf>=7):
                raw_sf = raw_sf - 2
            bin_num = int(bin_size_mltp*raw_sf*int(np.sqrt(len(carbon_data))))
            
            #hist_carbogram
            hist_carb, carb_bins = np.histogram(carbon_data, bins=bin_num)
            hist_buta, buta_bins = np.histogram(butanol_data, bins=carb_bins)

            #Bin Centers
            bin_centers = np.array([0.5*(buta_bins[i] + buta_bins[i+1]) for i in range(len(buta_bins)-1)])            
            
            #Stats BUTA
            #    -For high energy set peak_buta=0.035
            peak_buta   = 0.035
            #peak_buta   = buta_bins[np.argmax(hist_buta)]
            mean_buta   = np.average(bin_centers, weights=hist_buta)
            var_buta    = np.average((bin_centers-mean_buta)**2, weights=hist_buta)
            stddev_buta = np.sqrt(var_buta)
            num_buta    = len(butanol_data)
            max_buta    = np.max(hist_buta)
            
            #=================================================================================================
            #Fitting range
            #left_fit_range  = 2.5
            #right_fit_range = 2.5
            start_end = [(peak_buta - left_fit_range*np.abs(peak_buta - bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])), \
                         (peak_buta + right_fit_range*np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - peak_buta))]

            #Prevent fits to be delcared out of carbon dist. range
            carb_lims = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            if (start_end[0] < carb_lims[0]):
                start_end[0] = carb_lims[0]
            if (start_end[1] > carb_lims[1]):
                start_end[1] = carb_lims[1]
            
            #=================================================================================================
            #Bins within fitting range
            boolean_arry_bins = (bin_centers > start_end[0]) & (bin_centers < start_end[1])
            bin_centers_new   = bin_centers[boolean_arry_bins]

            #=================================================================================================
            #Subset of histogram within FITTING RANGE
            #    -this histogram will be used for final chisq minimization fitting!!!
            hist_buta_new = hist_buta[boolean_arry_bins]

            #Stats BUTA new
            mean_buta_new   = np.average(bin_centers_new, weights=hist_buta_new)
            var_buta_new    = np.average((bin_centers_new-mean_buta_new)**2, weights=hist_buta_new)
            stddev_buta_new = np.sqrt(var_buta_new)
            max_buta_new    = np.max(hist_buta_new)    
            sigma_buta      = np.sqrt(hist_buta_new)
        
            #Dont want any entries in sigma to be zero -> division by 0 error
            for i in range(len(sigma_buta)):
                if (sigma_buta[i] == 0):
                    sigma_buta[i] = 0.01
        
            #=================================================================================================
            #Fit buta
            def fit_buta(x, A, sigma1, B, mu, sigma2, a):
                gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
                gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
                poly  = a
                y     = gaus1 + gaus2 + poly
                return y
            
            #Bounds: needed variables
            bf_strt_end_dist = np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - 
                                      bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])         
            
            #initial guess + bounds
            buta_init_guess = [buta_init_guess_mltplr[0]*max_buta_new, \
                               buta_init_guess_mltplr[1]*stddev_buta_new, \
                               buta_init_guess_mltplr[2]*max_buta_new, \
                               buta_init_guess_mltplr[3]*peak_buta, \
                               buta_init_guess_mltplr[4]*stddev_buta_new, \
                               buta_init_guess_mltplr[5]*max_buta_new]        
            #Bounds of carb
            bounds_buta = ([bounds_buta_mltplr[0][0]*max_buta_new, bounds_buta_mltplr[0][1]*stddev_buta_new, \
                            bounds_buta_mltplr[0][2]*max_buta_new, bounds_buta_mltplr[0][3]*bf_strt_end_dist + peak_buta, 
                            bounds_buta_mltplr[0][4]*stddev_buta_new, \
                            bounds_buta_mltplr[0][5]*max_buta_new]
                           , \
                           [bounds_buta_mltplr[1][0]*max_buta_new, bounds_buta_mltplr[1][1]*stddev_buta_new, \
                            bounds_buta_mltplr[1][2]*max_buta_new, bounds_buta_mltplr[1][3]*bf_strt_end_dist + peak_buta, 
                            bounds_buta_mltplr[1][4]*stddev_buta_new, \
                            bounds_buta_mltplr[1][5]*max_buta_new])                          

            #Curve_fit
            try:
                buta_popt, buta_pconv = scipy.optimize.curve_fit(fit_buta, xdata=bin_centers_new, \
                                                                 ydata=hist_buta_new, \
                                                                 p0=buta_init_guess, bounds=bounds_buta, \
                                                                 sigma=sigma_buta, absolute_sigma=True)
            except RuntimeError as err1:
                cnt += 1
                err_dict["{:d}".format(cnt)] = [epho_bin, cos_bin, epho_bins[epho_bin], \
                                                cos_bins[cos_keys[epho_bin]][cos_bin], err1]
                #Set empty list
                buta_popt  = [0 for i in range(len(buta_init_guess))]
                buta_pconv = [0 for i in range(len(buta_init_guess))]
                    
            #=================================================================================================
            #Into dicts
            buta_1epho_1cos_dict = {"bin_centers_new":bin_centers_new, "buta_bins":buta_bins, \
                                   "num_buta":num_buta, "peak_buta":peak_buta, \
                                   "buta_popt":buta_popt, "start_end":start_end, \
                                   "bin_size_mltp":bin_size_mltp, "buta_pconv":buta_pconv} 

            #Append
            buta_1epho_cos_dict["cos_{:07.3f}".format(cos_bins[cos_keys[epho_bin]][cos_bin])] = buta_1epho_1cos_dict

        #=================================================================================================
        #Save to dictionary
        buta_fit_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = buta_1epho_cos_dict
        
    return buta_fit_dict, err_dict, epho_bins, cos_bins

In [21]:
#Call fit_buta
buta_fit_dict, buta_err_dict, epho_bins, cos_bins = \
fit_buta_global(buta_dict=buta_dict, carb_dict=carb_dict, \
                bf_params_dict=bf_params_dict, \
                init_sf_2_dict=init_sf_2_dict, \
                epho_bins=epho_bins, cos_bins=cos_bins, \
                left_fit_range=2.5, right_fit_range=2.5, \
#                 buta_init_guess_mltplr = [1, 0.5, 1, 1, 1, 1, 1], \
#                 bounds_buta_mltplr = ([0.5, 0.1, 0, 0, 0, 0, 0], \
#                                       [1.5, 1, 0.2, np.inf, np.inf, np.inf, np.inf]))
                buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
                bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 2, 1, 1, 5, 1]))

In [22]:
#Fit Buta:
def fit_carb_global(buta_dict={}, carb_dict={}, bf_params_dict={}, init_sf_2_dict={}, \
                    epho_bins=[], cos_bins={}, \
                    left_fit_range=2.5, right_fit_range=2.5, \
                    bin_size_mltp=0.5, \
                    carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 1], \
                    bounds_carb_mltplr = ([0, 0, 0, -1, 0, 0], [1, 10, 1, 2, 10, np.inf])):
    
    #Dict to store final
    carb_fit_dict = {}
    
    #Dict to store err msgs
    err_dict = {}
    cnt      = 0
    
    #keys for epho_bins
    epho_keys = sorted(carb_dict.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
        
        #Dict to store dict or whole cos
        carb_1epho_cos_dict = {}
        
        for cos_bin in range(len(cos_bins[j])-1):
            #df declare for easy reading
            df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
            df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]
            
            #Remove Outliers - region selection
            range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            
            #Remove Outliers - remove
            df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
            df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

            #=================================================================================================
            #Into numpy
            butanol_data = np.array(df_buta)
            carbon_data  = np.array(df_carb)

            #Bin Number Determine
            raw_sf = int(init_sf_2_dict[epho_keys[epho_bin]][cos_bin])
            if (raw_sf>=7):
                raw_sf = raw_sf - 2
            bin_num = int(bin_size_mltp*raw_sf*int(np.sqrt(len(carbon_data))))
            
            #hist_carbogram
            hist_carb, carb_bins = np.histogram(carbon_data, bins=bin_num)
            hist_buta, buta_bins = np.histogram(butanol_data, bins=carb_bins)

            #Bin Centers
            bin_centers = np.array([0.5*(buta_bins[i] + buta_bins[i+1]) for i in range(len(buta_bins)-1)])            
            
            #Stats BUTA
            #    -For high energy, set peak_buta manually
            peak_buta   = 0.035
            #peak_buta   = buta_bins[np.argmax(hist_buta)]
            mean_buta   = np.average(bin_centers, weights=hist_buta)
            var_buta    = np.average((bin_centers-mean_buta)**2, weights=hist_buta)
            stddev_buta = np.sqrt(var_buta)
            num_buta    = len(butanol_data)
            max_buta    = np.max(hist_buta)
            #Stats CARB
            peak_carb   = buta_bins[np.argmax(hist_carb)]
            mean_carb   = np.average(bin_centers, weights=hist_carb)
            var_carb    = np.average((bin_centers-mean_carb)**2, weights=hist_carb)
            stddev_carb = np.sqrt(var_carb)
            num_carb    = len(carbon_data)
            max_carb    = np.max(hist_carb)
            
            #=================================================================================================
            #Fitting range
            #left_fit_range  = 2.5
            #right_fit_range = 2.5
            start_end = [(peak_buta - left_fit_range*np.abs(peak_buta - bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])), \
                         (peak_buta + right_fit_range*np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - peak_buta))]

            #Prevent fits to be delcared out of carbon dist. range
            carb_lims = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            if (start_end[0] < carb_lims[0]):
                start_end[0] = carb_lims[0]
            if (start_end[1] > carb_lims[1]):
                start_end[1] = carb_lims[1]
            
            #=================================================================================================
            #Bins within fitting range
            boolean_arry_bins = (bin_centers > start_end[0]) & (bin_centers < start_end[1])
            bin_centers_new   = bin_centers[boolean_arry_bins]

            #=================================================================================================
            #Subset of histogram within FITTING RANGE
            hist_carb_new = hist_carb[boolean_arry_bins]

            #Stats CARB new
            mean_carb_new   = np.average(bin_centers_new, weights=hist_carb_new)
            var_carb_new    = np.average((bin_centers_new-mean_carb_new)**2, weights=hist_carb_new)
            stddev_carb_new = np.sqrt(var_carb_new)
            max_carb_new    = np.max(hist_carb_new)
            sigma_carb      = np.sqrt(hist_carb_new)

            #Dont want any entries in sigma to be zero -> division by 0 error
            for i in range(len(sigma_carb)):
                if (sigma_carb[i] == 0):
                    sigma_carb[i] = 0.01            
            
            #=================================================================================================
            #Carbon fit
            def fit_carb(x, A, sigma1, B, mu, sigma2, a):
                #remaining ice
                gaussian   = A*np.exp(-1.0*(x - peak_carb)**2 / (2*(sigma1**2)))
                gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
                poly       = a
                y          = gaussian + gaussian_2 + poly
                return y

            #Bounds: needed variables
            bf_strt_end_dist = np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - 
                                      bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])

            
            #initial guess + bounds
            carb_init_guess = [carb_init_guess_mltplr[0]*max_carb_new, \
                               carb_init_guess_mltplr[1]*stddev_carb_new, \
                               carb_init_guess_mltplr[2]*max_carb_new, \
                               carb_init_guess_mltplr[3]*peak_carb, \
                               carb_init_guess_mltplr[4]*stddev_carb_new, \
                               carb_init_guess_mltplr[5]*max_carb_new]      
            #Bounds of carb
            bounds_carb = ([bounds_carb_mltplr[0][0]*max_carb_new, bounds_carb_mltplr[0][1]*stddev_carb_new, \
                            bounds_carb_mltplr[0][2]*max_carb_new, bounds_carb_mltplr[0][3]*bf_strt_end_dist + peak_carb, 
                            bounds_carb_mltplr[0][4]*stddev_carb_new, \
                            bounds_carb_mltplr[0][5]*max_carb_new]
                           , \
                           [bounds_carb_mltplr[1][0]*max_carb_new, bounds_carb_mltplr[1][1]*stddev_carb_new, \
                            bounds_carb_mltplr[1][2]*max_carb_new, bounds_carb_mltplr[1][3]*bf_strt_end_dist + peak_carb, 
                            bounds_carb_mltplr[1][4]*stddev_carb_new, \
                            bounds_carb_mltplr[1][5]*max_carb_new])              
            
            try:
                #Curve_Fit carbon
                carb_popt, carb_pconv = scipy.optimize.curve_fit(fit_carb, xdata=bin_centers_new, ydata=hist_carb_new, \
                                                                 p0=carb_init_guess, bounds=bounds_carb, \
                                                                 sigma=sigma_carb, absolute_sigma=True)
            
            except RuntimeError as err1:
                cnt += 1
                err_dict["{:d}".format(cnt)] = [epho_bin, cos_bin, epho_bins[epho_bin], \
                                                cos_bins[cos_keys[epho_bin]][cos_bin], err1]
                #Set empty list
                carb_popt  = [0 for i in range(len(carb_init_guess))]
                carb_pconv = [0 for i in range(len(carb_init_guess))]
                
            #=================================================================================================
            #Into dicts
            carb_1epho_1cos_dict = {"bin_centers_new":bin_centers_new, "carb_bins":carb_bins, \
                                   "num_carb":num_carb, "peak_carb":peak_carb, \
                                   "carb_popt":carb_popt, "start_end":start_end, \
                                   "bin_size_mltp":bin_size_mltp, "carb_pconv":carb_pconv} 

            #Append
            carb_1epho_cos_dict["cos_{:07.3f}".format(cos_bins[cos_keys[epho_bin]][cos_bin])] = carb_1epho_1cos_dict

        #=================================================================================================
        #Save to dictionary
        carb_fit_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = carb_1epho_cos_dict
        
    return carb_fit_dict, err_dict, epho_bins, cos_bins

In [23]:
#Call fit_carb
carb_fit_dict, carb_err_dict, epho_bins, cos_bins = \
fit_carb_global(buta_dict=buta_dict, carb_dict=carb_dict, \
                bf_params_dict=bf_params_dict, \
                init_sf_2_dict=init_sf_2_dict, \
                epho_bins=epho_bins, cos_bins=cos_bins, \
                left_fit_range=2.5, right_fit_range=2.5, \
                carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.001], \
                bounds_carb_mltplr = ([0, 0, 0, -2, 0, 0], [1, 10, 1, 2, 10, 1]))

In [ ]:
# fit_result_sf_1 - Half epho bins (CHECK)
fit_result_sf_1, subplot = plt.subplots(10, 10, figsize=(30, 20))

#Spacing for subplots
fit_result_sf_1.subplots_adjust(hspace=0.5)
fit_result_sf_1.subplots_adjust(wspace=0.2)

#Keys for epho bins
epho_keys = sorted(buta_dict.keys())
cos_keys  = sorted(cos_bins.keys())

#Fit buta
def fit_buta(x, A, sigma1, B, mu, sigma2, a, peak_buta):
    gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
    gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
    poly  = a
    y     = gaus1 + gaus2 + poly
    return y

#Fit buta bound
def fit_buta_bound(x, A, sigma1, B, mu, sigma2, a, peak_buta):
    gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
    poly  = a
    y     = gaus2 + poly
    return y

#Carbon fit
def fit_carb(x, A, sigma1, B, mu, sigma2, a, peak_carb):
    #remaining ice
    gaussian   = A*np.exp(-1.0*(x - peak_carb)**2 / (2*(sigma1**2)))
    gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
    poly       = a
    y          = gaussian + gaussian_2 + poly
    return y

#Carbon fit
def fit_carb_bound(x, A, sigma1, B, mu, sigma2, a, peak_carb):
    #remaining ice
    gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
    poly       = a
    y          = gaussian_2 + poly
    return y

#Plot
for i in range(0, 10):
    
    #list to hold each cos_bin keys
    cos_keys_keys = list(buta_fit_dict[epho_keys[i]].keys())
    
    for j in range(10):
        try:
            #Ranges
            range_list = carb_dict[epho_keys[i]][j]["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

            #Use the bins defined in previous fitting procedure
            bins = buta_fit_dict[epho_keys[i]][cos_keys_keys[j]]["buta_bins"]        

            #Plot histograms
            subplot[i, j].hist(buta_dict[epho_keys[i]][j]["el_pc_mmsq_pi0"], label="butanol", \
                                                     bins=bins, alpha=0.7, range=(range_list[0], range_list[1]), \
                                                     color="#4C72B0")
            subplot[i, j].hist(carb_dict[epho_keys[i]][j]["el_pc_mmsq_pi0"], label="carbon", \
                                                     bins=bins, alpha=0.7, range=(range_list[0], range_list[1]), \
                                                     color="#55A868")        
            #lim
            subplot[i, j].set_xlim(range_list[0], range_list[1])

            #Results from fitting procedures
            peak_buta = buta_fit_dict[epho_keys[i]][cos_keys_keys[j]]["peak_buta"]
            peak_carb = carb_fit_dict[epho_keys[i]][cos_keys_keys[j]]["peak_carb"]
            buta_popt = buta_fit_dict[epho_keys[i]][cos_keys_keys[j]]["buta_popt"]
            carb_popt = carb_fit_dict[epho_keys[i]][cos_keys_keys[j]]["carb_popt"]
            bin_centers_new_buta = buta_fit_dict[epho_keys[i]][cos_keys_keys[j]]["bin_centers_new"]
            bin_centers_new_carb = carb_fit_dict[epho_keys[i]][cos_keys_keys[j]]["bin_centers_new"]

            #Plot fitting
            subplot[i, j].plot(bin_centers_new_buta, fit_buta(bin_centers_new_buta, *buta_popt, peak_buta), lw=1., \
                               color="red", alpha=0.7, ls="-")
            subplot[i, j].plot(bin_centers_new_buta, fit_buta_bound(bin_centers_new_buta, *buta_popt, peak_buta), lw=1., \
                               color="blue", alpha=0.7, ls="-")            
            subplot[i, j].plot(bin_centers_new_carb, fit_carb(bin_centers_new_carb, *carb_popt, peak_carb), lw=1., \
                               color="red", alpha=0.7, ls="--")
            subplot[i, j].plot(bin_centers_new_carb, fit_carb_bound(bin_centers_new_carb, *carb_popt, peak_carb), lw=1., \
                               color="blue", alpha=0.7, ls="--")            

        #For bins that fitting failed    
        except KeyError as err1:
            continue
            
#CROSS OUT bad bins
for i in range(0, 10):
    for j in range(10):
        #epho=0
        if (i==0 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=1
        elif (i==1 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==1 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=2
        elif (i==2 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==2 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=3
        elif (i==3 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==3 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==3 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=4
        elif (i==4 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==4 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==4 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==4 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=5
        elif (i==5 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==5 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==5 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==5 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=6
        elif (i==6 and j==5):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=7
        elif (i==7 and j==4):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==5):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==7 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=8
        elif (i==8 and j==0):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==8 and j==3):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==4):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==5):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==8 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=9
        elif (i==9 and j==0):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==9 and j==2):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==9 and j==3):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==4):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==5):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==9 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==9):
            subplot[i, j].set_facecolor("#C44E52")        

#grids
for i in subplot:
    for j in i:    
        #grids
        j.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
        j.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")
        
#Set COS Title
for i in range(10):
    for j in range(10):
        subplot[i, j].set_title("[{:.2f}, {:.2f}]".format(cos_bins[cos_keys[i]][j], cos_bins[cos_keys[i]][j+1]), \
                                fontsize=15)
#Set EPHO Title
for i in range(10):
    for j in range(1):
        subplot[i, j].set_ylabel("[{:.2f}, {:.2f}]".format(epho_bins[i], epho_bins[i+1]), fontsize=15)
        
    
#Label
fit_result_sf_1.text(0.9, 0.075, "Missing Mass Sq (${GeV}^2$)", ha="right", fontsize=40, weight="bold")
fit_result_sf_1.text(0.9, 0.925, "$\pi^0$ COS$\\theta_{cm}$", ha="right", fontsize=40, weight="bold")        
fit_result_sf_1.text(0.07, 0.5, "$E_{\gamma}$ (GeV)", va="center", rotation="vertical", fontsize=40, weight="bold")

#Legends
custom_label = [mpl.patches.Patch(facecolor="#4C72B0", alpha=0.7, label="Butanol"), \
                mpl.patches.Patch(facecolor="#55A868", alpha=0.7, label="Carbon"), \
                mpl.lines.Line2D([0], [0], color="red", alpha=0.7, linewidth=3, label="buta fit", linestyle="-"), \
                mpl.lines.Line2D([0], [0], color="blue", alpha=0.7, linewidth=3, label="buta bound fit", linestyle="-"), \
                mpl.lines.Line2D([0], [0], color="red", alpha=0.7, linewidth=3, label="carb fit", linestyle="--"), \
                mpl.lines.Line2D([0], [0], color="blue", alpha=0.7, linewidth=3, label="carb bound fit", linestyle="--"), \
               ]
fit_result_sf_1.legend(handles=custom_label, loc="upper left", bbox_to_anchor=(0.05, 0.9), ncol=6, fontsize=20)

plt.show()
# fit_result_sf_1.savefig("./run_period_1/fit_result_sf_1")

###### BUTA fit + CARB fit: Separately - INDIVIDUAL (MUST) 

In [24]:
#Buta fit individual
def fit_buta_indi(epho_bin=0, cos_bin=0, epho_bins=[], cos_bins={}, \
                  buta_dict={}, carb_dict={}, \
                  bf_params_dict={}, init_sf_2_dict={}, \
                  left_fit_range=2.5, right_fit_range=2.5, bin_size_mltp=1, \
                  buta_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.1], \
                  bounds_buta_mltplr = ([0.1, 0.1, 0.1, -1, 0.1, 0], [1, 2, 1, 1, 5, 1]), \
                  new_peak=0):
    
    #keys for epho_bins
    epho_keys     = sorted(carb_dict.keys())
    cos_keys      = sorted(cos_bins.keys())
    cos_keys_keys  = list(buta_fit_dict[epho_keys[epho_bin]].keys())

    #df declare for easy reading
    df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
    df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]

    #Remove Outliers - region selection
    range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

    #Remove Outliers - remove
    df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
    df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]
    
    #=================================================================================================
    #Into numpy
    butanol_data  = np.array(df_buta)
    carbon_data   = np.array(df_carb)
    #bin_size_mltp = 1

    #Bin Number Determine
    raw_sf = int(init_sf_2_dict[epho_keys[epho_bin]][cos_bin])
    if (raw_sf>=7):
        raw_sf = raw_sf - 2
    bin_num = int(bin_size_mltp*raw_sf*int(np.sqrt(len(carbon_data))))

    #hist_carbogram
    hist_carb, carb_bins = np.histogram(carbon_data, bins=bin_num)
    hist_buta, buta_bins = np.histogram(butanol_data, bins=carb_bins)

    #Bin Centers
    bin_centers = np.array([0.5*(buta_bins[i] + buta_bins[i+1]) for i in range(len(buta_bins)-1)])            

    #Stats BUTA
    #    -For high energy set peak_buta=0.035
    peak_buta   = 0.035
    #peak_buta   = buta_bins[np.argmax(hist_buta)]
    mean_buta   = np.average(bin_centers, weights=hist_buta)
    var_buta    = np.average((bin_centers-mean_buta)**2, weights=hist_buta)
    stddev_buta = np.sqrt(var_buta)
    num_buta    = len(butanol_data)
    max_buta    = np.max(hist_buta)    
    
    #New peak
    if (new_peak != 0):
        peak_buta = new_peak
    
    #=================================================================================================
    #Fitting range
    #left_fit_range  = 2.5
    #right_fit_range = 2.5
    start_end = [(peak_buta - left_fit_range*np.abs(peak_buta - bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])), \
                 (peak_buta + right_fit_range*np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - peak_buta))]

    #Prevent fits to be delcared out of carbon dist. range
    carb_lims = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
    if (start_end[0] < carb_lims[0]):
        start_end[0] = carb_lims[0]
    if (start_end[1] > carb_lims[1]):
        start_end[1] = carb_lims[1]

    #=================================================================================================
    #Bins within fitting range
    boolean_arry_bins = (bin_centers > start_end[0]) & (bin_centers < start_end[1])
    bin_centers_new   = bin_centers[boolean_arry_bins]

    #=================================================================================================
    #Subset of histogram within FITTING RANGE
    #    -this histogram will be used for final chisq minimization fitting!!!
    hist_buta_new = hist_buta[boolean_arry_bins]

    #Stats BUTA new
    mean_buta_new   = np.average(bin_centers_new, weights=hist_buta_new)
    var_buta_new    = np.average((bin_centers_new-mean_buta_new)**2, weights=hist_buta_new)
    stddev_buta_new = np.sqrt(var_buta_new)
    max_buta_new    = np.max(hist_buta_new)        
    sigma_buta      = np.sqrt(hist_buta_new)

    #Dont want any entries in sigma to be zero -> division by 0 error
    for i in range(len(sigma_buta)):
        if (sigma_buta[i] == 0):
            sigma_buta[i] = 0.01
            
    #=================================================================================================
    #Fit buta
    def fit_buta(x, A, sigma1, B, mu, sigma2, a):
        gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus1 + gaus2 + poly
        return y

    #Bounds: needed variables
    bf_strt_end_dist = np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - 
                              bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])         

    #initial guess + bounds
    buta_init_guess = [buta_init_guess_mltplr[0]*max_buta_new, \
                       buta_init_guess_mltplr[1]*stddev_buta_new, \
                       buta_init_guess_mltplr[2]*max_buta_new, \
                       buta_init_guess_mltplr[3]*peak_buta, \
                       buta_init_guess_mltplr[4]*stddev_buta_new, \
                       buta_init_guess_mltplr[5]*max_buta_new]        
    #Bounds of carb
    bounds_buta = ([bounds_buta_mltplr[0][0]*max_buta_new, bounds_buta_mltplr[0][1]*stddev_buta_new, \
                    bounds_buta_mltplr[0][2]*max_buta_new, bounds_buta_mltplr[0][3]*bf_strt_end_dist + peak_buta, 
                    bounds_buta_mltplr[0][4]*stddev_buta_new, \
                    bounds_buta_mltplr[0][5]*max_buta_new]
                   , \
                   [bounds_buta_mltplr[1][0]*max_buta_new, bounds_buta_mltplr[1][1]*stddev_buta_new, \
                    bounds_buta_mltplr[1][2]*max_buta_new, bounds_buta_mltplr[1][3]*bf_strt_end_dist + peak_buta, 
                    bounds_buta_mltplr[1][4]*stddev_buta_new, \
                    bounds_buta_mltplr[1][5]*max_buta_new])

    #Curve_Fit
#     buta_popt, buta_pconv = scipy.optimize.curve_fit(fit_buta, xdata=bin_centers_new, \
#                                                      ydata=hist_buta_new, \
#                                                      p0=buta_init_guess, bounds=bounds_buta)
    
    buta_popt, buta_pconv = scipy.optimize.curve_fit(fit_buta, xdata=bin_centers_new, \
                                                     ydata=hist_buta_new, \
                                                     p0=buta_init_guess, bounds=bounds_buta, \
                                                     sigma=sigma_buta, absolute_sigma=True)

    
    #Store redone fits into buta_fit_dict
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"] = bin_centers_new
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bins"]       = buta_bins
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["num_buta"]        = num_buta
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_buta"]       = peak_buta
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]       = buta_popt
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_pconv"]      = buta_pconv
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["start_end"]       = start_end
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_size_mltp"]   = bin_size_mltp

In [25]:
#Buta fit individual
def fit_carb_indi(epho_bin=0, cos_bin=0, epho_bins=[], cos_bins={}, \
                  buta_dict={}, carb_dict={}, \
                  bf_params_dict={}, init_sf_2_dict={}, \
                  left_fit_range=2.5, right_fit_range=2.5, bin_size_mltp=1, \
                  carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 1], \
                  bounds_carb_mltplr = ([0, 0, 0, -2, 0, 0], [1, 10, 1, 2, 10, np.inf]), \
                  new_peak=0):
    
    #keys for epho_bins
    epho_keys     = sorted(carb_dict.keys())
    cos_keys      = sorted(cos_bins.keys())
    cos_keys_keys = list(carb_fit_dict[epho_keys[epho_bin]].keys())
    
    #df declare for easy reading
    df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
    df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]

    #=================================================================================================
    #Remove Outliers - region selection
    range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

    #Remove Outliers - remove
    df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
    df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]
    
    #=================================================================================================
    #Into numpy
    butanol_data  = np.array(df_buta)
    carbon_data   = np.array(df_carb)
    #bin_size_mltp = 1
    
    #Bin Number Determine
    raw_sf = int(init_sf_2_dict[epho_keys[epho_bin]][cos_bin])
    if (raw_sf>=7):
        raw_sf = raw_sf - 2
    bin_num = int(bin_size_mltp*raw_sf*int(np.sqrt(len(carbon_data))))

    #hist_carbogram
    hist_carb, carb_bins = np.histogram(carbon_data, bins=bin_num)
    hist_buta, buta_bins = np.histogram(butanol_data, bins=carb_bins)

    #Bin Centers
    bin_centers = np.array([0.5*(buta_bins[i] + buta_bins[i+1]) for i in range(len(buta_bins)-1)])            

    #Stats BUTA
    #    -For high energy, set peak_buta manually
    peak_buta   = 0.035
    #peak_buta   = buta_bins[np.argmax(hist_buta)]
    mean_buta   = np.average(bin_centers, weights=hist_buta)
    var_buta    = np.average((bin_centers-mean_buta)**2, weights=hist_buta)
    stddev_buta = np.sqrt(var_buta)
    num_buta    = len(butanol_data)
    max_buta    = np.max(hist_buta)    
    #Stats CARB
    peak_carb   = buta_bins[np.argmax(hist_carb)]
    mean_carb   = np.average(bin_centers, weights=hist_carb)
    var_carb    = np.average((bin_centers-mean_carb)**2, weights=hist_carb)
    stddev_carb = np.sqrt(var_carb)
    num_carb    = len(carbon_data)
    max_carb    = np.max(hist_carb) 
    
    #New peak
    if (new_peak != 0):
        peak_buta = new_peak
        peak_carb = new_peak
    
    #=================================================================================================
    #Fitting range
    #left_fit_range  = 2.5
    #right_fit_range = 2.5
    start_end = [(peak_buta - left_fit_range*np.abs(peak_buta - bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])), \
                 (peak_buta + right_fit_range*np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - peak_buta))]

    #Prevent fits to be delcared out of carbon dist. range
    carb_lims = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
    if (start_end[0] < carb_lims[0]):
        start_end[0] = carb_lims[0]
    if (start_end[1] > carb_lims[1]):
        start_end[1] = carb_lims[1]

    #=================================================================================================
    #Bins within fitting range
    boolean_arry_bins = (bin_centers > start_end[0]) & (bin_centers < start_end[1])
    bin_centers_new   = bin_centers[boolean_arry_bins]

    #=================================================================================================
    #Subset of histogram within FITTING RANGE
    hist_carb_new = hist_carb[boolean_arry_bins]

    #Stats CARB new
    mean_carb_new   = np.average(bin_centers_new, weights=hist_carb_new)
    var_carb_new    = np.average((bin_centers_new-mean_carb_new)**2, weights=hist_carb_new)
    stddev_carb_new = np.sqrt(var_carb_new)
    max_carb_new    = np.max(hist_carb_new)       
    sigma_carb      = np.sqrt(hist_carb_new)

    #Dont want any entries in sigma to be zero -> division by 0 error
    for i in range(len(sigma_carb)):
        if (sigma_carb[i] == 0):
            sigma_carb[i] = 0.01
    
    #=================================================================================================
    #Carbon fit
    def fit_carb(x, A, sigma1, B, mu, sigma2, a):
        #remaining ice
        gaussian   = A*np.exp(-1.0*(x - peak_carb)**2 / (2*(sigma1**2)))
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian + gaussian_2 + poly
        return y

    #Bounds: needed variables
    bf_strt_end_dist = np.abs(bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1] - 
                              bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0])


    #initial guess + bounds
    carb_init_guess = [carb_init_guess_mltplr[0]*max_carb_new, \
                       carb_init_guess_mltplr[1]*stddev_carb_new, \
                       carb_init_guess_mltplr[2]*max_carb_new, \
                       carb_init_guess_mltplr[3]*peak_carb, \
                       carb_init_guess_mltplr[4]*stddev_carb_new, \
                       carb_init_guess_mltplr[5]*max_carb_new]        
    #Bounds of carb
    bounds_carb = ([bounds_carb_mltplr[0][0]*max_carb_new, bounds_carb_mltplr[0][1]*stddev_carb_new, \
                    bounds_carb_mltplr[0][2]*max_carb_new, bounds_carb_mltplr[0][3]*bf_strt_end_dist + peak_carb, 
                    bounds_carb_mltplr[0][4]*stddev_carb_new, \
                    bounds_carb_mltplr[0][5]*max_carb_new]
                   , \
                   [bounds_carb_mltplr[1][0]*max_carb_new, bounds_carb_mltplr[1][1]*stddev_carb_new, \
                    bounds_carb_mltplr[1][2]*max_carb_new, bounds_carb_mltplr[1][3]*bf_strt_end_dist + peak_carb, 
                    bounds_carb_mltplr[1][4]*stddev_carb_new, \
                    bounds_carb_mltplr[1][5]*max_carb_new])  

    #Curve_Fit
    carb_popt, carb_pconv = scipy.optimize.curve_fit(fit_carb, xdata=bin_centers_new, \
                                                     ydata=hist_carb_new, \
                                                     p0=carb_init_guess, bounds=bounds_carb, \
                                                     sigma=sigma_carb, absolute_sigma=True)

    #Store redone fits into carb_fit_dict
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"] = bin_centers_new
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_bins"]       = carb_bins
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["num_carb"]        = num_carb
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_carb"]       = peak_carb
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_popt"]       = carb_popt
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_pconv"]      = carb_pconv
    carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["start_end"]       = start_end
    buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_size_mltp"]   = bin_size_mltp    

In [26]:
#Indi fit plot
def plot_indi(epho_bin=0, cos_bin=0, bf_params_dict={}, epho_bins=[], cos_bins={}, \
              buta_fit_dict={}, carb_fit_dict={}, buta_dict={}, carb_dict={}):
    
    #keys
    epho_keys     = sorted(carb_dict.keys())
    cos_keys      = sorted(cos_bins.keys())
    cos_keys_keys = list(carb_fit_dict[epho_keys[epho_bin]].keys())    
    
    #Params
    buta_start_end_tpl = bf_params_dict[epho_keys[epho_bin]][cos_bin][6]
    peak_buta          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_buta"]
    peak_carb          = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_carb"]
    buta_popt          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]
    carb_popt          = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_popt"]
    bin_centers_new_buta    = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"]    
    bin_centers_new_carb    = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"]    
    
    #===================================================================================================
    #Fit buta
    def fit_buta(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus1 + gaus2 + poly
        return y

    #Fit buta bound
    def fit_buta_bound(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus2 + poly
        return y

    #Carbon fit
    def fit_carb(x, A, sigma1, B, mu, sigma2, a, peak_carb):
        #remaining ice
        gaussian   = A*np.exp(-1.0*(x - peak_carb)**2 / (2*(sigma1**2)))
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian + gaussian_2 + poly
        return y

    #Carbon fit
    def fit_carb_bound(x, A, sigma1, B, mu, sigma2, a, peak_carb):
        #remaining ice
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian_2 + poly
        return y
    #===================================================================================================
    #Plot
    fit_redo = plt.figure(figsize=(10,6))
    
    #Ranges
    range_list = carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

    #Use the bins defined in previous fitting procedure
    bins = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bins"]        
    
    #Histograms
    plt.hist(buta_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="butanol", \
             bins=bins, alpha=0.7, range=(range_list[0], range_list[1]), \
             color="#4C72B0")
    plt.hist(carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="carbon", \
             bins=bins, alpha=0.9, range=(range_list[0], range_list[1]), \
             color="#55A868")
    
    #Plot fitting
    plt.plot(bin_centers_new_buta, fit_buta(bin_centers_new_buta, *buta_popt, peak_buta), lw=1., \
             color="red", alpha=0.7, ls="-", label="buta fit")
    plt.plot(bin_centers_new_buta, fit_buta_bound(bin_centers_new_buta, *buta_popt, peak_buta), lw=1., \
             color="blue", alpha=0.7, ls="-", label="buta bound fit")            
    plt.plot(bin_centers_new_carb, fit_carb(bin_centers_new_carb, *carb_popt, peak_carb), lw=1., \
             color="red", alpha=0.7, ls="--", label="carb fit")
    plt.plot(bin_centers_new_carb, fit_carb_bound(bin_centers_new_carb, *carb_popt, peak_carb), lw=1., \
             color="blue", alpha=0.7, ls="--", label="carb bound_fit")            
    
    ##Butanol free nucleon sharp peak start/end positions
    #plt.axvline(buta_start_end_tpl[0], linestyle="--", color="red", alpha=0.5, label="Buta Free-Nucl Peak Start/End pos")
    #plt.axvline(buta_start_end_tpl[1], linestyle="--", color="red", alpha=0.5)

    #===================================================================================================
    #Labels, ticks, lim
    plt.xlabel("$M^2_X$. $[GeV^2/c^4]$", fontsize=15)
    plt.ylabel("Counts", fontsize=15)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)
    plt.xlim(range_list[0], range_list[1])

    #Grids
    plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
    plt.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")

    ##Texts
    #plt.text(0.03, 0.53, " Free-nucl peak start {:>14.3f}\n Free-nucl peak end {:>14.3f}" \
    #         .format(buta_start_end_tpl[0], buta_start_end_tpl[1]), \
    #         fontsize=10, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)

    #Set Title
    plt.title("$E_{{\gamma}} \in [ {:.2f}, {:.2f}]$ \t $cos\\theta_{{cm}} \in [ {:.2f}, {:.2f}]$" \
              .format(epho_bins[epho_bin], epho_bins[epho_bin+1], cos_bins[cos_keys[epho_bin]][cos_bin], \
                      cos_bins[cos_keys[epho_bin]][cos_bin+1]),\
              fontsize=15)
        
    #Legends
    plt.legend(fontsize=10, loc="upper left")

    plt.show()
    fit_redo.savefig("./run_period_1/sf_fit_redo_{:d}_{:d}".format(epho_bin, cos_bin))

In [27]:
# ##COS=0

epho_bin, cos_bin = 0, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.7, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0.5, 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 1, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.7, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1.5, right_fit_range=1., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0., 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 2, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=1.5, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1.5, right_fit_range=1., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0., 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [0.1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 3, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=1.5, bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.25, bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0., 0.5, 0., 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [0.5, 1, 0.25, 0.1, 1, 0.5]), \
              new_peak=0.025)

epho_bin, cos_bin = 4, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=1.5, bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1.5, right_fit_range=1.5, bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0., 0.5, 0., 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [0.1, 1, 0.5, 0.1, 1, 0.5]), \
              new_peak=0.025)

epho_bin, cos_bin = 5, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=1.5, bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=1.5, right_fit_range=1.5, bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0., 0.5, 0., 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [0.1, 1, 0.5, 0.1, 1, 0.5]), \
              new_peak=0.025)

epho_bin, cos_bin = 6, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=1.5, bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=1.5, bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0., 0.5, 0., 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [0.1, 1, 0.5, 0.1, 1, 0.5]), \
              new_peak=0.025)

epho_bin, cos_bin = 7, 0
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=2., bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.4, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2.5, right_fit_range=1.5, bin_size_mltp=.5, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0., 0.5, 0., 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [0.1, 1, 0.5, 0.1, 1, 0.5]), \
              new_peak=0.025)


In [28]:
# ##COS=1

epho_bin, cos_bin = 0, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.7, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.5, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0.5, 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 2, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.7, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 3, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.5, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 4, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.6, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.25, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 5, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.25, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 6, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 7, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 8, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 9, 1
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0., 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [0.1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

In [29]:
# # ##COS=2

epho_bin, cos_bin = 2, 2
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.025)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 5, 2
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=1.25, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

In [30]:
# # ##COS=5

epho_bin, cos_bin = 4, 5
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3., right_fit_range=2.25, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 5, 5
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=2., right_fit_range=2.25, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

In [31]:
# # ##COS=6

epho_bin, cos_bin = 1, 6
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3., right_fit_range=2.25, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)

epho_bin, cos_bin = 2, 6
#Call buta indi fit
fit_buta_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3.5, right_fit_range=2., bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              buta_init_guess_mltplr = [0.9, 0.5, 0.5, 1, 1, 0.1], \
              bounds_buta_mltplr = ([.5, 0.1, 0.1, -1, 0.1, 0], [1., 0.5, 1, 1, 5, 0.5]), \
              new_peak=0.035)
#Call carb indi fit
fit_carb_indi(epho_bin, cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              left_fit_range=3., right_fit_range=2.25, bin_size_mltp=.55, \
              #[A, sigma1, B, mu, sigma2, a]
              carb_init_guess_mltplr = [0.5, 0.5, 0.5, 1, 1, 0.5], \
              bounds_carb_mltplr = ([0., 0, 0, -1, 0, 0], [1, 1, 1, 0.1, 1, 1]), \
              new_peak=0.035)


In [ ]:
## Plot bin
epho_bin, cos_bin = 0, 8

#Check indi fits
plot_indi(epho_bin=epho_bin, cos_bin=cos_bin, bf_params_dict=bf_params_dict, \
          epho_bins=epho_bins, cos_bins=cos_bins, \
          buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
          buta_dict=buta_dict, carb_dict=carb_dict)

######  CHISQ min for final SF - GLOBAL (MUST)

* Chisq eq:
$$ \chi^2 =  \sum_i^N \frac{\left(B_{bg}(i) - sf \times C_{bg}(i)\right)^2}{ \left(B_{bg}(i) - sf^2 \times C_{bg}(i)\right) } $$
    * N = # of bins
    * i = each bin
    * $B_{bg}$ is fit function for butanol bound nucleon events (gaussian) + pedastal (constant)
    * $C_{bg}$ is fit function for carbon bound nucleon events (gaussian) + pedastal (constant)
    * sf is the scale factor we are solving for via minimizing the chisq

In [ ]:
"""
#=====================================================================================
#METHOD 2
    
-Fit buta (signal+background) and carb (sig + back) separately

-Chisq minimization: buta backgroun - sf*(fit_carb_background): to compute scale factor
    -Use only carbon background fitted fn in chisq
    -Use only buta background fitted fn in chisq
#=====================================================================================
#Stat Error

-Scale Factor Statistical Uncertainty
    0. Error progation!
        -since fitting has impact on sf in a way that we design the chisq:
            -d(sf)/d(B(i))
            -So, inver the chisq eq, so that sf in a function of chisq: integration
    1. Since I use FIT functions to compute sf, I should also include fitting uncertainties to sf stat error
        -Do this part when fitting is improved
    2. Since I used chisq minimization to compute sf, I need the error of minimization!
        -IChisq +/-1 methods as shown in [Brandt9.8], 
            -convert summation into INTEGRATION!! Integration to get a chisq in functiof of sf
                -set chisq +/- 1 and solve for sf (use scipy fsolve)

"""

In [32]:
#sf global
def sf_final(buta_dict={}, carb_dict={}, bf_params_dict={}, init_sf_2_dict={}, \
             epho_bins=[], cos_bins={}, \
             buta_fit_dict={}, carb_fit_dict={}, \
             chi_init_mult=[1], bounds_chi_mult=[0, 2]):

    #Dict to store final sf
    sf_dict = {}
    
    #keys for epho_bins
    epho_keys = sorted(cos_bins.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
        
        #Dict to store dict or whole cos
        sf_1epho_cos_dict = {}
    
        #keys for each cos_bin to access fit params
        cos_keys_keys  = list(buta_fit_dict[epho_keys[epho_bin]].keys())
    
        #Each epho_bin has diff cos bins defined
        for cos_bin in range(len(cos_bins[j])-1):

            #df declare for easy reading
            df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
            df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]
        
            #Remove Outliers - region selection
            range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            
            #Remove Outliers - remove
            df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
            df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

            #=================================================================================================
            #Into numpy
            butanol_data = np.array(df_buta)
            carbon_data  = np.array(df_carb)

            #Bin Number Determine
            raw_sf = int(init_sf_2_dict[epho_keys[epho_bin]][cos_bin])
            if (raw_sf>=7):
                raw_sf = raw_sf - 2
            bin_size_mltp = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_size_mltp"]    
            bin_num       = int(bin_size_mltp*raw_sf*int(np.sqrt(len(carbon_data))))
            
            #hist_carbogram
            hist_carb, carb_bins = np.histogram(carbon_data, bins=bin_num)
            hist_buta, buta_bins = np.histogram(butanol_data, bins=carb_bins)

            #Bin Centers
            bin_centers = np.array([0.5*(buta_bins[i] + buta_bins[i+1]) for i in range(len(buta_bins)-1)])            
            
            #Stats BUTA
            peak_buta   = buta_bins[np.argmax(hist_buta)]
                       
            #=================================================================================================
            #Fitting range            
            start_end = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["start_end"]

            #=================================================================================================
            #Bins within fitting range
            boolean_arry_bins = (bin_centers > start_end[0]) & (bin_centers < start_end[1])
            bin_centers_new   = bin_centers[boolean_arry_bins]

            #=================================================================================================
            #Subset of histogram within FITTING RANGE
            #    -this histogram will be used for final chisq minimization fitting!!!
            hist_buta_new = hist_buta[boolean_arry_bins]
            hist_carb_new = hist_carb[boolean_arry_bins]

            #Stats BUTA new
            mean_buta_new   = np.average(bin_centers_new, weights=hist_buta_new)
            var_buta_new    = np.average((bin_centers_new-mean_buta_new)**2, weights=hist_buta_new)
            stddev_buta_new = np.sqrt(var_buta_new)
            max_buta_new    = np.max(hist_buta_new)    
            sigma_buta      = np.sqrt(hist_buta_new)
            #Stats CARB new
            sigma_carb      = np.sqrt(hist_carb_new)
        
            #Dont want any entries in sigma to be zero -> division by 0 error
            for i in range(len(sigma_buta)):
                if (sigma_buta[i] == 0):
                    sigma_buta[i] = 0.01
            for i in range(len(sigma_carb)):
                if (sigma_carb[i] == 0):
                    sigma_carb[i] = 0.01

            #=================================================================================================
            #Params from fitting 
            buta_popt = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]
            carb_popt = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_popt"]
                    
            #=================================================================================================
            #Final chisq Fitting
            def chisq(sol, x=np.array([]), \
                      buta_popt=np.array([]), carb_popt=np.array([]), \
                      #sigma_buta=np.array([]), sigma_carb=np.array([]), \
                      hist_buta=np.array([]), hist_carb=np.array([])):
                #Buta bacground
                gaus_buta_back = buta_popt[2]*np.exp(-1.0*(x - buta_popt[3])**2 / (2*(buta_popt[4]**2)))
                poly_buta      = buta_popt[5]
                buta           = gaus_buta_back + poly_buta
                
                #Carb background
                gaus_carb_back = carb_popt[2]*np.exp(-1.0*(x - carb_popt[3])**2 / (2*(carb_popt[4]**2)))
                poly_carb      = carb_popt[5]
                carb           = gaus_carb_back + poly_carb
                
                #each bins
                chisq_list = []
                for i in range(len(x)):
                    #nume
                    nume = pow(buta[i] - sol*carb[i], 2)
                    #deno
                    #deno = pow(sigma_buta[i] + sigma_carb[i], 2)
                    deno = buta[i] + pow(sol, 2)*carb[i]
                    
                    #chisq/bin
                    chisq_bin = nume / deno
                    #chisq_bin = nume

                    #append
                    chisq_list.append(chisq_bin)
                #sum chisq/bin
                chisq = np.sum(chisq_list)
                return chisq            
            
            #chisq initial guess for scale factor
            chi_init_guess = [chi_init_mult[0]*raw_sf]
            
            #chisq bounds for scale factor
            bounds_chi = [(bounds_chi_mult[0]*raw_sf, bounds_chi_mult[1]*raw_sf)]
            
            #Minimize
            popt = scipy.optimize.minimize(chisq, x0=chi_init_guess, bounds=bounds_chi, \
                                           args=(bin_centers_new, buta_popt, carb_popt, \
                                                 hist_buta_new, hist_carb_new), \
                                           method='L-BFGS-B')

            #=================================================================================================
            #Goodness of fit 
            dof       = (len(bin_centers_new) - 1 ) - len(popt.x)
            chisq_per = np.abs(popt.fun) / dof
            
            #=================================================================================================
            #SF Stat uncertainties (NEED WORK)
            sf_stat_err_fit      = 0
            sf_stat_err_chisq    = 0
            
            sf_stat_err = sf_stat_err_fit + sf_stat_err_chisq

            #=================================================================================================
            sf_1epho_1cos_dict = {"popt.x": popt.x[0], "popt.fun":popt.fun, "chisq_per":chisq_per, \
                                  "popt":popt, "sf_stat_err":sf_stat_err}
            
            #Append
            sf_1epho_cos_dict["cos_{:07.3f}".format(cos_bins[cos_keys[epho_bin]][cos_bin])] = sf_1epho_1cos_dict
        
        #=================================================================================================
        #Save to dictionary
        sf_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = sf_1epho_cos_dict
            
    return sf_dict, epho_bins, cos_bins                

In [33]:
#Discard bad bin
def sf_discard_bad(epho_bins=[], cos_bins={}, sf_dict={}):
    
    #keys for epho_bins
    epho_keys = sorted(cos_bins.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):

        #keys for each cos_bin to access fit params
        cos_keys_keys  = list(sf_dict[epho_keys[epho_bin]].keys())
        
        #fn to call within next loop for discarding bins
        def discard_bad_bins(epho_bin=0, cos_bin=0):
            sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.x"] = np.nan
            sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.fun"]  = np.nan
            sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["chisq_per"] = np.nan

        #=============================================================================================
        #Each epho_bin has diff cos bins defined
        for cos_bin in range(len(cos_bins[j])-1):
            #epho=0
            if (epho_bin==0 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==0 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=1
            elif (epho_bin==1 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==1 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==1 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)
                
            #epho=2
            elif (epho_bin==2 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==2 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==2 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)
                
            #epho=3
            elif (epho_bin==3 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==3 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==3 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==3 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)
                
            #epho=4
            elif (epho_bin==4 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==4 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==4 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==4 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==4 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)

                
            #epho=5
            elif (epho_bin==5 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==5 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==5 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==5 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==5 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=6
            elif (epho_bin==6 and cos_bin==5):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==6 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==6 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==6 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==6 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==6 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)
            
            #epho=7
            elif (epho_bin==7 and cos_bin==4):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==7 and cos_bin==5):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==7 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==7 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==7 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==7 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==7 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)
                
            #epho=8
            elif (epho_bin==8 and cos_bin==0):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==3):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==4):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==5):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)
                
            #epho=9
            elif (epho_bin==9 and cos_bin==0):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==2):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==3):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==4):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==5):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)                

In [ ]:
#call sf_dict global
sf_dict, epho_bins, cos_bins = sf_final(buta_dict=buta_dict, carb_dict=carb_dict, \
                                        bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
                                        epho_bins=epho_bins, cos_bins=cos_bins, \
                                        buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
                                        chi_init_mult=[1], bounds_chi_mult=[0.1, 2.])

#Discard bad bins
sf_discard_bad(epho_bins, cos_bins, sf_dict)

In [ ]:
# sf_1 - Half epho bins (CHECK)
sf_1, subplot = plt.subplots(10, 10, figsize=(30, 20))

#Spacing for subplots
sf_1.subplots_adjust(hspace=0.5)
sf_1.subplots_adjust(wspace=0.2)

#Keys for epho bins
epho_keys = sorted(buta_dict.keys())
cos_keys  = sorted(cos_bins.keys())

#Fit buta
def fit_buta(x, A, sigma1, B, mu, sigma2, a, peak_buta):
    gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
    gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
    poly  = a
    y     = gaus1 + gaus2 + poly
    return y

#Fit buta bound
def fit_buta_bound(x, A, sigma1, B, mu, sigma2, a, peak_buta):
    gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
    poly  = a
    y     = gaus2 + poly
    return y

#Carbon fit
def fit_carb(x, A, sigma1, B, mu, sigma2, a, peak_carb):
    #remaining ice
    gaussian   = A*np.exp(-1.0*(x - peak_carb)**2 / (2*(sigma1**2)))
    gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
    poly       = a
    y          = gaussian + gaussian_2 + poly
    return y

#Carbon fit
def fit_carb_bound(x, A, sigma1, B, mu, sigma2, a, peak_carb):
    #remaining ice
    gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
    poly       = a
    y          = gaussian_2 + poly
    return y

#Plot
for i in range(0, 10):
    
    #list to hold each cos_bin keys
    cos_keys_keys = list(buta_fit_dict[epho_keys[i]].keys())
    
    for j in range(10):
        try:
            #Ranges
            range_list = carb_dict[epho_keys[i]][j]["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

            #Use the bins defined in previous fitting procedure
            bins = buta_fit_dict[epho_keys[i]][cos_keys_keys[j]]["buta_bins"]        

            #Plot histograms
            subplot[i, j].hist(buta_dict[epho_keys[i]][j]["el_pc_mmsq_pi0"], label="butanol", \
                                                     bins=bins, alpha=0.7, range=(range_list[0], range_list[1]), \
                                                     color="#4C72B0")
            subplot[i, j].hist(carb_dict[epho_keys[i]][j]["el_pc_mmsq_pi0"], label="carbon", \
                                                     bins=bins, alpha=0.7, range=(range_list[0], range_list[1]), \
                                                     color="#55A868")        
            #SF*carb
            sf                   = sf_dict[epho_keys[i]][cos_keys_keys[j]]["popt.x"]
            df_carb              = carb_dict[epho_keys[i]][j]
            hist_carb, bins_carb = np.histogram(np.array(df_carb["el_pc_mmsq_pi0"]), bins=bins)     
            hist_sc,   bins_sc   = sf*hist_carb, bins_carb            
            mids                 = [0.5*(bins[i] + bins[i+1]) for i in range(len(bins)-1)]
            
            #Plot Scaled Carbon
            subplot[i, j].bar(mids, hist_sc, width=bins_sc[1]-bins_sc[0], alpha=0.3, color="#C44E52", \
                              label="Scaled Carbon")

            #lim
            subplot[i, j].set_xlim(range_list[0], range_list[1])

            #Results from fitting procedures
            peak_buta = buta_fit_dict[epho_keys[i]][cos_keys_keys[j]]["peak_buta"]
            peak_carb = carb_fit_dict[epho_keys[i]][cos_keys_keys[j]]["peak_carb"]
            buta_popt = buta_fit_dict[epho_keys[i]][cos_keys_keys[j]]["buta_popt"]
            carb_popt = carb_fit_dict[epho_keys[i]][cos_keys_keys[j]]["carb_popt"]
            bin_centers_new_buta = buta_fit_dict[epho_keys[i]][cos_keys_keys[j]]["bin_centers_new"]
            bin_centers_new_carb = carb_fit_dict[epho_keys[i]][cos_keys_keys[j]]["bin_centers_new"]

            #Plot fitting
            subplot[i, j].plot(bin_centers_new_buta, fit_buta(bin_centers_new_buta, *buta_popt, peak_buta), lw=1., \
                               color="red", alpha=0.7, ls="-")
            subplot[i, j].plot(bin_centers_new_buta, fit_buta_bound(bin_centers_new_buta, *buta_popt, peak_buta), lw=1., \
                               color="blue", alpha=0.7, ls="-")            
            subplot[i, j].plot(bin_centers_new_carb, fit_carb(bin_centers_new_carb, *carb_popt, peak_carb), lw=1., \
                               color="red", alpha=0.7, ls="--")
            subplot[i, j].plot(bin_centers_new_carb, fit_carb_bound(bin_centers_new_carb, *carb_popt, peak_carb), lw=1., \
                               color="blue", alpha=0.7, ls="--")            

        #For bins that fitting failed    
        except KeyError as err1:
            continue

#CROSS OUT bad bins
for i in range(0, 10):
    for j in range(10):
        #epho=0
        if (i==0 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=1
        elif (i==1 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==1 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=2
        elif (i==2 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==2 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=3
        elif (i==3 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==3 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==3 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=4
        elif (i==4 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==4 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==4 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==4 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=5
        elif (i==5 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==5 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==5 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==5 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=6
        elif (i==6 and j==5):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=7
        elif (i==7 and j==4):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==5):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==7 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=8
        elif (i==8 and j==0):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==8 and j==3):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==4):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==5):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==8 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=9
        elif (i==9 and j==0):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==9 and j==2):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==9 and j==3):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==4):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==5):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==9 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==9):
            subplot[i, j].set_facecolor("#C44E52")            

#grids
for i in subplot:
    for j in i:    
        #grids
        j.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
        j.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")
        
#Set COS Title
for i in range(10):
    for j in range(10):
        subplot[i, j].set_title("[{:.2f}, {:.2f}]".format(cos_bins[cos_keys[i]][j], cos_bins[cos_keys[i]][j+1]), \
                                fontsize=15)
#Set EPHO Title
for i in range(10):
    for j in range(1):
        subplot[i, j].set_ylabel("[{:.2f}, {:.2f}]".format(epho_bins[i], epho_bins[i+1]), fontsize=15)
        
    
#Label
sf_1.text(0.9, 0.075, "Missing Mass Sq (${GeV}^2$)", ha="right", fontsize=40, weight="bold")
sf_1.text(0.9, 0.925, "$\pi^0$ COS$\\theta_{cm}$", ha="right", fontsize=40, weight="bold")        
sf_1.text(0.07, 0.5, "$E_{\gamma}$ (GeV)", va="center", rotation="vertical", fontsize=40, weight="bold")

#Legends
custom_label = [mpl.patches.Patch(facecolor="#4C72B0", alpha=0.7, label="Butanol"), \
                mpl.patches.Patch(facecolor="#55A868", alpha=0.7, label="Carbon"), \
                mpl.patches.Patch(facecolor="#C44E52", alpha=0.3, label="SF*Carbon"), \
                mpl.lines.Line2D([0], [0], color="red", alpha=0.7, linewidth=3, label="buta fit", linestyle="-"), \
                mpl.lines.Line2D([0], [0], color="blue", alpha=0.7, linewidth=3, label="buta bound fit", linestyle="-"), \
                mpl.lines.Line2D([0], [0], color="red", alpha=0.7, linewidth=3, label="carb fit", linestyle="--"), \
                mpl.lines.Line2D([0], [0], color="blue", alpha=0.7, linewidth=3, label="carb bound fit", linestyle="--"), \
                ]
sf_1.legend(handles=custom_label, loc="upper left", bbox_to_anchor=(0.05, 0.9), ncol=7, fontsize=20)

plt.show()
# sf_1.savefig("./run_period_1/sf_1")

In [ ]:
#Plot sf (CHECK)
sf_plot, subplot = plt.subplots(2, 5, figsize=(30, 10))

#epho_keys
epho_keys = sorted(cos_bins.keys())
cos_keys  = sorted(cos_bins.keys())

#Cos_bins_cnt in 2d list
cos_bins_cnts = []
for i in sorted(cos_bins.keys()):
    cos_bins_cnts_i = []
    for j in range(len(cos_bins[i])-1):
        cnt_i           = 0.5*(cos_bins[i][j] + cos_bins[i][j+1])
        cnt_i           = round(cnt_i, 4)
        cos_bins_cnts_i.append(cnt_i)
    cos_bins_cnts.append(cos_bins_cnts_i)

#all sf in 2d list
sf_list = []
for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
    cos_keys_keys = list(buta_fit_dict[epho_keys[epho_bin]].keys())
    #sf of 1epho bin - 10 cos bins
    sf_list_1epho = []
    for cos_bin in range(len(cos_bins[j])-1):        
        sf_i = sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.x"]
        sf_list_1epho.append(sf_i)
    sf_list.append(sf_list_1epho)    
    
#Plot sf
for i in range(2):
    for j in range(5):
        #Create list of wanted values from the dictionary
        sf_list_1epho = sf_list[5*i+j]
        #plot sf
        subplot[i, j].plot(cos_bins_cnts[5*i+j], sf_list_1epho, "o", color="#4C72B0")
    
#Set title
for i in range(2):
    for j in range(5):
        subplot[i, j].set_title("$E_{{\gamma}} = [ {:1.2f}, {:1.2f}]$"\
                      .format(epho_bins[5*i+j], epho_bins[5*i+j+1]), fontsize=25) 
    
#labels, lim, grids, ticks font, legends
for i in subplot:
    for j in i:
        j.set_xlabel("$cos\\theta_{cm}$", fontsize=20)
        j.set_ylabel("SF",fontsize=20)
        #lim
        j.set_ylim(0., 10.)
        j.set_xlim(-1, 1.)
        #grids
        j.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.grid(b=True, which="major", color="gray", linewidth=1.0, linestyle="--")
        j.grid(b=True, which="minor", color="gray", linewidth=0.5, linestyle="--")
        #Ticks fontsize change
        j.tick_params(labelsize=15) 
        #Legends
        #j.legend()    
    
#Spacing
sf_plot.subplots_adjust(hspace=0.5)
sf_plot.subplots_adjust(wspace=0.3)        

#Legends
# custom_label = [mpl.lines.Line2D([0], [0], color="#4C72B0", marker="o", ls="none", label="sf")]
# sf_plot.legend(handles=custom_label, loc="upper left", bbox_to_anchor=(0.05, 0.9), ncol=1, fontsize=30)
# sf_plot.legend(handles=custom_label, loc="upper left", ncol=1, fontsize=30)

plt.show()
# sf_plot.savefig("./results_sf_df/sf_plot")


######  CHISQ min for final SF - INDIVIDUAL (MUST)

In [35]:
#sf_final individual
def sf_final_indi(epho_bin=0, cos_bin=0, \
                  buta_dict={}, carb_dict={}, bf_params_dict={}, init_sf_2_dict={}, \
                  epho_bins=[], cos_bins={}, \
                  buta_fit_dict={}, carb_fit_dict={}, \
                  chi_init_mult=[1], bounds_chi_mult=[0, 2]):
    
    #keys for epho_bins
    epho_keys      = sorted(carb_dict.keys())
    cos_keys       = sorted(cos_bins.keys())
    cos_keys_keys  = list(buta_fit_dict[epho_keys[epho_bin]].keys())

    #df declare for easy reading
    df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
    df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]

    #Remove Outliers - region selection
    range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

    #Remove Outliers - remove
    df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
    df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

    #=================================================================================================
    #Into numpy
    butanol_data = np.array(df_buta)
    carbon_data  = np.array(df_carb)

    #Bin Number Determine
    raw_sf = int(init_sf_2_dict[epho_keys[epho_bin]][cos_bin])
    if (raw_sf>=7):
        raw_sf = raw_sf - 2
    bin_size_mltp = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_size_mltp"]    
    bin_num       = int(bin_size_mltp*raw_sf*int(np.sqrt(len(carbon_data))))

    #hist_carbogram
    hist_carb, carb_bins = np.histogram(carbon_data, bins=bin_num)
    hist_buta, buta_bins = np.histogram(butanol_data, bins=carb_bins)

    #Bin Centers
    bin_centers = np.array([0.5*(buta_bins[i] + buta_bins[i+1]) for i in range(len(buta_bins)-1)])            

    #Stats BUTA
    peak_buta   = buta_bins[np.argmax(hist_buta)]

    #=================================================================================================
    #Fitting range            
    start_end = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["start_end"]

    #=================================================================================================
    #Bins within fitting range
    boolean_arry_bins = (bin_centers > start_end[0]) & (bin_centers < start_end[1])
    bin_centers_new   = bin_centers[boolean_arry_bins]

    #=================================================================================================
    #Subset of histogram within FITTING RANGE
    #    -this histogram will be used for final chisq minimization fitting!!!
    hist_buta_new = hist_buta[boolean_arry_bins]
    hist_carb_new = hist_carb[boolean_arry_bins]

    #Stats BUTA new
    mean_buta_new   = np.average(bin_centers_new, weights=hist_buta_new)
    var_buta_new    = np.average((bin_centers_new-mean_buta_new)**2, weights=hist_buta_new)
    stddev_buta_new = np.sqrt(var_buta_new)
    max_buta_new    = np.max(hist_buta_new)    
    sigma_buta      = np.sqrt(hist_buta_new)
    #Stats CARB new
    sigma_carb      = np.sqrt(hist_carb_new)

    #Dont want any entries in sigma to be zero -> division by 0 error
    for i in range(len(sigma_buta)):
        if (sigma_buta[i] == 0):
            sigma_buta[i] = 0.01
    for i in range(len(sigma_carb)):
        if (sigma_carb[i] == 0):
            sigma_carb[i] = 0.01

    #=================================================================================================
    #Params from fitting 
    buta_popt = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]
    carb_popt = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_popt"]

    #=================================================================================================
    #Final chisq Fitting
    def chisq(sol, x=np.array([]), \
              buta_popt=np.array([]), carb_popt=np.array([]), \
              sigma_buta=np.array([]), sigma_carb=np.array([]), \
              hist_buta=np.array([]), hist_carb=np.array([])):
        #Buta bacground
        gaus_buta_back = buta_popt[2]*np.exp(-1.0*(x - buta_popt[3])**2 / (2*(buta_popt[4]**2)))
        poly_buta      = buta_popt[5]
        buta           = gaus_buta_back + poly_buta

        #Carb background
        gaus_carb_back = carb_popt[2]*np.exp(-1.0*(x - carb_popt[3])**2 / (2*(carb_popt[4]**2)))
        poly_carb      = carb_popt[5]
        carb           = gaus_carb_back + poly_carb

        #each bins
        chisq_list = []
        for i in range(len(x)):
            #nume
            nume = pow(buta[i] - sol*carb[i], 2)
            #deno
            deno = pow(sigma_buta[i] + sigma_carb[i], 2)
            #chisq/bin
            chisq_bin = nume / deno
            #chisq_bin = nume

            #append
            chisq_list.append(chisq_bin)
        #sum chisq/bin
        chisq = np.sum(chisq_list)
        return chisq            

    #chisq initial guess for scale factor
    chi_init_guess = [chi_init_mult[0]*raw_sf]

    #chisq bounds for scale factor
    bounds_chi = [(bounds_chi_mult[0]*raw_sf, bounds_chi_mult[1]*raw_sf)]

    #Minimize
    popt = scipy.optimize.minimize(chisq, x0=chi_init_guess, bounds=bounds_chi, \
                                   args=(bin_centers_new, buta_popt, carb_popt, \
                                         hist_buta_new, hist_carb_new), \
                                   method='L-BFGS-B')

    #=================================================================================================
    #Goodness of fit 
    dof       = (len(bin_centers_new) - 1 ) - len(popt.x)
    chisq_per = np.abs(popt.fun) / dof
    
    #Store redone fits
    sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.x"]    = popt.x[0]
    sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.fun"]  = popt.fun
    sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["chisq_per"] = chisq_per

In [89]:
#Indi fit plot
def plot_sf_indi(epho_bin=0, cos_bin=0, bf_params_dict={}, \
                 epho_bins=[], cos_bins={}, \
                 buta_fit_dict={}, carb_fit_dict={}, \
                 buta_dict={}, carb_dict={}, \
                 sf_dict={}):
    
    #keys
    epho_keys     = sorted(carb_dict.keys())
    cos_keys      = sorted(cos_bins.keys())
    cos_keys_keys = list(carb_fit_dict[epho_keys[epho_bin]].keys())    
    
    #Params
    buta_start_end_tpl = bf_params_dict[epho_keys[epho_bin]][cos_bin][6]
    peak_buta          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_buta"]
    peak_carb          = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_carb"]
    buta_popt          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]
    carb_popt          = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_popt"]
    bin_centers_new_buta    = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"]    
    bin_centers_new_carb    = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"]    

    #Use the bins defined in previous fitting procedure
    bins = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bins"]            
    
    #SF*carb
    sf                   = sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.x"]
    chisq_per            = sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["chisq_per"]
    df_carb              = carb_dict[epho_keys[epho_bin]][cos_bin]
    hist_carb, bins_carb = np.histogram(np.array(df_carb["el_pc_mmsq_pi0"]), bins=bins)     
    hist_sc,   bins_sc   = sf*hist_carb, bins_carb            
    mids                 = [0.5*(bins[i] + bins[i+1]) for i in range(len(bins)-1)]
    
    #===================================================================================================
    #Fit buta
    def fit_buta(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus1 + gaus2 + poly
        return y

    #Fit buta bound
    def fit_buta_bound(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus2 + poly
        return y

    #Carbon fit
    def fit_carb(x, A, sigma1, B, mu, sigma2, a, peak_carb):
        #remaining ice
        gaussian   = A*np.exp(-1.0*(x - peak_carb)**2 / (2*(sigma1**2)))
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian + gaussian_2 + poly
        return y

    #Carbon fit
    def fit_carb_bound(x, A, sigma1, B, mu, sigma2, a, peak_carb):
        #remaining ice
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian_2 + poly
        return y
    #===================================================================================================
    #Plot
    sf_redo = plt.figure(figsize=(15,10))
    
    #Ranges
    range_list = carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
    
    #Histograms
    plt.hist(buta_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="butanol", \
             bins=bins, alpha=1., range=(range_list[0], range_list[1]), \
             histtype="step", lw=2., \
             color="#4C72B0")
    plt.hist(carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="carbon", \
             bins=bins, alpha=1, range=(range_list[0], range_list[1]), \
             histtype="step", lw=2., \
             color="#55A868")
    
    #Plot Scaled Carbon
    plt.bar(mids, hist_sc, width=bins_sc[1]-bins_sc[0], alpha=0.5, label="Scaled Carbon", \
            #color="#C44E52")
            #color="#8C0900")
            color="grey")
    
    #Plot fitting
    plt.plot(bin_centers_new_buta, fit_buta(bin_centers_new_buta, *buta_popt, peak_buta), lw=2.5, \
             color="#DC143C", alpha=1., ls="-", label="buta fit")
             #color="red", alpha=1., ls="-", label="buta fit")
    plt.plot(bin_centers_new_buta, fit_buta_bound(bin_centers_new_buta, *buta_popt, peak_buta), lw=2.5, \
             color="black", alpha=1, ls="-", label="buta bound fit")            
             #color="#00FFFF", alpha=1, ls="-", label="buta bound fit")            
             #color="#7FFF00", alpha=1, ls="-", label="buta bound fit")            
    plt.plot(bin_centers_new_carb, fit_carb(bin_centers_new_carb, *carb_popt, peak_carb), lw=2.5, \
             color="#DC143C", alpha=1, ls="--", label="carb fit")
             #color="red", alpha=1, ls="--", label="carb fit")
    plt.plot(bin_centers_new_carb, fit_carb_bound(bin_centers_new_carb, *carb_popt, peak_carb), lw=2.5, \
             color="black", alpha=1, ls="--", label="carb bound_fit")            
             #color="#00FFFF", alpha=1, ls="--", label="carb bound_fit")            
             #color="#7FFF00", alpha=1, ls="--", label="carb bound_fit")            
    
    ##Butanol free nucleon sharp peak start/end positions
    #plt.axvline(buta_start_end_tpl[0], linestyle="--", color="red", alpha=0.5, label="Buta Free-Nucl Peak Start/End pos")
    #plt.axvline(buta_start_end_tpl[1], linestyle="--", color="red", alpha=0.5)

    #===================================================================================================
    #Labels, ticks, lim
    plt.xlabel("$M^2_X$. $[GeV^2/c^4]$", fontsize=30)
    plt.ylabel("Counts $(x10)$", fontsize=30)
    plt.xticks(fontsize=25, fontweight="ultralight")
    plt.yticks(fontsize=25, fontweight="ultralight")
    plt.xlim(range_list[0], range_list[1])

    #Grids
    plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
    plt.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")
    
    #Y-Axis Scaling
    def adjust_y_axis(x, pos):
        return "{:.0f}".format(x/10)
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(adjust_y_axis))    

    #Texts
    plt.text(0.03, 0.43, " Scale factor {:>14.3f}\n $\chi^2/dof$ {:>23.3f}" \
            .format(sf, chisq_per), \
            fontsize=22, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)
#     #Texts
#     plt.text(0.03, 0.53, " Scale factor {:>14.3f}" \
#             .format(sf), \
#             fontsize=10, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)



    #Set Title
    plt.title("$E_{{\gamma}} \in [ {:.2f}, {:.2f}]$ \t $cos\\theta_{{cm}} \in [ {:.2f}, {:.2f}]$" \
              .format(epho_bins[epho_bin], epho_bins[epho_bin+1], cos_bins[cos_keys[epho_bin]][cos_bin], \
                      cos_bins[cos_keys[epho_bin]][cos_bin+1]),\
              fontsize=30)
        
    #Legends
    plt.legend(fontsize=20, loc="upper left")

    plt.show()
    sf_redo.savefig("./run_period_1/sf_final_redo_high_{:d}_{:d}".format(epho_bin, cos_bin))

In [37]:
##COS=0

epho_bin, cos_bin = 0, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.4, 1.45])

epho_bin, cos_bin = 1, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.3, 1.35])

epho_bin, cos_bin = 2, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.35, 1.4])

epho_bin, cos_bin = 3, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

epho_bin, cos_bin = 4, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.2, 1.25])

epho_bin, cos_bin = 5, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

epho_bin, cos_bin = 7, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.15])

epho_bin, cos_bin = 8, 0
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.15])

In [38]:
##COS=1

epho_bin, cos_bin = 0, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

epho_bin, cos_bin = 1, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.3, 1.35])

epho_bin, cos_bin = 2, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.3, 1.35])

epho_bin, cos_bin = 3, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

epho_bin, cos_bin = 4, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.3, 1.35])

epho_bin, cos_bin = 5, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

epho_bin, cos_bin = 6, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])


epho_bin, cos_bin = 7, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.15])

epho_bin, cos_bin = 8, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.25, 1.3])

epho_bin, cos_bin = 9, 1
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

In [39]:
##COS=2

epho_bin, cos_bin = 0, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

epho_bin, cos_bin = 1, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.3, 1.35])

epho_bin, cos_bin = 2, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.15])

epho_bin, cos_bin = 3, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

epho_bin, cos_bin = 4, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.3, 1.35])

epho_bin, cos_bin = 5, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

epho_bin, cos_bin = 6, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])


epho_bin, cos_bin = 7, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.15])

epho_bin, cos_bin = 8, 2
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1., 1.1])

In [40]:
##COS=3

epho_bin, cos_bin = 1, 3
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.15])

epho_bin, cos_bin = 4, 3
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.3, 1.35])

epho_bin, cos_bin = 5, 3
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.15, 1.2])

In [41]:
##COS=4

epho_bin, cos_bin = 6, 4
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.15])


In [42]:
##COS=6

epho_bin, cos_bin = 0, 6
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.2, 1.25])

epho_bin, cos_bin = 1, 6
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.2, 1.25])

epho_bin, cos_bin = 2, 6
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.15])

epho_bin, cos_bin = 3, 6
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.1, 1.15])


In [43]:
##COS=7

epho_bin, cos_bin = 2, 7
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.2, 1.25])

In [44]:
##COS=8

epho_bin, cos_bin = 0, 8
#Call sf_final_indi
sf_final_indi(epho_bin=epho_bin, cos_bin=cos_bin, epho_bins=epho_bins, cos_bins=cos_bins, \
              buta_dict=buta_dict, carb_dict=carb_dict, \
              bf_params_dict=bf_params_dict, init_sf_2_dict=init_sf_2_dict, \
              buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
              chi_init_mult=[1], bounds_chi_mult=[1.2, 1.25])

In [ ]:
##Plot sf redone
epho_bin, cos_bin = 8, 1

#Plot
plot_sf_indi(epho_bin=epho_bin, cos_bin=cos_bin, bf_params_dict=bf_params_dict, \
                 epho_bins=epho_bins, cos_bins=cos_bins, \
                 buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
                 buta_dict=buta_dict, carb_dict=carb_dict, \
                 sf_dict=sf_dict)

###### Butanol Free Peak AGAIN - GLOBAL (MUST)

In [101]:
#Butanol - SF*Carbon Parameter Exraction
def buta_free_params_final(buta_dict={}, carb_dict={}, sf_dict={}, sf_init_dict={}, \
                           epho_bins=[], cos_bins={}):
    #=====================================================================================
    #Dictionary to be returned
    bf_params_dict = {}
    
    #keys for epho_bins
    epho_keys = sorted(carb_dict.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
        #list: For each epho bin, containing a list for each angle bin
        bf_1epho_cos_list = []
        
        #keys for each cos_bin to access fit params
        cos_keys_keys  = list(sf_dict[epho_keys[epho_bin]].keys())        
        
        for cos_bin in range(len(cos_bins[j])-1):
            #df declare for easy reading
            df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
            df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]
            
            #Remove Outliers - region selection
            range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
            
            #Remove Outliers - remove
            df_carb = df_carb[(df_carb["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_carb["el_pc_mmsq_pi0"] <= range_list[1])]
            df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

            #Corresponding Scale factor for the interested bin
            sf = sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.x"]

            #If final sf is nan since discarded bin, use initial sf just to avoid error
            if(np.isnan(sf)):
                sf = sf_init_dict[epho_keys[epho_bin]][cos_bin]

            #=====================================================================================
            #Determine Bin sizes
            bin_num = int(round(sf))*int(np.sqrt(len(df_carb))) 

            #Make histograms first
            hist_carb, bins_carb = np.histogram(np.array(df_carb["el_pc_mmsq_pi0"]), bins=bin_num)
            hist_buta, bins_buta = np.histogram(np.array(df_buta["el_pc_mmsq_pi0"]), bins=bins_carb)
                                
            #=====================================================================================
            #Free butanol peak histograms
            hist_bf, bins_bf = hist_buta - sf*hist_carb, bins_carb

            #If any bin in hist_bf is negative, set it to 0
            for i in range(len(hist_bf)):
                if(hist_bf[i] < 0):
                    hist_bf[i] = 0
            
            #Mids for plotting scale carbon HISTOGRAM
            mids     = np.array([0.5*(bins_bf[i] + bins_bf[i+1]) for i in range(len(bins_bf)-1)])    

            #Stats for free butanol histogram
            peak_bf  = bins_buta[np.argmax(hist_buta)]
            mean_bf  = np.average(mids, weights=hist_bf)
            var_bf   = np.average((mids - mean_bf)**2, weights=hist_bf)
            stddv_bf = np.sqrt(var_bf) 
            max_bf   = np.max(hist_bf)

            #=====================================================================================
            #Shorten the fit range
            #    -peak_bf +/- certain amount of range to enclose the peak and 1 background.
            #    -If the fit range is too wide, the background consists of many gaussians, hindering bg fits.
            xmin_fit = peak_bf - 2*stddv_bf
            xmax_fit = peak_bf + 2*stddv_bf
                        
            #=================================================================================================
            #Bins within fitting range
            boolean_arry_bins = (mids > xmin_fit) & (mids < xmax_fit)
            mids_fit          = mids[boolean_arry_bins]
            
            #subset of hist_bf for fitting
            hist_bf_fit = hist_bf[boolean_arry_bins]
            
            #Stats for free butanol histogram
            peak_bf_fit  = bins_buta[np.argmax(hist_buta)]
            mean_bf_fit  = np.average(mids_fit, weights=hist_bf_fit)
            var_bf_fit   = np.average((mids_fit - mean_bf_fit)**2, weights=hist_bf_fit)
            stddv_bf_fit = np.sqrt(var_bf_fit) 
            max_bf_fit   = np.max(hist_bf_fit)     
            pedestal     = max_bf_fit/20
            
            #=====================================================================================
            #Fit Function (inner)
            def fit_func_combined(x, A, sigma, B, sigma2):
                #Background function
                #Set mu fixed to mean_bf to reduce dof
                gaussian  = A*np.exp(-1.0*(x - mean_bf)**2  / (2*sigma**2))
                #Peak function
                #Set mu fixed to peak_bf to reduce dof
                gaussian2 = B*np.exp(-1.0*(x - peak_bf)**2 / (2*sigma2**2))
                #Final Func
                final_func = gaussian + gaussian2 + pedestal
                return final_func    

            #=====================================================================================
            #Fit
            try:
                params, pconv = curve_fit(fit_func_combined, xdata=mids_fit, ydata=hist_bf_fit, \
                                          p0=[round(max_bf_fit/2, 3), round(stddv_bf_fit, 3), \
                                              round(max_bf_fit, 3), round(stddv_bf_fit/5, 3)])

                #=====================================================================================
                #Butanol free peak start/end positions
                buta_start_end = [peak_bf - 3*params[3], peak_bf + 3*params[3]]
            
                #Few bins have bad fit -> start and end pos are swapped
                if(buta_start_end[0] > buta_start_end[1]):
                    temp = buta_start_end[0]
                    buta_start_end[0] = buta_start_end[1]
                    buta_start_end[1] = temp
            
            except RuntimeError as err1:
                #If fit is not good, set estimated width for free-nucleon peak. Then change later
                params         = [np.nan, np.nan, np.nan]
                buta_start_end = [peak_bf - 0.02, peak_bf + 0.02]
                
            #=====================================================================================
            #Append mids, bins_bf, hist_bf, and final params for each cos bins to bf_1epho_cos_lisabs
            bf_1epho_cos_list.append([mids, bins_bf, hist_bf, params, mean_bf, peak_bf, buta_start_end, \
                                      mids_fit, pedestal, pconv])
    
        #=========================================================================================
        bf_params_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = bf_1epho_cos_list
        
    return bf_params_dict, epho_bins, cos_bins    

In [ ]:
#bf_params_dist has Bins, x_coordinates, hist_bf, and fitting parameters for each epho/cos_bins 
bf_params_dict, epho_bins, cos_bins = buta_free_params_final(buta_dict, carb_dict, sf_dict, init_sf_2_dict,\
                                                             epho_bins, cos_bins)

In [ ]:
# mmsq_1 - Half epho bins (CHECK)
mmsq_1, subplot = plt.subplots(10, 10, figsize=(30, 20))

#Spacing for subplots
mmsq_1.subplots_adjust(hspace=0.5)
mmsq_1.subplots_adjust(wspace=0.2)

#Keys for epho bins
epho_keys = sorted(buta_dict.keys())
cos_keys  = sorted(cos_bins.keys())

#Plot
for i in range(0, 10):
    
    #list to hold each cos_bin keys
    cos_keys_keys = list(buta_fit_dict[epho_keys[i]].keys())
    
    for j in range(10):
        try:
            #Ranges
            range_list = carb_dict[epho_keys[i]][j]["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

            #Use the bins defined in previous fitting procedure
            bins = buta_fit_dict[epho_keys[i]][cos_keys_keys[j]]["buta_bins"]        

            #Plot histograms
            subplot[i, j].hist(buta_dict[epho_keys[i]][j]["el_pc_mmsq_pi0"], label="butanol", \
                                                     bins=bins, alpha=0.7, range=(range_list[0], range_list[1]), \
                                                     color="#4C72B0")
            subplot[i, j].hist(carb_dict[epho_keys[i]][j]["el_pc_mmsq_pi0"], label="carbon", \
                                                     bins=bins, alpha=0.7, range=(range_list[0], range_list[1]), \
                                                     color="#55A868")        

            #Butanol free nucleon sharp peak start/end positions
            buta_start_end_tpl = bf_params_dict[epho_keys[i]][j][6]
            subplot[i, j].axvline(buta_start_end_tpl[0], linestyle="--", color="red", alpha=0.5, label="Buta Free-Nucl Peak Start/End pos")
            subplot[i, j].axvline(buta_start_end_tpl[1], linestyle="--", color="red", alpha=0.5)
            
            #lim
            subplot[i, j].set_xlim(range_list[0], range_list[1])

        #For bins that fitting failed    
        except KeyError as err1:
            continue

#CROSS OUT bad bins
for i in range(0, 10):
    for j in range(10):
        #epho=0
        if (i==0 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=1
        elif (i==1 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==1 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=2
        elif (i==2 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==2 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=3
        elif (i==3 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==3 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==3 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=4
        elif (i==4 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==4 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==4 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==4 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=5
        elif (i==5 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==5 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==5 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==5 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=6
        elif (i==6 and j==5):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=7
        elif (i==7 and j==4):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==5):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==7 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=8
        elif (i==8 and j==0):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==8 and j==3):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==4):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==5):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==8 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=9
        elif (i==9 and j==0):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==9 and j==2):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==9 and j==3):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==4):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==5):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==9 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
            
#grids
for i in subplot:
    for j in i:    
        #grids
        j.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
        j.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")
        
#Set COS Title
for i in range(10):
    for j in range(10):
        subplot[i, j].set_title("[{:.2f}, {:.2f}]".format(cos_bins[cos_keys[i]][j], cos_bins[cos_keys[i]][j+1]), \
                                fontsize=15)
#Set EPHO Title
for i in range(10):
    for j in range(1):
        subplot[i, j].set_ylabel("[{:.2f}, {:.2f}]".format(epho_bins[i], epho_bins[i+1]), fontsize=15)
        
    
#Label
mmsq_1.text(0.9, 0.075, "Missing Mass Sq (${GeV}^2$)", ha="right", fontsize=40, weight="bold")
mmsq_1.text(0.9, 0.925, "$\pi^0$ COS$\\theta_{cm}$", ha="right", fontsize=40, weight="bold")        
mmsq_1.text(0.07, 0.5, "$E_{\gamma}$ (GeV)", va="center", rotation="vertical", fontsize=40, weight="bold")

#Legends
custom_label = [mpl.patches.Patch(facecolor="#4C72B0", alpha=0.7, label="Butanol"), \
                mpl.patches.Patch(facecolor="#55A868", alpha=0.7, label="Carbon"), \
                mpl.lines.Line2D([0], [0], color="red", alpha=1, linewidth=3, label="Buta Free-Nucl Peak Start/End pos", \
                                 linestyle="--")]
mmsq_1.legend(handles=custom_label, loc="upper left", bbox_to_anchor=(0.05, 0.9), ncol=6, fontsize=30)

plt.show()
# mmsq_1.savefig("./run_period_1/mmsq_1")

In [ ]:
#Plot overview of peaks + buta_free_peak positions (CHECK) 
Fit_params, subplot = plt.subplots(2, 5, figsize=(30, 10))

#Keys and new cos_bins to indicate epho/cos bins in sf_2_dict
epho_keys       = sorted(cos_bins.keys())


# cos_bins_center = [round(0.5 * (cos_bins[i] + cos_bins[i+1]), 2) for i in range(len(cos_bins) - 1)]

#Cos_bins_cnt in 2d list
cos_bins_cnts = []
for i in sorted(cos_bins.keys()):
    cos_bins_cnts_i = []
    for j in range(len(cos_bins[i])-1):
        cnt_i           = 0.5*(cos_bins[i][j] + cos_bins[i][j+1])
        cnt_i           = round(cnt_i, 4)
        cos_bins_cnts_i.append(cnt_i)
    cos_bins_cnts.append(cos_bins_cnts_i)
    
    

#Plot fit parameters
for i in range(2):
    for j in range(5):
        
        #Create list of wanted values from the dictionary
        peak_ls  = [bf_params_dict[epho_keys[5*i+j]][k][5] for k in range(len(bf_params_dict[epho_keys[5*i+j]]))]
        bf_start = [bf_params_dict[epho_keys[5*i+j]][k][6][0] for k in range(len(bf_params_dict[epho_keys[5*i+j]]))]
        bf_end   = [bf_params_dict[epho_keys[5*i+j]][k][6][1] for k in range(len(bf_params_dict[epho_keys[5*i+j]]))]
        
        #Peak plot
        subplot[i, j].plot(cos_bins_cnts[5*i+j], peak_ls, "o", color="#C44E52", label="Peak")
        subplot[i, j].plot(cos_bins_cnts[5*i+j], bf_start, "o", color="#55A868", label="Peak - $3\sigma$")
        subplot[i, j].plot(cos_bins_cnts[5*i+j], bf_end, "o", color="#4C72B0", label="Peak + $3\sigma$")

#Set title
for i in range(2):
    for j in range(5):
        subplot[i, j].set_title("$E_{{\gamma}} = [ {:1.2f}, {:1.2f}]$"\
                      .format(epho_bins[5*i+j], epho_bins[5*i+j+1]), fontsize=25) 
        
#labels, lim, grids, ticks font, legends
for i in subplot:
    for j in i:
        j.set_xlabel("$cos\\theta_{cm}$", fontsize=20)
        j.set_ylabel("MM",fontsize=20)
        #lim
        j.set_ylim(-0.2, 0.2)
        #grids
        j.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.grid(b=True, which="major", color="gray", linewidth=1.0, linestyle="--")
        j.grid(b=True, which="minor", color="gray", linewidth=0.5, linestyle="--")
        #Ticks fontsize change
        j.tick_params(labelsize=15) 
        #Legends
        #j.legend()

#Spacing
Fit_params.subplots_adjust(hspace=0.5)
Fit_params.subplots_adjust(wspace=0.5)        
# Fit_params.tight_layout()

# #axes for legneds
# legend_axes = Fit_params.add_axes([0.1, 1., 0.2, 0.05])
# legend_axes.axis("off")
    
# #Legends
# custom_label = [mpl.lines.Line2D([0], [0], color="#C44E52", marker="o", ls="none", label="Buta peak"), \
#                 mpl.lines.Line2D([0], [0], color="#55A868", marker="o", ls="none", label="Buta Free-Nucl start"), \
#                 mpl.lines.Line2D([0], [0], color="#4C72B0", marker="o", ls="none", label="Buta Free-Nucl end")]

# legend_axes.legend(handles=custom_label, loc="upper left", ncol=3, fontsize=30)
# # legend_axes.legend(handles=custom_label, loc="upper left", bbox_to_anchor=(0.1, 0.95), ncol=3, fontsize=20)

plt.show()
# Fit_params.savefig("./run_period_1/Fit_params")

###### Butanol Free Peak AGAIN - INDIVIDUAL (MUST)

In [103]:
#bf_params_final_indi
def buta_free_params_indi(epho_bin=0, cos_bin=0, bf_start=-0.01, bf_end=0.07, 
                         epho_bins=[], cos_bins={}):
    #Keys for epho bins
    epho_keys = sorted(cos_bins.keys())
    
    #Store redone bf ranges
    bf_params_dict[epho_keys[epho_bin]][cos_bin][6] = (bf_start, bf_end)

In [104]:
#Plot bf indi
def plot_bf_indi(epho_bin=0, cos_bin=0, bf_params_dict={}, \
                 epho_bins=[], cos_bins={}, \
                 buta_fit_dict={}, carb_fit_dict={}, \
                 buta_dict={}, carb_dict={}, \
                 sf_dict={}):
    
    #keys
    epho_keys     = sorted(carb_dict.keys())
    cos_keys      = sorted(cos_bins.keys())
    cos_keys_keys = list(carb_fit_dict[epho_keys[epho_bin]].keys())    
    
    #Params
    buta_start_end_tpl = bf_params_dict[epho_keys[epho_bin]][cos_bin][6]
    peak_buta          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_buta"]
    peak_carb          = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_carb"]
    buta_popt          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]
    carb_popt          = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["carb_popt"]
    bin_centers_new_buta    = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"]    
    bin_centers_new_carb    = carb_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["bin_centers_new"]    

    #Use the bins defined in previous fitting procedure
    bins = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bins"]            
    
    #SF*carb
    sf                   = sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["popt.x"]
    chisq_per            = sf_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["chisq_per"]
    df_carb              = carb_dict[epho_keys[epho_bin]][cos_bin]
    hist_carb, bins_carb = np.histogram(np.array(df_carb["el_pc_mmsq_pi0"]), bins=bins)     
    hist_sc,   bins_sc   = sf*hist_carb, bins_carb            
    mids                 = [0.5*(bins[i] + bins[i+1]) for i in range(len(bins)-1)]
    
    #===================================================================================================
    #Fit buta
    def fit_buta(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus1 + gaus2 + poly
        return y

    #Fit buta bound
    def fit_buta_bound(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus2 + poly
        return y

    #Carbon fit
    def fit_carb(x, A, sigma1, B, mu, sigma2, a, peak_carb):
        #remaining ice
        gaussian   = A*np.exp(-1.0*(x - peak_carb)**2 / (2*(sigma1**2)))
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian + gaussian_2 + poly
        return y

    #Carbon fit
    def fit_carb_bound(x, A, sigma1, B, mu, sigma2, a, peak_carb):
        #remaining ice
        gaussian_2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly       = a
        y          = gaussian_2 + poly
        return y
    #===================================================================================================
    #Plot
    bf_redo = plt.figure(figsize=(15,10))
    
    #Ranges
    range_list = carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()
    
    #Histograms
    plt.hist(buta_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="butanol", \
             bins=bins, alpha=0.7, range=(range_list[0], range_list[1]), \
             color="#4C72B0")
    plt.hist(carb_dict[epho_keys[epho_bin]][cos_bin]["el_pc_mmsq_pi0"], label="carbon", \
             bins=bins, alpha=0.9, range=(range_list[0], range_list[1]), \
             color="#55A868")
    
#     #Plot Scaled Carbon
#     plt.bar(mids, hist_sc, width=bins_sc[1]-bins_sc[0], alpha=0.3, color="#C44E52", label="Scaled Carbon")
    
#     #Plot fitting
#     plt.plot(bin_centers_new_buta, fit_buta(bin_centers_new_buta, *buta_popt, peak_buta), lw=1., \
#              color="red", alpha=0.7, ls="-", label="buta fit")
#     plt.plot(bin_centers_new_buta, fit_buta_bound(bin_centers_new_buta, *buta_popt, peak_buta), lw=1., \
#              color="blue", alpha=0.7, ls="-", label="buta bound fit")            
#     plt.plot(bin_centers_new_carb, fit_carb(bin_centers_new_carb, *carb_popt, peak_carb), lw=1., \
#              color="red", alpha=0.7, ls="--", label="carb fit")
#     plt.plot(bin_centers_new_carb, fit_carb_bound(bin_centers_new_carb, *carb_popt, peak_carb), lw=1., \
#              color="blue", alpha=0.7, ls="--", label="carb bound_fit")            
    
    #Butanol free nucleon sharp peak start/end positions
    plt.axvline(buta_start_end_tpl[0], linestyle="--", color="red", alpha=1, lw=2, label="Buta Free-Nucl Peak Start/End pos")
    plt.axvline(buta_start_end_tpl[1], linestyle="--", color="red", alpha=1, lw=2)

    #===================================================================================================
    #Y-Axis Scaling
    def adjust_y_axis(x, pos):
        return "{:.0f}".format(x/100)
    plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(adjust_y_axis))

    #Labels, ticks, lim
    plt.xlabel("$M^2_X$. $[GeV^2/c^4]$", fontsize=30)
    plt.ylabel("Counts $(x10^2)$", fontsize=30)
    plt.xticks(fontsize=25, fontweight="ultralight")
    plt.yticks(fontsize=25, fontweight="ultralight")
    plt.xlim(range_list[0], range_list[1])

    #Grids
    plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
    plt.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
    plt.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")

    #Texts
    plt.text(0.6, 0.7, " Free-nucl peak start {:>14.3f}\n Free-nucl peak end {:>14.3f}" \
             .format(buta_start_end_tpl[0], buta_start_end_tpl[1]), \
             fontsize=20, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)

    #Set Title
    plt.title("$E_{{\gamma}} \in [ {:.2f}, {:.2f}]$ \t $cos\\theta_{{cm}} \in [ {:.2f}, {:.2f}]$" \
              .format(epho_bins[epho_bin], epho_bins[epho_bin+1], cos_bins[cos_keys[epho_bin]][cos_bin], \
                      cos_bins[cos_keys[epho_bin]][cos_bin+1]),\
              fontsize=30)
        
    #Legends
    plt.legend(fontsize=17.5, loc="upper right")

    plt.show()
    bf_redo.savefig("./run_period_1/bf_final_redo_{:d}_{:d}".format(epho_bin, cos_bin))

In [105]:
##COS=0

epho_bin, cos_bin = 0, 0
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

epho_bin, cos_bin = 2, 0
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

epho_bin, cos_bin = 6, 0
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

epho_bin, cos_bin = 7, 0
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

In [106]:
##COS=1

epho_bin, cos_bin = 3, 1
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

epho_bin, cos_bin = 8, 1
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

epho_bin, cos_bin = 9, 1
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)

In [107]:
##COS=2

epho_bin, cos_bin = 8, 2
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)


In [108]:
##COS=3

epho_bin, cos_bin = 7, 3
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)


In [109]:
##COS=4

epho_bin, cos_bin = 6, 4
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)


In [110]:
##COS=5

epho_bin, cos_bin = 5, 5
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)


In [111]:
##COS=7

epho_bin, cos_bin = 2, 7
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)


In [112]:
##COS=7

epho_bin, cos_bin = 0, 8
#Call buta indi fit
buta_free_params_indi(epho_bin, cos_bin, \
                      bf_start=-0.045, bf_end=0.108, \
                      epho_bins=epho_bins, cos_bins=cos_bins)


In [ ]:
# Plot
epho_bin, cos_bin = 5, 5

#Plot
plot_bf_indi(epho_bin=epho_bin, cos_bin=cos_bin, bf_params_dict=bf_params_dict, \
                 epho_bins=epho_bins, cos_bins=cos_bins, \
                 buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
                 buta_dict=buta_dict, carb_dict=carb_dict, \
                 sf_dict=sf_dict)

#### Dilution Factor 

###### DF function GLOBAL (MUST) 

In [ ]:
#df inidi
def df_global(buta_dict={}, carb_dict={}, bf_params_dict={}, init_sf_2_dict={}, \
              epho_bins=[], cos_bins={}, \
              buta_fit_dict={}, carb_fit_dict={}, \
              sf_dict={}):
    
    #df dict
    df_dict = {}
    
    #keys
    epho_keys     = sorted(cos_bins.keys())
    cos_keys      = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
    
        #Dict to store dict or whole cos
        df_1epho_cos_dict = {}    
    
        #keys for each cos_bin to access fit params
        cos_keys_keys = list(buta_fit_dict[epho_keys[epho_bin]].keys())

        #Each epho_bin has diff cos bins defined
        for cos_bin in range(len(cos_bins[j])-1):
        
            #=================================================================================================
            df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
            df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]

            #Remove Outliers - region selection
            range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

            #Remove Outliers - remove
            df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                              (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

            #Into numpy
            butanol_data = np.array(df_buta)

            #=================================================================================================
            #params from previous steps
            buta_start_end_tpl = bf_params_dict[epho_keys[epho_bin]][cos_bin][6]
            peak_buta          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_buta"]
            buta_popt          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]

            #Use the bins defined in previous fitting procedure
            #    -mids for whole range of histograms
            bins                    = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bins"] #same bisn for buta&carb       
            mids                    = np.array([0.5*(bins[i] + bins[i+1]) for i in range(len(bins)-1)])

            #histograms
            hist_buta, buta_bins = np.histogram(butanol_data, bins=bins)

            #=================================================================================================
            #Integrand for buta backgroun
            def fit_buta(x, A, sigma1, B, mu, sigma2, a, peak_buta):
                gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
                gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
                poly  = a
                y     = gaus1 + gaus2 + poly
                return y    

            #Fit buta bound
            def fit_buta_bound(x, A, sigma1, B, mu, sigma2, a, peak_buta):
                gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
                poly  = a
                y     = gaus2 + poly
                return y
            
            #Need to have df=np.nan to discard bins
            try:
                #Integrate under fit_buta background
                buta_bg_intg = scipy.integrate.quad(fit_buta_bound, buta_start_end_tpl[0], buta_start_end_tpl[1], \
                                                    args=(buta_popt[0], buta_popt[1], buta_popt[2], \
                                                          buta_popt[3], buta_popt[4], buta_popt[5], peak_buta))

                #=================================================================================================
                #New histrogram for butanol - range of interest
                boolean_arry_bins = (mids >= buta_start_end_tpl[0]) & (mids <= buta_start_end_tpl[1])
                bin_centers_new   = mids[boolean_arry_bins]

                #Subset of buta histogram 
                hist_buta_new = hist_buta[boolean_arry_bins]

                #bin width
                bin_width = bin_centers_new[1] - bin_centers_new[0]

                #Counting BUTA 
                buta_total_sum = np.sum([bin_width * hist_buta_new[i] for i in range(len(hist_buta_new))])

                #=================================================================================================
                #DF calculation
                df = (buta_total_sum - buta_bg_intg[0]) / buta_total_sum

            #For discarded bins
            except ZeroDivisionError as err1:
                df = np.nan
            
            except IndexError as err1:            
                df = np.nan

            #=================================================================================================
            df_1epho_1cos_dict = {"df":df, "buta_bf_intg":buta_bg_intg[0], "buta_total_sum":buta_total_sum, \
                                  "buta_bf_intg_err":buta_bg_intg[1]}

            #Append
            df_1epho_cos_dict["cos_{:07.3f}".format(cos_bins[cos_keys[epho_bin]][cos_bin])] = df_1epho_1cos_dict
            
        #=================================================================================================
        df_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = df_1epho_cos_dict
    
    return df_dict

In [ ]:
#Discard bad bin
def df_discard_bad(epho_bins=[], cos_bins={}, df_dict={}):
    
    #keys for epho_bins
    epho_keys = sorted(cos_bins.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):

        #keys for each cos_bin to access fit params
        cos_keys_keys  = list(df_dict[epho_keys[epho_bin]].keys())
        
        #fn to call within next loop for discarding bins
        def discard_bad_bins(epho_bin=0, cos_bin=0):
            df_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["df"]             = np.nan
            df_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bf_intg"]   = np.nan
            df_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_total_sum"] = np.nan

        #=============================================================================================
        #Each epho_bin has diff cos bins defined
        for cos_bin in range(len(cos_bins[j])-1):
            #epho=0
            if (epho_bin==0 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==0 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=1
            elif (epho_bin==1 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==1 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==1 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)
                
            #epho=2
            elif (epho_bin==2 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==2 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==2 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)
                
            #epho=3
            elif (epho_bin==3 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==3 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==3 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==3 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)
                
            #epho=4
            elif (epho_bin==4 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==4 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==4 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==4 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==4 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)

                
            #epho=5
            elif (epho_bin==5 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==5 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==5 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==5 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==5 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)

            #epho=6
            elif (epho_bin==6 and cos_bin==5):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==6 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==6 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==6 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==6 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==6 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)
            
            #epho=7
            elif (epho_bin==7 and cos_bin==4):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==7 and cos_bin==5):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==7 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==7 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==7 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==7 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==7 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)
                
            #epho=8
            elif (epho_bin==8 and cos_bin==0):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==3):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==4):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==5):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==8 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)
                
            #epho=9
            elif (epho_bin==9 and cos_bin==0):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==2):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==3):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==4):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==5):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==6):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==7):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==8):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin==9):
                discard_bad_bins(epho_bin, cos_bin)
            elif (epho_bin==9 and cos_bin>=10):
                discard_bad_bins(epho_bin, cos_bin)                

In [ ]:
#call df_global
df_dict = df_global(buta_dict=buta_dict, carb_dict=carb_dict, \
                    bf_params_dict=bf_params_dict, \
                    init_sf_2_dict=init_sf_2_dict, \
                    epho_bins=epho_bins, cos_bins=cos_bins, \
                    buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
                    sf_dict=sf_dict)    

In [ ]:
#Discard bad bins
df_discard_bad(epho_bins, cos_bins, df_dict)

###### DF function INDIVIDUAL (CHECK) 

In [ ]:
#df inidi
def df_indi(epho_bin=0, cos_bin=0, \
           buta_dict={}, carb_dict={}, bf_params_dict={}, init_sf_2_dict={}, \
           epho_bins=[], cos_bins={}, \
           buta_fit_dict={}, carb_fit_dict={}, \
           sf_dict={}):
    
    #keys
    epho_keys     = sorted(cos_bins.keys())
    cos_keys      = sorted(cos_bins.keys())
    cos_keys_keys = list(buta_fit_dict[epho_keys[epho_bin]].keys())

    
    #=================================================================================================
    #df declare for easy reading
    df_buta = buta_dict[epho_keys[epho_bin]][cos_bin]
    df_carb = carb_dict[epho_keys[epho_bin]][cos_bin]

    #Remove Outliers - region selection
    range_list = df_carb["el_pc_mmsq_pi0"].quantile([0.01, 0.999]).tolist()

    #Remove Outliers - remove
    df_buta = df_buta[(df_buta["el_pc_mmsq_pi0"] >= range_list[0]) & \
                      (df_buta["el_pc_mmsq_pi0"] <= range_list[1])]

    #Into numpy
    butanol_data = np.array(df_buta)

    #=================================================================================================
    #params from previous steps
    buta_start_end_tpl = bf_params_dict[epho_keys[epho_bin]][cos_bin][6]
    peak_buta          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["peak_buta"]
    buta_popt          = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_popt"]

    #Use the bins defined in previous fitting procedure
    #    -mids for whole range of histograms
    bins                    = buta_fit_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bins"] #same bisn for buta&carb       
    mids                    = np.array([0.5*(bins[i] + bins[i+1]) for i in range(len(bins)-1)])
    
    #histograms
    hist_buta, buta_bins = np.histogram(butanol_data, bins=bins)
    
    #=================================================================================================
    #Integrand for buta backgroun
    def fit_buta(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus1 = A*np.exp(-1.0*(x - peak_buta)**2 / (2*(sigma1**2)))
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus1 + gaus2 + poly
        return y    

    #Fit buta bound
    def fit_buta_bound(x, A, sigma1, B, mu, sigma2, a, peak_buta):
        gaus2 = B*np.exp(-1.0*(x - mu)**2 / (2*(sigma2**2)))
        poly  = a
        y     = gaus2 + poly
        return y
    
    #Need to have df=np.nan to discard bins
    try:
        #Integrate under fit_buta background
        buta_bg_intg = scipy.integrate.quad(fit_buta_bound, buta_start_end_tpl[0], buta_start_end_tpl[1], \
                                            args=(buta_popt[0], buta_popt[1], buta_popt[2], \
                                                  buta_popt[3], buta_popt[4], buta_popt[5], peak_buta))

        #=================================================================================================
        #New histrogram for butanol - range of interest
        boolean_arry_bins = (mids >= buta_start_end_tpl[0]) & (mids <= buta_start_end_tpl[1])
        bin_centers_new   = mids[boolean_arry_bins]

        #Subset of buta histogram 
        hist_buta_new = hist_buta[boolean_arry_bins]

        #bin width
        bin_width = bin_centers_new[1] - bin_centers_new[0]

        #Counting BUTA 
        buta_total_sum = np.sum([bin_width * hist_buta_new[i] for i in range(len(hist_buta_new))])

        #=================================================================================================
        #DF calculation
        df = (buta_total_sum - buta_bg_intg[0]) / buta_total_sum

    #For discarded bins
    except ZeroDivisionError as err1:
        df = np.nan
    except IndexError as err1:            
        df = np.nan
        
    return df, buta_bg_intg[0], buta_total_sum, buta_bg_intg[1]

In [ ]:
#bins to run
epho_bin, cos_bin = 12,3

#Call df_indi
df, buta_bg_intg, buta_total_sum, buta_bf_intg_err = df_indi(epho_bin=epho_bin, cos_bin=cos_bin, \
                                                             buta_dict=buta_dict, carb_dict=carb_dict, \
                                                             bf_params_dict=bf_params_dict, \
                                                             init_sf_2_dict=init_sf_2_dict, \
                                                             epho_bins=epho_bins, cos_bins=cos_bins, \
                                                             buta_fit_dict=buta_fit_dict, carb_fit_dict=carb_fit_dict, \
                                                             sf_dict=sf_dict)

###### Plot DF (CHECK)

In [ ]:
#Plot df (CHECK)
df_plot, subplot = plt.subplots(2, 5, figsize=(30, 10))

#epho_keys
epho_keys = sorted(cos_bins.keys())
cos_keys  = sorted(cos_bins.keys())

#Cos_bins_cnt in 2d list
cos_bins_cnts = []
for i in sorted(cos_bins.keys()):
    cos_bins_cnts_i = []
    for j in range(len(cos_bins[i])-1):
        cnt_i           = 0.5*(cos_bins[i][j] + cos_bins[i][j+1])
        cnt_i           = round(cnt_i, 4)
        cos_bins_cnts_i.append(cnt_i)
    cos_bins_cnts.append(cos_bins_cnts_i)

#all df in 2d list
df_list = []
for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
    cos_keys_keys = list(buta_fit_dict[epho_keys[epho_bin]].keys())
    #df of 1epho bin - 10 cos bins
    df_list_1epho = []
    for cos_bin in range(len(cos_bins[j])-1):        
        df_i = df_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["df"]
        df_list_1epho.append(df_i)
    df_list.append(df_list_1epho)    
    
#Plot df
for i in range(2):
    for j in range(5):
        #Create list of wanted values from the dictionary
        df_list_1epho = df_list[5*i+j]
        #plot df
        subplot[i, j].plot(cos_bins_cnts[5*i+j], df_list_1epho, "o", color="#4C72B0")
        
#Horizaontal line to show df low lim
for i in range(2):
    for j in range(5):
        subplot[i, j].axhline(10/74, linewidth=2.0, linestyle="--", color="black", alpha=0.5)
        
    
#Set title
for i in range(2):
    for j in range(5):
        subplot[i, j].set_title("$E_{{\gamma}} = [ {:1.2f}, {:1.2f}]$"\
                      .format(epho_bins[5*i+j], epho_bins[5*i+j+1]), fontsize=25) 
    
#labels, lim, grids, ticks font, legends
for i in subplot:
    for j in i:
        j.set_xlabel("$cos\\theta_{cm}$", fontsize=20)
        j.set_ylabel("DF",fontsize=20)
        #lim
        j.set_ylim(0., 1.)
        j.set_xlim(-1, 1.)
        #grids
        j.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.grid(b=True, which="major", color="gray", linewidth=1.0, linestyle="--")
        j.grid(b=True, which="minor", color="gray", linewidth=0.5, linestyle="--")
        #Ticks fontsize change
        j.tick_params(labelsize=15) 
        #Legends
        #j.legend()    
    
#Spacing
df_plot.subplots_adjust(hspace=0.5)
df_plot.subplots_adjust(wspace=0.3)        

#Legends
# custom_label = [mpl.lines.Line2D([0], [0], color="#4C72B0", marker="o", ls="none", label="df")]
# df_plot.legend(handles=custom_label, loc="upper left", bbox_to_anchor=(0.05, 0.9), ncol=1, fontsize=30)
# df_plot.legend(handles=custom_label, loc="upper left", ncol=1, fontsize=30)

plt.show()
# df_plot.savefig("./results_sf_df/df_plot")


#### Final check on POLARIZATION STATES

###### Compute Photon Polarization Statistical Error (MUST)

In [ ]:
#Optimized Version - Vectorization numpy
def comp_pho_Pol_stat(epho, runNum):
    pol_pho_stat = 0
    
    #E_Beam for run ranges
    if   (runNum >= 55521 and runNum <= 55536):
        E_ebeam   = 1.647
    elif (runNum >= 55537 and runNum <= 55552):
        E_ebeam   = 1.647
    elif (runNum >= 55557 and runNum <= 55594):
        E_ebeam   = 1.647
    elif (runNum >= 55604 and runNum <= 55625):
        E_ebeam   = 2.478
    elif (runNum >= 55630 and runNum <= 55678):
        E_ebeam   = 2.478
    elif (runNum >= 56164 and runNum <= 56193):
        E_ebeam   = 2.478
    elif (runNum >= 56196 and runNum <= 56233):
        E_ebeam   = 2.478
        
    #Error for Polarization of E beam for measured run ranges [Steffen AN sec3.1]
    #CORRECT value used since 2021_05
    E_beam_pol     = [.85228, .78523, .79150, .887, .84167, .84725, .86531, .88409, .87753, \
                      .82534, .79450, .80060, .83267, .83248]
    
    E_beam_pol_err = [.142, .1350 , .126, .1480, .1330, .1530, .1380, .1440, .1480, \
                      .1400, .1410, .1400, .1380, .1320]
     
    #Use std of E_beam_pol as ebeam err - a bit skeptical approach. See Steffen's AN
    ebeam_stat_err = np.std(E_beam_pol)
    
    
    #Variables
    x              = epho / E_ebeam
    x2             = x*x
    pol_pho_stat   = np.sqrt((ebeam_stat_err**2) * ((4*x - x2) / (4 - 4*x + 3*x2))**2)
    
    return pol_pho_stat

#Vectorize
vect_comp_pho_Pol_stat = np.vectorize(comp_pho_Pol_stat, otypes=[np.float])
pol_pho_stat           = vect_comp_pho_Pol_stat(df_GMSH["epho"].values, df_GMSH["runNum"].values)

#Append to df
df_GMSH["pol_pho_stat"] = pd.Series(pol_pho_stat, index=df_GMSH.index)

#### MMSQ Selection 

In [ ]:
#MMSQ GLOBAL()
def mmsq_global(df=pd.DataFrame(), epho_bins=[], cos_bins={}, bf_params_dict={}):
    
    #dict to contain df partitioned in epho & cos bins + events within bf peak ranges
    mmsq_dict = {}
    
    #keys
    epho_keys = sorted(cos_bins.keys())
    cos_keys  = sorted(cos_bins.keys())
    
    #================================================================================================================
    #Butanol only
    df_buta = df[df["T_fin"] == 0]
    
    #================================================================================================================
    #For each Eg_bon
    for epho_bin, j in zip(range(len(epho_bins)-1), cos_keys):
        #Partition by epho bin
        df_buta_Eg = df_buta[(df_buta["epho"] > epho_bins[epho_bin]) & \
                             (df_buta["epho"] < epho_bins[epho_bin+1])]
    
        #List of dataframes in same epho bins
        buta_epho_list = []

        #========================================================================================================
        #For each cos_bin
        for cos_bin in range(len(cos_bins[j])-1):
            #Partition by cos
            df_buta_Eg_cos = df_buta_Eg \
                             [(df_buta_Eg["cthe_cm"] > cos_bins[j][cos_bin]) & \
                              (df_buta_Eg["cthe_cm"] < cos_bins[j][cos_bin+1])]

            #Start and end positions of butanol free nucleon peak
            bf_start = bf_params_dict[epho_keys[epho_bin]][cos_bin][6][0]
            bf_end   = bf_params_dict[epho_keys[epho_bin]][cos_bin][6][1]
            
            #========================================================================================================            
            #MMSQ SELECTION
            df_buta_Eg_cos.drop(df_buta_Eg_cos[(df_buta_Eg_cos["el_pc_mmsq_pi0"] < bf_start) | \
                                               (df_buta_Eg_cos["el_pc_mmsq_pi0"] > bf_end)].index, \
                                inplace=True)

            #Append to its epho list
            buta_epho_list.append(df_buta_Eg_cos)

        #========================================================================================================            
        #Append to mmsq_dict for each energy bins
        mmsq_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = buta_epho_list
        
    return mmsq_dict

In [ ]:
#Apply mmsq_global()
mmsq_dict = mmsq_global(df_GMSH, epho_bins, cos_bins, bf_params_dict)

In [ ]:
# mmsq_after_1 - Half epho bins (CHECK)
mmsq_after_1, subplot = plt.subplots(10, 10, figsize=(30, 20))

#Spacing for subplots
mmsq_after_1.subplots_adjust(hspace=0.5)
mmsq_after_1.subplots_adjust(wspace=0.2)

#Keys for epho bins
epho_keys = sorted(mmsq_dict.keys())
cos_keys  = sorted(cos_bins.keys())

#Plot
for i in range(0, 10):
    for j in range(10):
        try:
            #Ranges
            range_list = (-0.1, 0.2)

            #Plot histograms
            subplot[i, j].hist(mmsq_dict[epho_keys[i]][j]["el_pc_mmsq_pi0"], label="butanol", \
                                                     bins="fd", alpha=0.7, range=(range_list[0], range_list[1]), \
                                                     color="#4C72B0")

            #Butanol free nucleon sharp peak start/end positions
            buta_start_end_tpl = bf_params_dict[epho_keys[i]][j][6]
            subplot[i, j].axvline(buta_start_end_tpl[0], linestyle="--", color="red", alpha=0.5, label="Buta Free-Nucl Peak Start/End pos")
            subplot[i, j].axvline(buta_start_end_tpl[1], linestyle="--", color="red", alpha=0.5)
            
            #lim
            subplot[i, j].set_xlim(range_list[0], range_list[1])

        #For bins that fitting failed    
        except KeyError as err1:
            continue

#CROSS OUT bad bins
for i in range(0, 10):
    for j in range(10):
        #epho=0
        if (i==0 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=1
        elif (i==1 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==1 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=2
        elif (i==2 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==2 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=3
        elif (i==3 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==3 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==3 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=4
        elif (i==4 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==4 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==4 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==4 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=5
        elif (i==5 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==5 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==5 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==5 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=6
        elif (i==6 and j==5):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==6 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=7
        elif (i==7 and j==4):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==5):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==7 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==7 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=8
        elif (i==8 and j==0):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==8 and j==3):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==4):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==5):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==8 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==8 and j==9):
            subplot[i, j].set_facecolor("#C44E52")
        #epho=9
        elif (i==9 and j==0):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==9 and j==2):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==9 and j==3):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==4):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==5):
            subplot[i, j].set_facecolor("#C44E52")        
        elif (i==9 and j==6):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==7):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==8):
            subplot[i, j].set_facecolor("#C44E52")
        elif (i==9 and j==9):
            subplot[i, j].set_facecolor("#C44E52")            

#grids
for i in subplot:
    for j in i:    
        #grids
        j.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
        j.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")
        
#Set COS Title
for i in range(10):
    for j in range(10):
        subplot[i, j].set_title("[{:.2f}, {:.2f}]".format(cos_bins[cos_keys[i]][j], cos_bins[cos_keys[i]][j+1]), \
                                fontsize=15)
#Set EPHO Title
for i in range(10):
    for j in range(1):
        subplot[i, j].set_ylabel("[{:.2f}, {:.2f}]".format(epho_bins[i], epho_bins[i+1]), fontsize=15)
        
    
#Label
mmsq_after_1.text(0.9, 0.075, "Missing Mass Sq (${GeV}^2$)", ha="right", fontsize=40, weight="bold")
mmsq_after_1.text(0.9, 0.925, "$\pi^0$ COS$\\theta_{cm}$", ha="right", fontsize=40, weight="bold")        
mmsq_after_1.text(0.07, 0.5, "$E_{\gamma}$ (GeV)", va="center", rotation="vertical", fontsize=40, weight="bold")

#Legends
custom_label = [mpl.patches.Patch(facecolor="#4C72B0", alpha=0.7, label="Butanol"), \
                mpl.lines.Line2D([0], [0], color="red", alpha=1, linewidth=3, label="Buta Free-Nucl Peak Start/End pos", \
                                 linestyle="--")]
mmsq_after_1.legend(handles=custom_label, loc="upper left", bbox_to_anchor=(0.05, 0.9), ncol=6, fontsize=30)

plt.show()
# mmsq_after_1.savefig("./run_period_1/mmsq_after_1")

#### Helicity_E function 

###### HAE 

In [16]:
#HAE func
def HAE(epho_bins=[], cos_bins={}, mmsq_dict={}, df_dict={}):
    
    #HAE dict
    HAE_dict          = {}
    HAE_stat_err_dict = {}
    
    #keys
    epho_keys     = sorted(cos_bins.keys())
    cos_keys      = sorted(cos_bins.keys())
    
    #=============================================================================================
    #For each epho bins
    for epho_bin, j in zip(range(len(epho_keys)), cos_keys):

        #Dict to store dict or whole cos
        HAE_1epho_cos_dict          = {}    
        HAE_stat_err_1epho_cos_dict = {}    
    
        #keys for each cos_bin to access fit params
        cos_keys_keys = list(df_dict[epho_keys[epho_bin]].keys())

        #Each epho_bin has diff cos bins defined
        for cos_bin in range(len(cos_bins[j])-1):
        
            #=================================================================================================
            #DF of current bin
            df = mmsq_dict[epho_keys[epho_bin]][cos_bin]
        
            #Target polarization
            pol_tar      = np.abs(df["tar_Pol"].astype(float)).mean()

            #========================================================================================================
            #PARA and ANPA 
            #    -THIS SHOULD VARY DEPENDING ON TARGET POLARIZATION + bm_helicity
            PARA = len(df[(df["tar_Pol_sign"]==-1) & (df["bm_helicity"]==-1)]) + \
                   len(df[(df["tar_Pol_sign"]==1) & (df["bm_helicity"]==1)])
            
            ANPA = len(df[(df["tar_Pol_sign"]==-1) & (df["bm_helicity"]==1)]) + \
                   len(df[(df["tar_Pol_sign"]==1) & (df["bm_helicity"]==-1)])            

            #========================================================================================================
            #Photon Polarization
            pol_pho_avg = df["pol_pho"].astype(float).mean()
            
            #========================================================================================================
            #Dilution facotr
            dil_fact = df_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["df"]
            
            #========================================================================================================
            #Compute HAE
            if((PARA + ANPA) == 0):
                HAE = np.nan
            
            #If dil_fact undefineds since discarded bin
            elif(np.isnan(dil_fact)):
                HAE = np.nan
                
            #If good bins - HAE EQUATION HERE!!!
            elif (np.isnan(dil_fact) == False):
                HAE = (1/dil_fact) * (1/pol_pho_avg) * (1/(pol_tar)) * ((ANPA - PARA) / (PARA + ANPA))
                HAE = round(HAE, 4)
                
            #========================================================================================================
            #Statistical Uncertainty 
            #    -Refer to [Brant ex 6.1] for error of mean calc. Better than Bohm
            
            #df stat error
            df_stat_err = df_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]["buta_bf_intg_err"]
            
            #Target polarization error
            pol_tar_err = np.sqrt(np.sum(pow(df["tar_Pol_stat"], 2) / len(df)))
            
            #Photon polarization error
            pol_pho_avg_stat_err = np.sqrt(np.sum(pow(df["pol_pho_stat"], 2) / len(df)))
            
            #Fianl HAE_stat_err compute
            try:
                #W/ df_stat_err
                HAE_stat_err = np.sqrt(pow(HAE, 2)*(pow((df_stat_err/dil_fact), 2) + \
                                                    pow((pol_tar_err/pol_tar), 2) + \
                                                    pow((pol_pho_avg_stat_err/pol_pho_avg), 2) + \
                                                    (4*ANPA*PARA)/((ANPA + PARA)*pow((PARA - ANPA),2))))                
                
            except ZeroDivisionError:
                HAE_stat_err = np.nan
                
            #=================================================================================================
            #Append
            HAE_1epho_cos_dict["cos_{:07.3f}".format(cos_bins[cos_keys[epho_bin]][cos_bin])]          = HAE
            HAE_stat_err_1epho_cos_dict["cos_{:07.3f}".format(cos_bins[cos_keys[epho_bin]][cos_bin])] = HAE_stat_err
            
        #=================================================================================================
        HAE_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = HAE_1epho_cos_dict
        HAE_stat_err_dict["Eg_{:07.3f}".format(epho_bins[epho_bin])] = HAE_stat_err_1epho_cos_dict
    
    return HAE_dict, HAE_stat_err_dict

In [19]:
#Call HAE()
HAE_dict, HAE_stat_err_dict = HAE(epho_bins=epho_bins, cos_bins=cos_bins, \
                                  mmsq_dict=mmsq_dict, df_dict=df_dict)

###### FInal HAE PLOT 6

In [ ]:
#Plot HAE (CHECK)
HAE_plot, subplot = plt.subplots(2, 5, figsize=(30, 10))

#epho_keys
epho_keys = sorted(cos_bins.keys())
cos_keys  = sorted(cos_bins.keys())

#cos bins for SAID
cos_bins_theo   = [(i)/10 for i in range(-10, 11, 1)]

#Theo dict keys for SAID and MAID
theo_dict_keys      = list(theo_dict.keys())
theo_dict_MAID_keys = list(theo_dict_MAID.keys())
theo_dict_BNGA_keys = list(theo_dict_BNGA.keys())

#Cos_bins_cnt in 2d list
cos_bins_cnts = []
for i in sorted(cos_bins.keys()):
    cos_bins_cnts_i = []
    for j in range(len(cos_bins[i])-1):
        cnt_i           = 0.5*(cos_bins[i][j] + cos_bins[i][j+1])
        cnt_i           = round(cnt_i, 4)
        cos_bins_cnts_i.append(cnt_i)
    cos_bins_cnts.append(cos_bins_cnts_i)

#All HAE in 2d list
HAE_list = []
for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
    cos_keys_keys = list(HAE_dict[epho_keys[epho_bin]].keys())
    #HAE of 1epho bin - 10 cos bins
    HAE_list_1epho = []
    for cos_bin in range(len(cos_bins[j])-1):        
        HAE_i = HAE_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]
        HAE_list_1epho.append(HAE_i)
    HAE_list.append(HAE_list_1epho) 
    
#All HAE_stat_err in 2d list
HAE_stat_err_list = []
for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
    cos_keys_keys = list(HAE_stat_err_dict[epho_keys[epho_bin]].keys())
    #HAE of 1epho bin - 10 cos bins
    HAE_stat_err_list_1epho = []
    for cos_bin in range(len(cos_bins[j])-1):        
        HAE_stat_err_i = HAE_stat_err_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]
        HAE_stat_err_list_1epho.append(HAE_stat_err_i)
    HAE_stat_err_list.append(HAE_stat_err_list_1epho)     
    
#Plot HAE
for i in range(2):
    for j in range(5):
        #Create list of wanted values from the dictionary
        HAE_list_1epho          = HAE_list[5*i+j]
        HAE_stat_err_list_1epho = HAE_stat_err_list[5*i+j]
        
        #plot HAE
        #subplot[i, j].plot(cos_bins_cnts[5*i+j], HAE_list_1epho, "o", color="#4C72B0")
    
        subplot[i, j].errorbar(cos_bins_cnts[5*i+j], HAE_list_1epho, yerr=HAE_stat_err_list_1epho, \
                               fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, color="#001C7F", 
                               alpha=0.7)
        
#Theo SAID
subplot[0, 0].plot(cos_bins_theo, theo_dict[theo_dict_keys[20]], label="SAID", color="#55A868")
subplot[0, 1].plot(cos_bins_theo, theo_dict[theo_dict_keys[21]], label="SAID", color="#55A868")
subplot[0, 2].plot(cos_bins_theo, theo_dict[theo_dict_keys[22]], label="SAID", color="#55A868")
subplot[0, 3].plot(cos_bins_theo, theo_dict[theo_dict_keys[23]], label="SAID", color="#55A868")
subplot[0, 4].plot(cos_bins_theo, theo_dict[theo_dict_keys[24]], label="SAID", color="#55A868")
subplot[1, 0].plot(cos_bins_theo, theo_dict[theo_dict_keys[25]], label="SAID", color="#55A868")
subplot[1, 1].plot(cos_bins_theo, theo_dict[theo_dict_keys[26]], label="SAID", color="#55A868")
subplot[1, 2].plot(cos_bins_theo, theo_dict[theo_dict_keys[27]], label="SAID", color="#55A868")
subplot[1, 3].plot(cos_bins_theo, theo_dict[theo_dict_keys[28]], label="SAID", color="#55A868")
subplot[1, 4].plot(cos_bins_theo, theo_dict[theo_dict_keys[29]], label="SAID", color="#55A868")       

#Theo BNGA
subplot[0, 0].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[20]], label="BnGa", color="#8172B2")
subplot[0, 1].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[21]], label="BnGa", color="#8172B2")
subplot[0, 2].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[22]], label="BnGa", color="#8172B2")
subplot[0, 3].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[23]], label="BnGa", color="#8172B2")
subplot[0, 4].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[24]], label="BnGa", color="#8172B2")
subplot[1, 0].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[25]], label="BnGa", color="#8172B2")
subplot[1, 1].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[26]], label="BnGa", color="#8172B2")
subplot[1, 2].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[27]], label="BnGa", color="#8172B2")
subplot[1, 3].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[28]], label="BnGa", color="#8172B2")
subplot[1, 4].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[29]], label="BnGa", color="#8172B2")

##CBELSA
# subplot[0, 0].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
subplot[0, 1].errorbar(CB_cthe_bins, CB_dict["1.650"], yerr=CB_E_err_dict["1.650"], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[0, 2].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
subplot[0, 3].errorbar(CB_cthe_bins, CB_dict["1.770"], yerr=CB_E_err_dict["1.770"], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[0, 4].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
subplot[1, 0].errorbar(CB_cthe_bins, CB_dict["1.890"], yerr=CB_E_err_dict["1.890"], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[1, 1].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict["1.770"], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
subplot[1, 2].errorbar(CB_cthe_bins, CB_dict["2.010"], yerr=CB_E_err_dict["2.010"], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
subplot[1, 3].errorbar(CB_cthe_bins, CB_dict["2.130"], yerr=CB_E_err_dict["2.130"], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
subplot[1, 4].errorbar(CB_cthe_bins, CB_dict["2.250"], yerr=CB_E_err_dict["2.250"], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[2, 0].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[2, 1].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[2, 2].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[2, 3].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[2, 4].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[3, 0].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[3, 1].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[3, 2].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[3, 3].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[3, 4].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)

#Zero line
for i in range(2):
    for j in range(5):
        subplot[i, j].axhline(0, linestyle="-", color="black", alpha=0.6)
        
    
#Set title
for i in range(2):
    for j in range(5):
        subplot[i, j].set_title("$E_{{\gamma}} = [ {:1.2f}, {:1.2f}]$"\
                      .format(epho_bins[5*i+j], epho_bins[5*i+j+1]), fontsize=25) 
    
#labels, lim, grids, ticks font, legends
for i in subplot:
    for j in i:
        j.set_xlabel("$cos\\theta_{cm}$", fontsize=20)
        j.set_ylabel("E", rotation=0, fontsize=20)
        #lim
        j.set_ylim(-1.1, 1.1)
        j.set_xlim(-1, 1.)
        #grids
        j.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
        j.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")
        #Ticks fontsize change
        j.tick_params(labelsize=15) 
        #Legends
        #j.legend()    
    
#Spacing
HAE_plot.subplots_adjust(hspace=0.5)
HAE_plot.subplots_adjust(wspace=0.3)        

#axes for legneds
legend_axes = HAE_plot.add_axes([0.1, 1., 0.2, 0.05])
legend_axes.axis("off")

#Legends
custom_label = [mpl.patches.Patch(facecolor="#001C7F", alpha=0.7, label="CLAS"), \
                mpl.patches.Patch(facecolor="#C44E52", alpha=0.6, label="CBELSA"), \
                mpl.patches.Patch(facecolor="#55A868", alpha=0.7, label="SAID"), \
               mpl.patches.Patch(facecolor="#8172B2", alpha=0.7, label="BnGa")]
legend_axes.legend(handles=custom_label, loc="upper left", bbox_to_anchor=(0.1, 1.), ncol=4, fontsize=30)

plt.show()
# HAE_plot.savefig("./results_HAE/HAE_plot")

In [ ]:
#Plot HAE - W (CHECK)
HAE_plot, subplot = plt.subplots(2, 5, figsize=(30, 10))

#epho_keys
epho_keys = sorted(cos_bins.keys())
cos_keys  = sorted(cos_bins.keys())

#cos bins for SAID
cos_bins_theo   = [(i)/10 for i in range(-10, 11, 1)]

#Theo dict keys for SAID and MAID
theo_dict_keys      = list(theo_dict.keys())
theo_dict_MAID_keys = list(theo_dict_MAID.keys())
theo_dict_BNGA_keys = list(theo_dict_BNGA.keys())

#Cos_bins_cnt in 2d list
cos_bins_cnts = []
for i in sorted(cos_bins.keys()):
    cos_bins_cnts_i = []
    for j in range(len(cos_bins[i])-1):
        cnt_i           = 0.5*(cos_bins[i][j] + cos_bins[i][j+1])
        cnt_i           = round(cnt_i, 4)
        cos_bins_cnts_i.append(cnt_i)
    cos_bins_cnts.append(cos_bins_cnts_i)

#All HAE in 2d list
HAE_list = []
for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
    cos_keys_keys = list(HAE_dict[epho_keys[epho_bin]].keys())
    #HAE of 1epho bin - 10 cos bins
    HAE_list_1epho = []
    for cos_bin in range(len(cos_bins[j])-1):        
        HAE_i = HAE_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]
        HAE_list_1epho.append(HAE_i)
    HAE_list.append(HAE_list_1epho) 
    
#All HAE_stat_err in 2d list
HAE_stat_err_list = []
for epho_bin, j in zip(range(len(epho_keys)), cos_keys):
    cos_keys_keys = list(HAE_stat_err_dict[epho_keys[epho_bin]].keys())
    #HAE of 1epho bin - 10 cos bins
    HAE_stat_err_list_1epho = []
    for cos_bin in range(len(cos_bins[j])-1):        
        HAE_stat_err_i = HAE_stat_err_dict[epho_keys[epho_bin]][cos_keys_keys[cos_bin]]
        HAE_stat_err_list_1epho.append(HAE_stat_err_i)
    HAE_stat_err_list.append(HAE_stat_err_list_1epho)     
    
#Plot HAE
for i in range(2):
    for j in range(5):
        #Create list of wanted values from the dictionary
        HAE_list_1epho          = HAE_list[5*i+j]
        HAE_stat_err_list_1epho = HAE_stat_err_list[5*i+j]
        
        #plot HAE
        #subplot[i, j].plot(cos_bins_cnts[5*i+j], HAE_list_1epho, "o", color="#4C72B0")
    
        subplot[i, j].errorbar(cos_bins_cnts[5*i+j], HAE_list_1epho, yerr=HAE_stat_err_list_1epho, \
                               fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, color="#001C7F", 
                               alpha=0.7)
        
#Theo SAID
subplot[0, 0].plot(cos_bins_theo, theo_dict[theo_dict_keys[20]], label="SAID", color="#55A868")
subplot[0, 1].plot(cos_bins_theo, theo_dict[theo_dict_keys[21]], label="SAID", color="#55A868")
subplot[0, 2].plot(cos_bins_theo, theo_dict[theo_dict_keys[22]], label="SAID", color="#55A868")
subplot[0, 3].plot(cos_bins_theo, theo_dict[theo_dict_keys[23]], label="SAID", color="#55A868")
subplot[0, 4].plot(cos_bins_theo, theo_dict[theo_dict_keys[24]], label="SAID", color="#55A868")
subplot[1, 0].plot(cos_bins_theo, theo_dict[theo_dict_keys[25]], label="SAID", color="#55A868")
subplot[1, 1].plot(cos_bins_theo, theo_dict[theo_dict_keys[26]], label="SAID", color="#55A868")
subplot[1, 2].plot(cos_bins_theo, theo_dict[theo_dict_keys[27]], label="SAID", color="#55A868")
subplot[1, 3].plot(cos_bins_theo, theo_dict[theo_dict_keys[28]], label="SAID", color="#55A868")
subplot[1, 4].plot(cos_bins_theo, theo_dict[theo_dict_keys[29]], label="SAID", color="#55A868")       

#Theo BNGA
subplot[0, 0].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[20]], label="BnGa", color="#8172B2")
subplot[0, 1].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[21]], label="BnGa", color="#8172B2")
subplot[0, 2].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[22]], label="BnGa", color="#8172B2")
subplot[0, 3].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[23]], label="BnGa", color="#8172B2")
subplot[0, 4].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[24]], label="BnGa", color="#8172B2")
subplot[1, 0].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[25]], label="BnGa", color="#8172B2")
subplot[1, 1].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[26]], label="BnGa", color="#8172B2")
subplot[1, 2].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[27]], label="BnGa", color="#8172B2")
subplot[1, 3].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[28]], label="BnGa", color="#8172B2")
subplot[1, 4].plot(cos_bins_theo, theo_dict_BNGA[theo_dict_BNGA_keys[29]], label="BnGa", color="#8172B2")

##CBELSA
# subplot[0, 0].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
subplot[0, 1].errorbar(CB_cthe_bins, CB_dict["1.650"], yerr=CB_E_err_dict["1.650"], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[0, 2].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
subplot[0, 3].errorbar(CB_cthe_bins, CB_dict["1.770"], yerr=CB_E_err_dict["1.770"], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[0, 4].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
subplot[1, 0].errorbar(CB_cthe_bins, CB_dict["1.890"], yerr=CB_E_err_dict["1.890"], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[1, 1].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict["1.770"], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
subplot[1, 2].errorbar(CB_cthe_bins, CB_dict["2.010"], yerr=CB_E_err_dict["2.010"], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
subplot[1, 3].errorbar(CB_cthe_bins, CB_dict["2.130"], yerr=CB_E_err_dict["2.130"], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
subplot[1, 4].errorbar(CB_cthe_bins, CB_dict["2.250"], yerr=CB_E_err_dict["2.250"], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[2, 0].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[2, 1].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[2, 2].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[2, 3].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[2, 4].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[3, 0].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[3, 1].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[3, 2].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[3, 3].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)
# subplot[3, 4].errorbar(CB_cthe_bins, CB_dict[""], yerr=CB_E_err_dict[""], fmt="o", markeredgewidth=1, capsize=5, elinewidth=0.5, label="CBELSA", color="#C44E52", alpha=0.6)

#Zero line
for i in range(2):
    for j in range(5):
        subplot[i, j].axhline(0, linestyle="-", color="black", alpha=0.6)
        
    
# #Set title
# for i in range(2):
#     for j in range(5):
#         subplot[i, j].set_title("$E_{{\gamma}} = [ {:1.2f}, {:1.2f}]$"\
#                       .format(epho_bins[5*i+j], epho_bins[5*i+j+1]), fontsize=25) 
        
#Set title
for i in range(2):
    for j in range(5):
        
        m1  = 0.93827
        w   = math.sqrt(pow(m1, 2) + 2*m1*epho_bins[5*i+j])
        w_1 = math.sqrt(pow(m1, 2) + 2*m1*epho_bins[5*i+j+1])
        
        subplot[i, j].set_title(" $E_{{\gamma}} = [ {:1.2f}, {:1.2f}]$ \n $W = [ {:1.2f}, {:1.2f}]$"\
                      .format(epho_bins[5*i+j], epho_bins[5*i+j+1], \
                              w, w_1), fontsize=25)         
    
#labels, lim, grids, ticks font, legends
for i in subplot:
    for j in i:
        j.set_xlabel("$cos\\theta_{cm}$", fontsize=20)
        j.set_ylabel("E", rotation=0, fontsize=20)
        #lim
        j.set_ylim(-1.1, 1.1)
        j.set_xlim(-1, 1.)
        #grids
        j.get_xaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.get_yaxis().set_minor_locator(mpl.ticker.AutoMinorLocator(5))
        j.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
        j.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")
        #Ticks fontsize change
        j.tick_params(labelsize=15) 
        #Legends
        #j.legend()    
    
#Spacing
HAE_plot.subplots_adjust(hspace=0.75)
HAE_plot.subplots_adjust(wspace=0.3)        

#axes for legneds
legend_axes = HAE_plot.add_axes([0.1, 1.05, 0.2, 0.05])
legend_axes.axis("off")

#Legends
custom_label = [mpl.patches.Patch(facecolor="#001C7F", alpha=0.7, label="HAE"), \
                mpl.patches.Patch(facecolor="#C44E52", alpha=0.6, label="CBELSA"), \
                mpl.patches.Patch(facecolor="#55A868", alpha=0.7, label="SAID"), \
               mpl.patches.Patch(facecolor="#8172B2", alpha=0.7, label="BnGa")]
legend_axes.legend(handles=custom_label, loc="upper left", bbox_to_anchor=(0.1, 1.), ncol=4, fontsize=30)

plt.show()
# HAE_plot.savefig("./results_HAE/HAE_plot")

# Save and Resume 

## Data Save/Resume - all runs COMBINED

###### For BF + SF + DF + MMSQ

In [3]:
#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH_4 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_4/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH_5 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_5/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH_6 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_6/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

#Read Initial PlUS (ML* + zvtx + SF)
df_GMSH_7 = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_7/df_GMSH_dstmaker_Ppi0_4_initial_plus.pkl")

#Combine
df_GMSH = pd.DataFrame()
df_GMSH = pd.concat([df_GMSH_4, df_GMSH_5, df_GMSH_6, df_GMSH_7], \
                     ignore_index=True)

#Delete unused
del df_GMSH_4
del df_GMSH_5
del df_GMSH_6
del df_GMSH_7

In [ ]:
"""
-DONT DROP - doing HAE or mmsq step
-DROP      - if doing SF and DF step
"""

In [5]:
#Drop unneeded columns to speed up computation
column_drop_list = []
columns          = df_GMSH.columns
needed_columns   = ["cthe_cm", "epho", "el_pc_mmsq_pi0", "T_fin"]
drop_list        = []

for i in columns:
    if (i not in needed_columns):
        drop_list.append(i)
        
#Drop-
df_GMSH.drop(columns=drop_list, axis=1, inplace=True)

###### After MMSQ - all runs combined 

In [ ]:
# #Save after mmsq selection: mmsq_dict
# mmsq = open("./results_mmsq_all/high_energy/mmsq_dict.pkl", "wb")
# pickle.dump(mmsq_dict, mmsq)
# mmsq.close()

In [18]:
#Read MMSQ selection: mmsq_dict
mmsq       = open("./results_mmsq_all/high_energy/mmsq_dict.pkl", "rb")
mmsq_dict  = pickle.load(mmsq)
mmsq.close()

## Results: HAE, sf, df, bf 

#### bf_params_dict 

In [ ]:
# #Write bf_params_dict
# bf_save = open("./results_sf_df/high_energy/bf_params_dict.pkl", "wb")
# pickle.dump(bf_params_dict, bf_save)
# bf_save.close()

In [3]:
#Read SF
bf_save         = open("./results_sf_df/high_energy/bf_params_dict.pkl", "rb")
bf_params_dict  = pickle.load(bf_save)
bf_save.close()

#### SF

In [ ]:
# #Write SF
# sf_save = open("./results_sf_df/high_energy/sf_dict.pkl", "wb")
# pickle.dump(sf_dict, sf_save)
# sf_save.close()

In [4]:
#Read SF
sf_save  = open("./results_sf_df/high_energy/sf_dict.pkl", "rb")
sf_dict  = pickle.load(sf_save)
sf_save.close()

In [ ]:
# #Write SF stat err
# sf_stat_err_save = open("./results_sf_df/high_energy/sf_stat_err_dict.pkl", "wb")
# pickle.dump(sf_stat_err_dict, sf_stat_err_save)
# sf_stat_err_save.close()

In [ ]:
# #Read SF stat err
# sf_stat_err_save  = open("./results_sf_df/high_energy/sf_stat_err_dict.pkl", "rb")
# sf_stat_err_dict  = pickle.load(sf_stat_err_save)
# sf_stat_err_save.close()

#### DF 

In [ ]:
# #Write df
# df_save = open("./results_sf_df/high_energy/df_dict.pkl", "wb")
# pickle.dump(df_dict, df_save)
# df_save.close()

In [5]:
#Read df
df_save  = open("./results_sf_df/high_energy/df_dict.pkl", "rb")
df_dict  = pickle.load(df_save)
df_save.close()

In [ ]:
# #Write DF stat err
# df_stat_err_save = open("./results_sf_df/high_energy/df_stat_err_dict.pkl", "wb")
# pickle.dump(df_stat_err_dict, df_stat_err_save)
# df_stat_err_save.close()

In [ ]:
# #Read DF stat err
# df_stat_err_save  = open("./results_sf_df/high_energy/df_stat_err_dict.pkl", "rb")
# df_stat_err_dict  = pickle.load(df_stat_err_save)
# df_stat_err_save.close()

#### HAE

In [ ]:
# #Write HAE
# HAE_save = open("./results_HAE/binning_1/all_runs_combined/high_energy/HAE_dict.pkl", "wb")
# pickle.dump(HAE_dict, HAE_save)
# HAE_save.close()

In [ ]:
# #Write HAE_stat_err
# HAE_stat_err_save = open("./results_HAE/binning_1/all_runs_combined/high_energy/HAE_stat_err_dict.pkl", "wb")
# pickle.dump(HAE_stat_err_dict, HAE_stat_err_save)
# HAE_stat_err_save.close()

In [6]:
#Read HAE
HAE_save  = open("./results_HAE/binning_1/all_runs_combined/high_energy/HAE_dict.pkl", "rb")
HAE_dict  = pickle.load(HAE_save)
HAE_save.close()

In [7]:
#Read HAE_stat_err
HAE_stat_err_save  = open("./results_HAE/binning_1/all_runs_combined/high_energy/HAE_stat_err_dict.pkl", "rb")
HAE_stat_err_dict  = pickle.load(HAE_stat_err_save)
HAE_stat_err_save.close()

## Binning

###### High Energy 

In [2]:
# #Read df from each runs (MUST)`
# df_GMSH = pd.DataFrame()

# for i in range(4, 8):
#     df_GMSH_i = pd.read_pickle("./dstmaker_Ppi0_4_eloss/high_{:d}/df_GMSH_dstmaker_Ppi0_4_initial.pkl".format(i))[["epho", "cthe_cm", "el_pc_mmsq_pi0"]]
#     df_GMSH = df_GMSH.append(df_GMSH_i, ignore_index=True)

# df_GMSH = df_GMSH.reset_index(drop=True)

In [ ]:
# #Plot all events' mmsq (MUST)
# global_mmsq = plt.figure(figsize=(15, 10))

# #Histogram
# n, b, p = plt.hist(df_GMSH["el_pc_mmsq_pi0"], bins=6500, label="$M_X^2$", alpha=0.7, color="#4C72B0")

# #Stats
# bin_max = np.argmax(n)
# peak    = b[bin_max]
# mean    = df_GMSH["el_pc_mmsq_pi0"].mean()
# stddev  = df_GMSH["el_pc_mmsq_pi0"].std()

# #===================================================================================================
# #Selection range for High energy
# #    -SAVE to be used for dropping events
# frst_ln  = 0.035 - 0.05*stddev
# scnd_ln  = 0.035 + 0.05*stddev
# #===================================================================================================

# #Draw lines
# plt.axvline(frst_ln, linestyle="-", linewidth=3, alpha=0.5, color="#C44E52")
# plt.axvline(scnd_ln,  linestyle="--", linewidth=3, alpha=0.5, color="#C44E52")

# #Texts
# plt.text(0.05, 0.8, " {:<12} {:>10.3f} \n {:<12} {:>10.3f} \n {:<21} {:>10.3f}" \
#          .format("peak", peak, "mean", mean, "$\sigma$", stddev), \
#          fontsize=22, bbox=dict(facecolor='none', pad=5.0), transform=plt.gca().transAxes)

# #Grids
# plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
# plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
# plt.grid(b=True, which="major", color="gray", linewidth=1.0, linestyle="--")
# plt.grid(b=True, which="minor", color="gray", linewidth=0.5, linestyle="--")

# #For 1 plt
# plt.xticks(fontsize=22.5, fontweight="ultralight")
# plt.yticks(fontsize=22.5, fontweight="ultralight")

# #Y-Axis Scaling
# def adjust_y_axis(x, pos):
#     return "{:.0f}".format(x/10000)
# plt.gca().yaxis.set_major_formatter(mpl.ticker.FuncFormatter(adjust_y_axis))

# #Labels, ticks, lim
# plt.xlim(-2.0, 2.0)
# plt.xlabel("$M_X^2$ $[GeV^2/c^4]$", fontsize=30)
# plt.ylabel("Counts $(x10^4)$", fontsize=30)

# #Legends
# custom_label = [mpl.lines.Line2D([0], [0], color="#C44E52", alpha=0.7, linewidth=3, \
#                                  label="peak - $\sigma$    = {:.3f}".format(frst_ln), linestyle="-"), 
#                 mpl.lines.Line2D([0], [0], color="#C44E52", alpha=0.7, linewidth=3, \
#                                  label="peak + $\sigma$   = {:.3f}".format(scnd_ln), linestyle="--")]
# plt.legend(handles=custom_label, fontsize=22, loc="upper right")

# # plt.legend()
# plt.show()
# # global_mmsq.savefig("./binning_global_mmsq_high_energy")

In [ ]:
# #Drop events outside of +/- sigma (MUST)
# df_GMSH.drop(df_GMSH[(df_GMSH["el_pc_mmsq_pi0"] < frst_ln) | (df_GMSH["el_pc_mmsq_pi0"] > scnd_ln)].index, \
#              inplace=True)

In [ ]:
# #Binning Method 1 (MUST)
# bin_partition_eg_perc = [i/10 for i in range(0, 11, 1)]
# # bin_partition_eg_perc = [i/20 for i in range(0, 21, 1)]
# bin_pos_eg_list    = df_GMSH["epho"].quantile(bin_partition_eg_perc).tolist()
# epho_bins          = [round(i, 4) for i in bin_pos_eg_list]

# #cos_bins are dict now
# cos_bins = {}
# bin_partition_cos_perc = [i/20 for i in range(0, 21, 1)]
# # bin_partition_cos_perc = [i/10 for i in range(0, 11, 1)]

# #Partitions
# for i in range(len(epho_bins)-1):
#     #cos binning per epho bin
#     bin_pos_cos_list = df_GMSH["cthe_cm"][(epho_bins[i] <= df_GMSH["epho"]) & \
#                                           (df_GMSH["epho"] <= epho_bins[i+1])] \
#                                          .quantile(bin_partition_cos_perc).tolist()

#     #Into cos_bins dict
#     cos_bins["Eg_{:07.3f}".format(epho_bins[i])] = [round(i, 4) for i in bin_pos_cos_list]

# #Print
# print(bin_partition_eg_perc)
# print(len(bin_partition_eg_perc))
# print(bin_partition_cos_perc)
# print(len(bin_partition_cos_perc))
# print("epho_bins = ", epho_bins)
# print("len(epho_bins) = ", len(epho_bins))
# #Print cos bins for each epho bins
# for i in sorted(cos_bins.keys()):
#     print(i, cos_bins[i])

In [ ]:
# #epho vs angle -(CHECK)
# binning = plt.figure(figsize=(15, 15))

# #Hist
# plt.hist2d(df_GMSH["cthe_cm"], df_GMSH["epho"], bins=100, cmap="jet", cmin=0, \
#            norm=mpl.colors.LogNorm())

# #Lines to indicate epho bins
# for i, k in zip(sorted(cos_bins.keys()), range(len(epho_bins))):
#     for j in range(len(cos_bins[i])):
#         plt.hlines(y=epho_bins[k], xmin=cos_bins[i][0], xmax=cos_bins[i][-1], \
#                    linestyle="-", linewidth=1.75, color="black")
#         plt.hlines(y=epho_bins[k+1], xmin=cos_bins[i][0], xmax=cos_bins[i][-1], \
#                    linestyle="-", linewidth=1.75, color="black")
        
# #Manually draw last epho line
# plt.hlines(y=epho_bins[-1], xmin=cos_bins[sorted(cos_bins.keys())[-1]][0], \
#            xmax=cos_bins[sorted(cos_bins.keys())[-1]][-1], \
#            linestyle="-", linewidth=1.75, color="black")
        
        
# #Lines to indicate cos bins    
# for i, k in zip(sorted(cos_bins.keys()), range(len(epho_bins)-1)):
#     for j in range(len(cos_bins[i])):
#         plt.vlines(x=cos_bins[i][j], ymin=epho_bins[k], ymax=epho_bins[k+1], \
#                    linestyle="-", linewidth=1.75, color="black")
    
# #lim, label, ticks    
# plt.xlim(-1.1, 1.1)
# plt.ylim(1.55, 2.4)
# plt.xlabel("$cos\\theta_{cm}$", fontsize=40)
# plt.ylabel("$E_{\gamma}$", fontsize=40)
# plt.xticks(fontsize=20)
# plt.yticks(fontsize=20)

# #Grids
# plt.axes().xaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
# plt.axes().yaxis.set_minor_locator(mpl.ticker.AutoMinorLocator(5))
# plt.grid(b=True, which="major", color="gray", linewidth=0.5, linestyle="--")
# plt.grid(b=True, which="minor", color="gray", linewidth=0.25, linestyle="--")

# cbaxes = binning.add_axes([0.91, 0.125, 0.02, 0.755])   # x_pos, y_pos, width, length
# cbaxes.tick_params(labelsize=20)
# plt.colorbar(cax=cbaxes)
# plt.show
# # binning.savefig("./binning_high_energy")

In [ ]:
# #Binning method 1 # of events in each bins (CHECK)
# for i, j in zip(range(len(epho_bins)-1), sorted(cos_bins.keys())):
#     print("==============================", epho_bins[i], "=============================")
#     for k in range(len(cos_bins[j])-1):

#         print(len(df_GMSH[(epho_bins[i]   <= df_GMSH["epho"])    & (df_GMSH["epho"]    <= epho_bins[i+1]) & \
#                           (cos_bins[j][k] <= df_GMSH["cthe_cm"]) & (df_GMSH["cthe_cm"] <= cos_bins[j][k+1])]))
        

###### Save & Read last Binning (High Energy)

In [ ]:
# #Save epho
# epho_dict = {"epho_bins": epho_bins}
# df_epho_dict = pd.DataFrame(epho_dict)

# #save to csv
# df_epho_dict.to_csv("./results_Binning/high_energy/epho_bins")

In [ ]:
# #Save cos_bins dict
# with open("./results_Binning/high_energy/cos_bins", "w") as f:
#     writer = csv.writer(f, delimiter="\t")
    
#     for key in sorted(cos_bins.keys()):
#         writer.writerow([key] + cos_bins[key])

In [ ]:
#Read
df_epho_dict = pd.read_csv("./results_Binning/high_energy/epho_bins", index_col=0)
#Into epho_bins list
epho_bins = df_epho_dict[df_epho_dict.columns[0]].tolist()

print(epho_bins)

In [ ]:
#Read cos_bins dict
df       = pd.read_csv("./results_Binning/high_energy/cos_bins", index_col=0, delimiter="\t", header=None)
cos_bins = df.transpose().to_dict(orient="list")

#Print
for i in sorted(cos_bins.keys()):
    for j in range(len(cos_bins[i])):
        cos_bins[i] = [round(a, 4) for a in cos_bins[i]]
    print(i, cos_bins[i])

## CBELSA Results

In [10]:
#CBELSA epho bins
CB_epho_bins = [0.615, 0.645, 0.675, 0.705, 0.735, 0.765, 0.795, 0.825, 0.855, 0.885, 0.915, 0.945, 0.975, 1.005, \
                1.035, 1.065, 1.095, 1.125, 1.155, 1.185, 1.215, 1.260, 1.320, 1.380, 1.440, 1.500, 1.560, 1.650, \
                1.770, 1.890, 2.010, 2.130, 2.250]

In [11]:
#CBELSA cos bins
CB_cthe_bins = [round(np.cos(math.radians(21.56)), 2), round(np.cos(math.radians(36.87)), 2), round(np.cos(math.radians(47.93)), 2), round(np.cos(math.radians(57.99)), 2), round(np.cos(math.radians(66.42)), 2), round(np.cos(math.radians(74.34)), 2), round(np.cos(math.radians(82.53)), 2), 
                round(np.cos(math.radians(90.00)), 2), round(np.cos(math.radians(97.47)), 2), round(np.cos(math.radians(105.66)), 2), round(np.cos(math.radians(113.58)), 2), round(np.cos(math.radians(122.00)), 2), \
                round(np.cos(math.radians(132.07)), 2), round(np.cos(math.radians(143.13)), 2), round(np.cos(math.radians(158.43)), 2)]

# CB_clab_bins = [21.56, 36.87, 47.93, 57.99, 66.42, \
#                 74.34, 82.53, 90.00, 97.47, 105.66, \
#                 113.58, 122.00, 132.07, 143.13, 158.43]

print(CB_cthe_bins)
# print(len(CB_cthe_bins))
# print(CB_clab_bins)

[0.93, 0.8, 0.67, 0.53, 0.4, 0.27, 0.13, 0.0, -0.13, -0.27, -0.4, -0.53, -0.67, -0.8, -0.93]


In [12]:
#CBELSA E results
CB_E = [[np.nan, np.nan, 0.0015, -0.0629, -0.1353, -0.3661, -0.2979, -0.1994, -0.2636, -0.0663, -0.0172 ,0.1123, 0.3991, 0.7536, 0.8491],                 

        [np.nan, np.nan, -0.1692, -0.0021, -0.1265, 0.0663, -0.1704, -0.0726, -0.0623, -0.0653, 0.1143, 0.2125, 0.5552, 0.8889, 1.4455],

        [np.nan, np.nan, 0.0156, 0.1954, 0.0027, -0.0823, -0.0034, 0.0394, 0.0915, 0.0747, 0.2820, 0.2827, 0.4554, 0.7236, 1.2329],

        [np.nan, np.nan, 0.1281, -0.0234, -0.0574, -0.0038, 0.0019, 0.0075, 0.0558, 0.2420, 0.2290, 0.4753, 0.2824, 0.7036, 1.0317],

        [np.nan, 0.5582, 0.1372, 0.1828, -0.0032, -0.0165, 0.0333, 0.0278, 0.0887, 0.0839, 0.2838, 0.3589, 0.4580, 0.5474, 0.7395],

        [np.nan, -0.2082, 0.2728, 0.1172, -0.0282, -0.0379, -0.0224, -0.0586, -0.0005, -0.0258,  0.0951, 0.1593, 0.0857, 0.2533, 0.5864],

        [np.nan, 0.2018, 0.5574, 0.2934, 0.1629, 0.1160, -0.0203, -0.0034, -0.1162, -0.1228, -0.1000, -0.0910, -0.0495, 0.1819, 0.7059],

        [np.nan, 0.3512, 0.3159, 0.4468, 0.2317, 0.1464, -0.0198, -0.0562, 0.0408, -0.0438, -0.0750, -0.1247, -0.0150, 0.1957, 0.6391],

        [np.nan, 0.4744, 0.7781, 0.6678, 0.5257, 0.1691, -0.1074, -0.0554, -0.1662, -0.1879, -0.3567, -0.2905, -0.2377, 0.0272, 0.4681],

        [np.nan, 0.5148, 0.7452, 1.0316, 0.3470, 0.3642, 0.0690, -0.0942, -0.2309, -0.3624, -0.2520, -0.3128, -0.3627, -0.1052,  0.2241],

        [np.nan, 0.3823, 0.5604, 0.6436, 0.6631, 0.2328, 0.0280, -0.0662, -0.2732, -0.4590, -0.5066, -0.5475, -0.6087, -0.3126, 0.1329],

        [np.nan, -0.1274, 0.3183, 0.4101, 0.6431, 0.3035, -0.0083, -0.3579, -0.4131, -0.4783, -0.7549, -0.7243, -0.6907, -0.4782, -0.1360],

        [np.nan, -0.2946, -0.0825, 0.3722, 0.3333, 0.3959, 0.1548, -0.0509, -0.4148, -0.6002, -0.7086, -0.7216, -0.6863, -0.5778, -0.3427],

        [np.nan, -0.4603, -0.1159, 0.0131, 0.1555, 0.1071, 0.0297, -0.2945, -0.2576, -0.7003, -0.6245, -0.7323, -0.7239, -0.7635, -0.1133],

        [np.nan, -0.3184, -0.2973, -0.1671, -0.0749, 0.0438, 0.0277, 0.0005, -0.2995, -0.7181, -0.6503, -0.7071, -0.6938, -0.5213, -0.1651],

        [np.nan, -0.2337, -0.1828, -0.1166, -0.0170, -0.0614, 0.1062, 0.0324, -0.2395, -0.4859, -0.6554, -0.8636, -0.8546, -0.5470, 0.0502],

        [np.nan, -0.1904, -0.1449, -0.0965, 0.0027, 0.2382, 0.2797, -0.0581, -0.3199, -0.2689, -0.7832, -0.7907, -0.8087, -0.3852, 0.4191],

        [np.nan, -0.2461, 0.1715, 0.0680, 0.1345, 0.4459, 0.3530, 0.0715, -0.0928, -0.6950, -0.6919, -0.8145, -0.6714, -0.2320, 0.1638],

        [np.nan, 0.0886, -0.1207, 0.0911, 0.2705, 0.4264, 0.2284, 0.3408, 0.1137, -0.2690, -0.4694, -0.7753, -0.7382, -0.0602, 0.5163],

        [np.nan, -0.0917, 0.2661, 0.3544, 0.1817, 0.4449, 0.4648, 0.2505, 0.1568, -0.4094, -0.7125, -0.7109, -0.5650, -0.2797, 0.4065],

        [np.nan, -0.0985, 0.2592, -0.0552, 0.2020, 0.4248, 0.1987, 0.1501, 0.0052, -0.5989, -0.5642, -0.5829, -0.7876, -0.2959, 0.3163],

        [0.3467, -0.1654, -0.1335, 0.2404, 0.0958, 0.2173, 0.1270, 0.1986, -0.3027, -0.4195, -0.6013, -0.4076, -0.5815, -0.3197, 0.3967],

        [-0.5779, -0.3456, -0.1538, -0.1044, -0.0306, -0.0190, -0.2370, -0.3266, -0.4607, -0.4713, -0.5742, -0.7493, -0.4236, -0.3338, 0.1706],

        [-0.1127, -0.2864, -0.1445, -0.0395, -0.0519, -0.3010, -0.5393, -0.4986, -0.6986, -0.5865, -0.4831, -0.5093, -0.5339, -0.2656, 0.1925],

        [-0.2229, -0.2297, -0.3603, -0.5438, -0.5006, -0.5919, -0.7950, -0.7361, -0.6607, -0.4303, -0.4748, -0.3804, -0.4513, -0.5148, 0.0733],

        [-0.1060, -0.2223, -0.1041, -0.3589, -0.5930, -0.7952, -0.7828, -0.7641, -0.5847, -0.6211, -0.4033, -0.3009, -0.3869, -0.4659, 0.1745],

        [-0.1130, 0.0021, -0.4931, -0.3416, -1.0154, -0.9438, -0.9927, -0.8791, -0.6881, -0.6643, -0.1848, -0.3092, -0.2532, -0.3694, -0.0098],

        [0.1885, -0.0212, 0.0193, 0.2122, -0.5132, -1.4549, -0.9943, -0.8358, -0.6744, -0.5688, -0.3099, -0.2194, -0.5403, -0.4332, 0.0326],

        [0.0290, 0.1191, 0.0332, 0.5761, -0.5948, -0.9595, -0.8040, -0.8159, -0.7263, -0.5527, -0.1850, -0.1406, -0.2772, -0.4588, 0.3200],

        [0.0149, 0.2045, 0.2014, -0.3656, -0.7657, -1.1956, -1.0687, -0.9365, -0.7623, -0.4845, -0.3478, -0.2299, -0.3320, -0.6107, 0.0357],

        [0.2938, 0.2697, 0.2995, -0.2255, -0.3602, -0.7192, -1.0092, -0.7713, -0.7456, -0.4490, -0.6460, -0.0840, -0.2686, -0.8203, -0.7747],

        [0.2496, 0.3684, -0.1458, 0.3769, -0.1954, -0.6625, -1.1378, -0.8401, -0.8751, -0.5348, -0.1520, 0.1541, 0.8396, -0.2626, -1.4107],

        [0.1958, 0.1467, 0.0202, -0.4788, -0.4607, -0.3732, -0.6079, -0.5856, -0.5933, -0.3230, -0.3822, 0.4657, np.nan, np.nan, np.nan]]

In [13]:
#CBELSA E errors
CB_E_err = [[np.nan, np.nan, 0.1899, 0.0927, 0.0887, 0.0795, 0.0738, 0.0744, 0.0694, 0.0734, 0.0755, 0.0837, 0.1325, 0.1582, 0.3424],

            [np.nan, np.nan, 0.2435, 0.1098, 0.1196, 0.0964, 0.0860, 0.0815, 0.0831, 0.0863, 0.0888, 0.0945, 0.1363, 0.1578, 0.3541],

            [np.nan, np.nan, 0.1569, 0.0997, 0.1059, 0.0837, 0.0758, 0.0767, 0.0736, 0.0762, 0.1002, 0.0916, 0.1107, 0.1364, 0.2727],

            [np.nan, np.nan, 0.1101, 0.0822, 0.0694, 0.0633, 0.0717, 0.0665, 0.0669, 0.0655, 0.0682, 0.0778, 0.1150, 0.1190, 0.2199],

            [np.nan, 0.3759, 0.0931, 0.0667, 0.0575, 0.0599, 0.0518, 0.0517, 0.0512, 0.0538, 0.0572, 0.0618, 0.0755, 0.0883, 0.1398],

            [np.nan, 0.3626, 0.0791, 0.0617, 0.0614, 0.0524, 0.0498, 0.0551, 0.0530, 0.0539, 0.0548, 0.0627, 0.0750, 0.0774, 0.1248],

            [np.nan, 0.2737, 0.1017, 0.0828, 0.0712, 0.0625, 0.0623, 0.0627, 0.0578, 0.0600, 0.0635, 0.0684, 0.0705, 0.0962, 0.1450],

            [np.nan, 0.2755, 0.1580, 0.0925, 0.0827, 0.0704, 0.0663, 0.0747, 0.0704, 0.0641, 0.0667, 0.0790, 0.0867, 0.0852, 0.1511],

            [np.nan, 0.3160, 0.1792, 0.1064, 0.1136, 0.0761, 0.0761, 0.0699, 0.0694, 0.0726, 0.0919, 0.0719, 0.0828, 0.0916, 0.1340],

            [np.nan, 0.2607, 0.1570, 0.1497, 0.1078, 0.0966, 0.0758, 0.0751, 0.0787, 0.0712, 0.0789, 0.0689, 0.0738, 0.0795, 0.1305],

            [np.nan, 0.2007, 0.1499, 0.1376, 0.1292, 0.0956, 0.0893, 0.0802, 0.0931, 0.0776, 0.0782, 0.0773, 0.0755, 0.0816, 0.1215],

            [np.nan, 0.1306, 0.1109, 0.1042, 0.1246, 0.0966, 0.0789, 0.0780, 0.0693, 0.0663, 0.0735, 0.0637, 0.0646, 0.0700, 0.1031],

            [np.nan, 0.1159, 0.0853, 0.1024, 0.0978, 0.1004, 0.0773, 0.0882, 0.0775, 0.0761, 0.0745, 0.0628, 0.0689, 0.0893, 0.1082],

            [np.nan, 0.1001, 0.0872, 0.0865, 0.0797, 0.0859, 0.0838, 0.0947, 0.0800, 0.0899, 0.0724, 0.0679, 0.0663, 0.0920, 0.1206],

            [np.nan, 0.0890, 0.0811, 0.0727, 0.0740, 0.0862, 0.0753, 0.0736, 0.0747, 0.0805, 0.0657, 0.0730, 0.0688, 0.0700, 0.0959],

            [np.nan, 0.0889, 0.0826, 0.0882, 0.0782, 0.0847, 0.0910, 0.0987, 0.0930, 0.0899, 0.0816, 0.0949, 0.1058, 0.0904, 0.1300],

            [np.nan, 0.1099, 0.1116, 0.0904, 0.0865, 0.0881, 0.0964, 0.1039, 0.1201, 0.1042, 0.1035, 0.0942, 0.0937, 0.1136, 0.1502],

            [np.nan, 0.0989, 0.1310, 0.0902, 0.0779, 0.0793, 0.0921, 0.1020, 0.1124, 0.1498, 0.1291, 0.1015, 0.0913, 0.1175, 0.1298],

            [np.nan, 0.1073, 0.1132, 0.1008, 0.0913, 0.0836, 0.1053, 0.1275, 0.1085, 0.1231, 0.1467, 0.0948, 0.1057, 0.1221, 0.1405],

            [np.nan, 0.1470, 0.1395, 0.1181, 0.0977, 0.1033, 0.1246, 0.1125, 0.1260, 0.1322, 0.1276, 0.1254, 0.1090, 0.1193, 0.1409],

            [np.nan, 0.1396, 0.1740, 0.1610, 0.1377, 0.1692, 0.1205, 0.1513, 0.1499, 0.2320, 0.1412, 0.1366, 0.1578, 0.1599, 0.1496],

            [0.2597, 0.0806, 0.0962, 0.1030, 0.0850, 0.0976, 0.0841, 0.0912, 0.0944, 0.1020, 0.1076, 0.0974, 0.1026, 0.0854, 0.0963],

            [0.2085, 0.0703, 0.0863, 0.1214, 0.0866, 0.0713, 0.0838, 0.0707, 0.0822, 0.0931, 0.0879, 0.0866, 0.0800, 0.0777, 0.0783],

            [0.1407, 0.0688, 0.0904, 0.1381, 0.1146, 0.0888, 0.0775, 0.0906, 0.0836, 0.0968, 0.0903, 0.0923, 0.1088, 0.0759, 0.0800],

            [0.1362, 0.0565, 0.0914, 0.1900, 0.1552, 0.1083, 0.0769, 0.0767, 0.0729, 0.0884, 0.0885, 0.0865, 0.1326, 0.0970, 0.0761],

            [0.1079, 0.0629, 0.0919, 0.1775, 0.2710, 0.1697, 0.0700, 0.0679, 0.0764, 0.0779, 0.0893, 0.0867, 0.1128, 0.0838, 0.0890],

            [0.1435, 0.0645, 0.1154, 0.2246, 0.3466, 0.2283, 0.1083, 0.0755, 0.0863, 0.1043, 0.1267, 0.1003, 0.1041, 0.1042, 0.1019],

            [0.1225, 0.0496, 0.0742, 0.1882, 0.1995, 0.1485, 0.0720, 0.0572, 0.0557, 0.0720, 0.0697, 0.0918, 0.0966, 0.1025, 0.0875],

            [0.0912, 0.0564, 0.0759, 0.2071, 0.3689, 0.1941, 0.0965, 0.0747, 0.0699, 0.0746, 0.0828, 0.0911, 0.1293, 0.1605, 0.1563],

            [0.0970, 0.0618, 0.1043, 0.2173, 0.2756, 0.2000, 0.1102, 0.0826, 0.1124, 0.0800, 0.1023, 0.1381, 0.1390, 0.1891, 0.1839],

            [0.1081, 0.0621, 0.1105, 0.1783, 0.1666, 0.1269, 0.1234, 0.0936, 0.1111, 0.0878, 0.2125, 0.1585, 0.2609, 0.4244, 0.2998],

            [0.1064, 0.0772, 0.1478, 0.4112, 0.4194, 0.1988, 0.2733, 0.1641, 0.1163, 0.1485, 0.1779, 0.1850, 1.3129, 0.5791, 0.3868],

            [0.1262, 0.0915, 0.1598, 0.5493, 0.3833, 0.4234, 0.1773, 0.1729, 0.1189, 0.1639, 0.2029, 0.2736, np.nan, np.nan, np.nan]]

In [14]:
#dict
CB_dict = {}
CB_E_err_dict = {}

for i in range(len(CB_epho_bins)):
    CB_dict["{:.3f}".format(CB_epho_bins[i])] = CB_E[i]

for i in range(len(CB_epho_bins)):
    CB_E_err_dict["{:.3f}".format(CB_epho_bins[i])] = CB_E_err[i]

In [ ]:
# for key in sorted(CB_dict.keys()):
#     print(key, CB_dict[key])

In [ ]:
# for key in sorted(CB_E_err_dict.keys()):
#     print(key, CB_E_err_dict[key])

## Theoretical Predictions 

#### SAID

###### Read theo: csv $\rightarrow$ dict

In [15]:
#cos bins for SAID
cos_bins_theo   = [(i)/10 for i in range(-10, 11, 1)]

#Read theoretical data from csv (dictionary is saved)
df_theo=pd.read_csv("./theo_cbelsa/theo_dict", index_col=0, decimal=",", header=None)

#Change dtype to float
for i in df_theo.columns:
    df_theo[i] = df_theo[i].astype(float)

#Save theo into dictionary
theo_dict = df_theo.transpose().to_dict(orient="list")

###### Convert SAID raw format to dict 

In [ ]:
# #Format Conversion: raw txt -> dictionary
# content = []
# theo_dict = {}

# with open("./theo_cbelsa/SAID_E_theo_raw.txt") as f:
#     lines = f.readlines()

# #Strip - remove leading/trailing white spaces
# for x in lines:
#     content.append(x.strip())
    
# for i in content:
#     if (i == ''):
#         content.remove(i)
        
# for i in range(len(content)):
#     if (content[i][0]=="E"):
#         E_list = []
#         for j in range(1, 22):
#             E_list.append(content[i + j])
        
#         theo_dict[content[i]] = E_list

###### Save theo: dict $\rightarrow$ csv 

In [ ]:
# #Save theo_dict into csv
# with open("./theo_cbelsa/theo_dict", "w") as f:
#     writer = csv.writer(f, delimiter=",")
    
#     for key in sorted(theo_dict.keys()):
#         writer.writerow([key] + theo_dict[key])

#### MAID

###### Read theo: csv $\rightarrow$ dict

In [16]:
#Read theoretical data from csv (dictionary is saved)
df_theo=pd.read_csv("./theo_cbelsa/theo_dict_MAID", index_col=0, decimal=",", header=None)

#Change dtype to float
for i in df_theo.columns:
    df_theo[i] = df_theo[i].astype(float)

#Save theo into dictionary
theo_dict_MAID = df_theo.transpose().to_dict(orient="list")

###### Convert MAID raw format to dict 

In [ ]:
# #Format Conversion: raw txt -> dictionary
# content = []
# theo_dict_MAID = {}

# with open("./theo_cbelsa/MAID_E_theo_raw.txt") as f:
#     lines = f.readlines()

# #Strip - remove leading/trailing white spaces
# for x in lines:
#     content.append(x.strip())
    
# for i in content:
#     if (i == ''):
#         content.remove(i)
        
# for i in range(len(content)):
#     if (content[i][0]=="E"):
#         E_list = []
#         for j in range(1, 22):
#             E_list.append(content[i + j])
        
#         theo_dict_MAID[content[i]] = E_list

###### Save theo: dict $\rightarrow$ csv 

In [ ]:
# #Save theo_dict_MAID into csv
# with open("./theo_cbelsa/theo_dict_MAID", "w") as f:
#     writer = csv.writer(f, delimiter=",")
    
#     for key in sorted(theo_dict_MAID.keys()):
#         writer.writerow([key] + theo_dict_MAID[key])

#### BNGA

###### Read theo: csv $\rightarrow$ dict

In [17]:
#Read theoretical data from csv (dictionary is saved)
df_theo=pd.read_csv("./theo_cbelsa/theo_dict_BNGA", index_col=0, decimal=",", header=None)

#Change dtype to float
for i in df_theo.columns:
    df_theo[i] = df_theo[i].astype(float)

#Save theo into dictionary
theo_dict_BNGA = df_theo.transpose().to_dict(orient="list")

###### Convert BNGA raw format to dict 

In [ ]:
# #Format Conversion: raw txt -> dictionary
# content = []
# theo_dict_BNGA = {}

# with open("./theo_cbelsa/BNGA_E_theo_raw.txt") as f:
#     lines = f.readlines()

# #Strip - remove leading/trailing white spaces
# for x in lines:
#     content.append(x.strip())
    
# for i in content:
#     if (i == ''):
#         content.remove(i)
        
# for i in range(len(content)):
#     if (content[i][0]=="E"):
#         E_list = []
#         for j in range(1, 22):
#             E_list.append(content[i + j])
        
#         theo_dict_BNGA[content[i]] = E_list

###### Save theo: dict $\rightarrow$ csv 

In [ ]:
# #Save theo_dict_BNGA into csv
# with open("./theo_cbelsa/theo_dict_BNGA", "w") as f:
#     writer = csv.writer(f, delimiter=",")
    
#     for key in sorted(theo_dict_BNGA.keys()):
#         writer.writerow([key] + theo_dict_BNGA[key])